### IMPORT PACKAGES

In [1]:
"""
    References:  https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
                 https://github.com/keras-team/keras/blob/master/examples
                 http://www.aaai.org/ocs/index.php/AAAI/AAAI15/paper/view/9745
                 https://github.com/airalcorn2/Recurrent-Convolutional-Neural-Network-Text-Classifier/blob/master/recurrent_convolutional_keras.py
"""
import os
import re
import nltk
import shutil
import math
import urllib.request
import numpy as np
import pandas as pd
import os
import string
import keras
import csv
nltk.download('stopwords')
nltk.download('punkt')
from google.colab import drive
from google.colab import files
from keras.models import load_model
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.initializers import Constant
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras import backend
from keras.optimizers import Adamax, Adam, Nadam
from keras.layers import BatchNormalization
from keras.layers import Conv1D, Dense, Input, Lambda, CuDNNLSTM, CuDNNGRU, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPooling1D, MaxPooling1D, Dropout, SpatialDropout1D
from keras.layers.merge import concatenate
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### MOUNT GOOGLE DRIVE AND MOVE FILES TO LOCAL DRIVE IN COLAB

In [2]:
file_paths = ['test.csv', 'valid.csv', 'train.csv', 'glove.6B.100d.txt']
# Connect to Google Drive
drive.mount('/content/gdrive')
base_url = 'gdrive/My Drive/'

for path in file_paths:
    if not os.path.isfile(path):
        try:
            shutil.copy(base_url + "data/" + path, path)
        except:
            os.mkdir(base_url + "data")
            os.mkdir(base_url + "data/valid_prediction")
            os.mkdir(base_url + "data/test_prediction")
            print(path + " Not Found in Google Drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


### DOWNLOAD FILES IF IT IS NOT FOUND THEN COPY FILES TO GOOGLE DRIVE

In [0]:
test_path = "https://firebasestorage.googleapis.com/v0/b/hyungyu415.appspot.com/o/test.csv?alt=media&token=979d05cf-d252-4d15-8ec4-a8ed556c6fe9"
valid_path = "https://firebasestorage.googleapis.com/v0/b/hyungyu415.appspot.com/o/valid.csv?alt=media&token=deb7ef2d-1442-48b1-8e17-153aff0bed94"
train_path = "https://firebasestorage.googleapis.com/v0/b/hyungyu415.appspot.com/o/train.csv?alt=media&token=a2f73b7a-f471-4f4a-a15c-57714a6870b2"
embedding_path = "https://firebasestorage.googleapis.com/v0/b/ryan-blog415.appspot.com/o/glove.6B.100d.txt?alt=media&token=e234f3ae-f47a-45ec-a303-159ea4111fbb"
download_paths = [test_path, valid_path, train_path, embedding_path]
file_link_dict = dict(zip(file_paths, download_paths))
for path in file_link_dict.keys():
    if not os.path.isfile(path):
        filedata = urllib.request.urlopen(file_link_dict[path])
        datatowrite = filedata.read()

        with open(path, 'wb') as f:  
            f.write(datatowrite)
    else:
        print("Already Has %s" % path)

def filter_text(string):
    string = str(string)
    string = re.sub('\[[^]]*\]', '', string)
    string = string.replace('\n', ' ')
    return string

file_names = ['valid.csv', 'test.csv', 'train.csv']
for file_name in file_names:
    df = pd.read_csv(file_name)
    df['text'] = df['text'].apply(filter_text)
    extra = df['cool'] + df['funny'] + df['useful']
    df['extra'] = extra
    df = df.drop([i for i in df.columns if i not in ['extra', 'stars', 'text', 'review_id']], axis=1)
    df.to_csv(file_name, index=False)
    
for path in file_paths:
    shutil.copy(path, base_url + "data/" + path)

### LOAD EMBEDDING AND PROCESS INPUT FILES

In [3]:
MAX_SEQUENCE_LENGTH = 250
MAX_NUM_WORDS = 30000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 20000/120000

# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join('glove.6B.100d.txt'), encoding='UTF-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

# second, prepare text samples and their labels
print('Processing text dataset')

texts = []  # list of text samples
labels = []  # list of labels
extras = []

files = ['train.csv', 'valid.csv', 'test.csv']

for file_name in files:
    file = pd.read_csv(file_name)
    for i in range(len(file['text'])):
        if math.isnan(file['stars'][i]) or math.isnan(file['extra'][i]):
            continue
        texts.append(str(file['text'][i]))
        labels.append(int(file['stars'][i]))
        extras.append(int(file['extra'][i]))

VALIDATION_SPLIT = 20000/len(texts)

print('Found %s texts.' % len(texts))

# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

_, y_val_raw = train_test_split(labels, test_size=VALIDATION_SPLIT, shuffle=False)
y_val_raw, _ = train_test_split(y_val_raw, test_size=0.5, shuffle=False)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT / 2 * data.shape[0])
num_test_samples = num_validation_samples

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=VALIDATION_SPLIT, shuffle=False)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, shuffle=False)
x_extra, x_val_extra = train_test_split(extras, test_size=VALIDATION_SPLIT, shuffle=False)
x_val_extra, x_test_extra = train_test_split(x_val_extra, test_size=0.5, shuffle=False)

print('Preparing embedding matrix.')
# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Indexing word vectors.
Found 400000 word vectors.
Processing text dataset
Found 120000 texts.
Found 95827 unique tokens.
Shape of data tensor: (120000, 250)
Shape of label tensor: (120000, 6)
Preparing embedding matrix.


### ADD EXTRA FEATURES

In [4]:
x_val_w_extra = x_val
x_w_extra = x_train
x_test_w_extra = x_test
feature_weight = 10
try:
    if check:
        check = check
except:
    check = True

x_val_w_extra = list(x_val_w_extra)
for j in range(feature_weight):
    for i, each in enumerate(x_val_w_extra):
        each = list(each)
        each.append(x_val_extra[i])
        x_val_w_extra[i] = np.array(each)
x_val_w_extra = np.array(x_val_w_extra)

x_w_extra = list(x_w_extra)
for j in range(feature_weight):
    for i, each in enumerate(x_w_extra):
        each = list(each)
        each.append(x_extra[i])
        x_w_extra[i] = np.array(each)
x_w_extra = np.array(x_w_extra)

x_test_w_extra = list(x_test_w_extra)
for j in range(feature_weight):
    for i, each in enumerate(x_test_w_extra):
        each = list(each)
        each.append(x_test_extra[i])
        x_test_w_extra[i] = np.array(each)
x_test_w_extra = np.array(x_test_w_extra)

if check:
    MAX_SEQUENCE_LENGTH = MAX_SEQUENCE_LENGTH + feature_weight
    check = False
x_test_w_extra.shape

(10000, 260)

### LOAD MODEL NAMES FOUND IN GOOGLE DRIVE

In [0]:
drive.mount('/content/gdrive')
base_url = 'gdrive/My Drive/model_1'
# base_url = 'gdrive/My Drive/model/model_250_old'
models = [i for i in os.listdir(base_url) if i.endswith('.h5')]
models.sort()
prediction_list=list()
test_prediction_list=list()
print("count: %d" % len(models))
print(models)
print("Unfinished Models Below 100", end=": ")
print([i for i in range(100) if i not in [int(i.split("model")[1].split(".h5")[0]) for i in models]])

### IF PREDICTIONS ARE NOT FOUND, GENERATE PREDICTIONS AND SAVE IT TO GOOGLE DRIVE

In [0]:
valid_prediction_list = [i for i in os.listdir(base_url + "/valid_prediction") if i.endswith('.csv')]
test_prediction_list = [i for i in os.listdir(base_url + "/test_prediction") if i.endswith('.csv')]
for model_name in models:
    out_name = model_name.split('.')[0] + ".csv"
    model = None
    
    if out_name not in valid_prediction_list:
        if model is None:
            model = load_model(base_url + "/" + model_name)
        print("train: " + out_name, end=", ")
        valid_prediction = model.predict(x_val_w_extra)
        pd.DataFrame(valid_prediction).to_csv(base_url + "/valid_prediction/" + out_name, index=False)

    if out_name not in test_prediction_list:
        if model is None:
            model = load_model(base_url + "/" + model_name)
        print("test: " + out_name, end="")
        test_prediction = model.predict(x_test_w_extra)
        pd.DataFrame(test_prediction).to_csv(base_url + "/test_prediction/" + out_name, index=False)
    print()

### BASED ON PREDICTIONS, FILTER THOSE THAT WERE BELOW EXPECTATION AND SAVE IT TO MEMORY

In [33]:
accuracy_threshold = 0.682

def find_max_index(some_list):
    maximum = -10000
    index = -10000
    for i, v in enumerate(some_list):
        if maximum < v:
            index = i
            maximum = v
    return index

valid_url = base_url + "/valid_prediction"
valid_sets = [i for i in os.listdir(valid_url) if i.endswith('.csv')]
valid_sets.sort()
print("Total Number of Valid Prediction: %d" % len(valid_sets))
valid_prediction_list = list()

test_url = base_url + "/test_prediction"
test_sets = [i for i in os.listdir(test_url) if i.endswith('.csv')]
test_sets.sort()
print("Total Number of Test Prediction: %d" % len(test_sets))
test_prediction_list = list()

print("Models above %.3f accuracy: " % accuracy_threshold)
for i in range(len(valid_sets)):
    df = pd.read_csv(valid_url + "/" + valid_sets[i])
    max_list = list()
    for row in range(len(df)):
        max_list.append(find_max_index([df['0'][row], df['1'][row],
                                      df['2'][row], df['3'][row],
                                      df['4'][row], df['5'][row]]))
    if accuracy_score(y_val_raw, max_list) > accuracy_threshold:
        print(valid_sets[i], end=" ")
        valid_prediction_list.append(max_list)

        df = pd.read_csv(test_url + "/" + valid_sets[i])
        max_list = list()
        for row in range(len(df)):
            max_list.append(find_max_index([df['0'][row], df['1'][row],
                                          df['2'][row], df['3'][row],
                                          df['4'][row], df['5'][row]]))
        test_prediction_list.append(max_list)
print()
print("Total number of Models used for train: %d" % len(valid_prediction_list))
print("Total number of Models used for test: %d" % len(test_prediction_list))

Total Number of Valid Prediction: 100
Total Number of Test Prediction: 100
Models above 0.682 accuracy: 
model00.csv model05.csv model06.csv model07.csv model09.csv model13.csv model19.csv model20.csv model23.csv model25.csv model26.csv model27.csv model32.csv model34.csv model38.csv model42.csv model43.csv model44.csv model47.csv model55.csv model56.csv model58.csv model59.csv model63.csv model64.csv model67.csv model71.csv model74.csv model75.csv model79.csv model81.csv model82.csv model83.csv model86.csv model87.csv model89.csv model92.csv model96.csv model99.csv 
Total number of Models used for train: 39
Total number of Models used for test: 39


### USE THOSE PREDICTIONS FOR VOTING AND GENERATE FINAL PREDICTION

In [34]:
final_prediction = list()
for i in range(x_val.shape[0]):
    count_list=[0, 0, 0, 0, 0]
    for j in valid_prediction_list:
        count_list[int(j[i] - 1)] = count_list[int(j[i] - 1)] + 1
    max = 0
    index = -1
    for k, j in enumerate(count_list):
        if(count_list[k] >= max):
            max = count_list[k]
            index = k
    final_prediction.append(index + 1)

print(final_prediction)
print("Accuracy: %f" % accuracy_score(y_val_raw, final_prediction))

final_prediction = list()
for i in range(x_val.shape[0]):
    count_list=[0, 0, 0, 0, 0]
    for j in test_prediction_list:
        count_list[int(j[i] - 1)] = count_list[int(j[i] - 1)] + 1
    max = 0
    index = -1
    for k, j in enumerate(count_list):
        if(count_list[k] >= max):
            max = count_list[k]
            index = k
    final_prediction.append(index + 1)

df = pd.read_csv('test.csv')
df = pd.DataFrame(df['review_id'])
df['stars'] = np.array(final_prediction)
df.to_csv('sub.csv', index=False)

[5, 1, 5, 5, 4, 5, 4, 4, 5, 4, 5, 3, 3, 5, 5, 5, 2, 5, 3, 5, 3, 5, 2, 1, 5, 5, 1, 4, 5, 5, 3, 3, 4, 1, 5, 5, 4, 5, 5, 1, 5, 4, 5, 5, 5, 5, 1, 5, 3, 4, 5, 5, 5, 5, 5, 4, 4, 1, 5, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 3, 1, 1, 4, 1, 4, 3, 1, 5, 5, 2, 5, 5, 1, 5, 5, 5, 1, 4, 3, 1, 3, 5, 5, 5, 2, 5, 1, 3, 5, 5, 5, 5, 1, 5, 5, 5, 4, 5, 5, 3, 4, 1, 5, 5, 5, 5, 4, 5, 4, 5, 1, 4, 5, 2, 5, 2, 4, 5, 4, 3, 1, 5, 3, 1, 5, 5, 4, 5, 1, 5, 4, 4, 3, 1, 5, 4, 1, 4, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 5, 4, 1, 1, 5, 5, 5, 4, 4, 4, 5, 3, 4, 1, 5, 1, 4, 5, 1, 4, 5, 1, 4, 3, 5, 4, 1, 4, 1, 5, 1, 1, 3, 1, 4, 4, 5, 5, 2, 5, 1, 3, 4, 1, 1, 5, 4, 3, 3, 5, 5, 1, 3, 1, 1, 4, 5, 4, 5, 1, 5, 3, 1, 3, 3, 1, 5, 5, 5, 2, 2, 1, 5, 5, 5, 5, 1, 5, 3, 5, 4, 2, 5, 1, 2, 5, 4, 5, 4, 3, 5, 5, 5, 5, 5, 5, 5, 1, 1, 5, 2, 4, 5, 1, 5, 5, 4, 2, 4, 4, 1, 2, 5, 3, 4, 2, 5, 5, 1, 3, 5, 1, 5, 2, 4, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 1, 5, 5, 5, 5, 5, 5, 3, 1, 5, 5, 5, 1, 3, 5, 2, 4, 3, 2, 5, 1, 2, 1, 5, 5, 5, 1, 5, 3, 1, 5, 5, 4, 4, 

In [46]:
def find_max_index(some_list):
    maximum = -1000000.0
    index = -10000000
    for i, v in enumerate(some_list):
        if maximum < v:
            index = i
            maximum = v
    return index

max_list = None
base_url2 = "gdrive/My Drive/model_1"
valid_url2 = base_url2 + "/valid_prediction"

base_url = base_url2

valid_url = base_url + "/valid_prediction"
valid_sets = [i for i in os.listdir(valid_url) if i.endswith('.csv')]
valid_sets.sort()
test_url = base_url + "/test_prediction"
test_sets = [i for i in os.listdir(test_url) if i.endswith('.csv')]
test_sets.sort()

limit = 100
valid_sets = valid_sets[:limit]
test_sets = test_sets[:limit]

# import random
# rand_list = list()
# for i in range(100):
#     rand_list.append(random.randint(0, 99))
# rand_list = set(rand_list)
# print(len(rand_list))
# print(rand_list)
# valid_sets = [valid_sets[i] for i in rand_list]
# test_sets = [test_sets[i] for i in rand_list]

for i in range(len(valid_sets)):
    df = pd.read_csv(valid_url + "/" + valid_sets[i])
    if max_list is None:
        max_list = df
    max_list = max_list + df
    
final_prediction = list()
df = max_list

for row in range(len(df)):
    final_prediction.append(find_max_index([df['0'][row], df['1'][row],
                                  df['2'][row], df['3'][row],
                                  df['4'][row], df['5'][row]]))

print(final_prediction)
print("Accuracy: %f" % accuracy_score(y_val_raw, final_prediction))

max_list = None

for i in range(len(test_sets)):
    df = pd.read_csv(test_url + "/" + test_sets[i])
    if max_list is None:
        max_list = df
    max_list = max_list + df
    
final_prediction = list()
df = max_list
# df.to_csv(base_url2 + "/test_prediction/model00.csv", index=False)
for row in range(len(df)):
    final_prediction.append(find_max_index([df['0'][row], df['1'][row],
                                  df['2'][row], df['3'][row],
                                  df['4'][row], df['5'][row]]))
df = pd.read_csv('test.csv')
df = pd.DataFrame(df['review_id'])
df['stars'] = np.array(final_prediction)
df = df.drop([i for i in df.columns if i not in ['review_id', 'stars']], axis=1)
df.to_csv('sub.csv', index=False)
print(final_prediction)
print()

[5, 1, 5, 5, 4, 5, 4, 4, 5, 4, 5, 3, 4, 5, 5, 5, 2, 5, 3, 5, 3, 5, 2, 1, 5, 5, 1, 4, 5, 5, 3, 3, 4, 1, 5, 5, 4, 5, 5, 1, 5, 4, 5, 5, 5, 5, 1, 5, 3, 4, 5, 5, 5, 5, 5, 4, 4, 1, 5, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 4, 5, 3, 1, 1, 4, 1, 4, 3, 1, 5, 5, 3, 5, 5, 1, 5, 5, 5, 1, 4, 3, 1, 3, 5, 5, 5, 2, 5, 1, 3, 5, 5, 5, 5, 1, 5, 5, 5, 4, 5, 5, 3, 4, 1, 5, 5, 5, 5, 4, 5, 4, 5, 1, 4, 5, 2, 5, 2, 4, 5, 4, 3, 1, 5, 3, 1, 5, 5, 4, 5, 1, 5, 3, 4, 3, 1, 5, 4, 1, 4, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 5, 4, 1, 1, 5, 5, 5, 4, 4, 4, 5, 3, 4, 1, 5, 1, 4, 5, 1, 4, 4, 1, 4, 3, 5, 4, 1, 3, 1, 5, 1, 1, 3, 1, 4, 4, 5, 5, 2, 5, 1, 3, 4, 1, 1, 5, 4, 3, 3, 5, 5, 1, 3, 3, 1, 4, 5, 4, 5, 1, 5, 3, 1, 3, 2, 1, 5, 5, 5, 2, 2, 1, 5, 5, 5, 5, 1, 5, 3, 5, 4, 2, 5, 1, 2, 5, 4, 5, 4, 3, 5, 5, 5, 5, 5, 5, 5, 1, 1, 5, 2, 4, 5, 1, 5, 5, 4, 2, 4, 4, 1, 2, 5, 3, 4, 2, 5, 5, 1, 4, 5, 1, 5, 2, 4, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 1, 5, 5, 5, 5, 5, 5, 3, 1, 5, 5, 5, 1, 3, 5, 3, 4, 3, 2, 5, 1, 2, 1, 5, 5, 5, 1, 4, 3, 1, 5, 5, 4, 4, 

In [45]:
# Accuracy: 0.716600
list_best=[2, 5, 5, 5, 3, 5, 5, 5, 5, 4, 2, 5, 3, 5, 5, 5, 5, 5, 3, 1, 3, 2, 5, 1, 1, 5, 5, 5, 5, 1, 1, 5, 5, 5, 3, 1, 5, 5, 2, 5, 1, 1, 2, 1, 4, 4, 1, 1, 5, 4, 5, 1, 4, 5, 5, 5, 5, 5, 1, 5, 2, 3, 3, 3, 5, 5, 2, 1, 3, 5, 5, 5, 5, 4, 3, 2, 5, 1, 3, 5, 5, 5, 3, 5, 1, 4, 4, 4, 4, 5, 4, 1, 5, 4, 5, 5, 3, 5, 4, 5, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 1, 3, 1, 5, 5, 5, 5, 1, 4, 5, 1, 4, 5, 5, 4, 5, 5, 3, 4, 4, 5, 4, 5, 5, 4, 2, 5, 1, 4, 5, 5, 4, 5, 1, 5, 5, 5, 5, 3, 5, 5, 3, 5, 5, 5, 5, 5, 4, 5, 1, 5, 3, 1, 3, 5, 1, 1, 1, 4, 4, 4, 5, 5, 5, 4, 5, 5, 4, 4, 5, 1, 5, 5, 1, 5, 5, 5, 1, 1, 4, 2, 5, 5, 4, 4, 5, 1, 5, 3, 5, 5, 5, 2, 5, 2, 4, 1, 5, 5, 5, 4, 4, 5, 5, 5, 5, 5, 5, 4, 4, 5, 4, 5, 1, 4, 4, 5, 1, 3, 5, 5, 5, 5, 5, 5, 1, 4, 5, 5, 5, 1, 1, 2, 4, 1, 4, 3, 2, 1, 5, 3, 5, 5, 4, 5, 3, 4, 1, 1, 1, 4, 2, 5, 5, 5, 4, 3, 5, 1, 1, 5, 5, 5, 4, 2, 5, 5, 5, 4, 5, 5, 1, 5, 5, 2, 5, 4, 5, 1, 3, 5, 2, 5, 5, 2, 1, 1, 4, 4, 5, 5, 3, 5, 4, 5, 5, 2, 5, 2, 2, 1, 5, 3, 5, 4, 5, 5, 2, 2, 5, 5, 5, 3, 5, 5, 5, 1, 4, 5, 5, 5, 4, 3, 4, 1, 4, 4, 4, 3, 3, 4, 5, 5, 5, 1, 4, 5, 2, 5, 5, 4, 1, 5, 5, 5, 5, 3, 5, 1, 4, 3, 1, 4, 1, 5, 5, 4, 4, 1, 3, 5, 3, 1, 5, 1, 5, 5, 1, 1, 5, 5, 1, 4, 5, 2, 4, 5, 5, 2, 5, 4, 4, 5, 5, 4, 5, 5, 4, 3, 5, 5, 5, 4, 1, 4, 1, 1, 5, 1, 4, 1, 5, 1, 1, 4, 5, 3, 3, 5, 5, 5, 5, 4, 5, 2, 5, 3, 5, 5, 4, 4, 5, 5, 1, 4, 3, 1, 5, 4, 5, 5, 1, 5, 5, 5, 4, 2, 2, 5, 4, 4, 3, 4, 3, 5, 5, 4, 5, 3, 4, 1, 1, 1, 5, 4, 5, 5, 5, 4, 1, 5, 4, 5, 4, 5, 4, 4, 5, 5, 4, 5, 3, 5, 4, 1, 2, 5, 5, 5, 3, 4, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 1, 1, 5, 5, 3, 5, 1, 5, 5, 4, 5, 4, 1, 5, 5, 5, 5, 2, 1, 4, 5, 3, 5, 1, 5, 1, 5, 5, 5, 2, 5, 5, 2, 5, 5, 5, 5, 4, 5, 5, 4, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 1, 5, 5, 1, 4, 3, 5, 4, 4, 5, 5, 4, 4, 4, 4, 5, 2, 5, 5, 1, 5, 5, 5, 5, 1, 2, 5, 4, 4, 5, 5, 5, 5, 5, 2, 5, 3, 5, 1, 4, 5, 2, 1, 5, 4, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5, 5, 4, 3, 3, 5, 4, 1, 1, 5, 1, 5, 3, 4, 4, 2, 5, 5, 2, 4, 5, 3, 5, 5, 5, 3, 5, 5, 4, 5, 5, 5, 5, 1, 5, 1, 5, 2, 5, 4, 5, 4, 5, 5, 4, 5, 5, 1, 4, 3, 4, 5, 4, 5, 1, 5, 5, 1, 5, 4, 5, 1, 4, 2, 5, 1, 1, 5, 4, 1, 5, 5, 3, 3, 4, 1, 3, 5, 4, 4, 5, 5, 5, 3, 1, 5, 4, 5, 5, 4, 4, 5, 4, 5, 5, 1, 5, 4, 5, 1, 5, 5, 5, 3, 1, 5, 2, 4, 5, 5, 4, 1, 5, 4, 4, 5, 4, 5, 5, 2, 5, 5, 4, 5, 5, 5, 1, 2, 4, 2, 2, 3, 3, 5, 5, 3, 5, 3, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5, 2, 1, 4, 5, 5, 4, 4, 5, 5, 1, 5, 5, 1, 5, 2, 5, 2, 5, 5, 5, 4, 5, 5, 2, 2, 5, 3, 1, 1, 5, 2, 5, 1, 4, 1, 5, 4, 5, 5, 5, 5, 4, 3, 4, 5, 4, 5, 5, 4, 5, 3, 5, 4, 1, 5, 1, 4, 4, 5, 3, 1, 5, 5, 5, 3, 3, 5, 5, 2, 5, 1, 4, 4, 5, 5, 4, 5, 3, 1, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 5, 5, 4, 1, 5, 4, 3, 5, 5, 4, 5, 4, 5, 5, 2, 5, 1, 5, 5, 5, 5, 1, 3, 5, 5, 4, 1, 1, 5, 5, 5, 4, 5, 5, 1, 5, 5, 5, 5, 4, 5, 1, 5, 4, 5, 5, 5, 4, 2, 5, 2, 1, 2, 5, 1, 5, 5, 5, 3, 5, 4, 5, 3, 5, 1, 3, 5, 5, 1, 1, 4, 5, 5, 5, 1, 1, 3, 5, 5, 2, 5, 3, 3, 3, 5, 5, 1, 5, 5, 5, 3, 4, 5, 4, 3, 5, 2, 2, 3, 5, 2, 3, 1, 3, 5, 1, 4, 4, 1, 2, 3, 5, 4, 4, 5, 4, 5, 4, 5, 4, 3, 5, 4, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 4, 2, 5, 1, 5, 1, 3, 5, 4, 4, 1, 1, 1, 5, 5, 5, 4, 5, 4, 5, 5, 5, 5, 1, 5, 1, 5, 5, 5, 3, 5, 5, 4, 1, 5, 5, 5, 1, 5, 5, 5, 5, 5, 4, 4, 1, 5, 5, 1, 3, 1, 5, 4, 4, 5, 5, 2, 5, 5, 5, 3, 5, 4, 4, 4, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 1, 1, 5, 2, 5, 1, 2, 5, 5, 3, 5, 1, 4, 4, 5, 5, 4, 5, 4, 5, 4, 5, 1, 3, 2, 1, 3, 5, 1, 3, 5, 5, 3, 5, 4, 5, 5, 5, 3, 5, 4, 5, 5, 5, 5, 5, 2, 4, 5, 4, 5, 5, 5, 4, 5, 2, 5, 5, 5, 2, 5, 5, 4, 1, 5, 4, 4, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5, 1, 2, 2, 5, 5, 3, 5, 1, 5, 5, 5, 2, 5, 1, 3, 5, 5, 1, 1, 5, 5, 4, 5, 5, 4, 5, 4, 2, 4, 5, 3, 5, 5, 5, 5, 4, 4, 3, 3, 4, 4, 5, 5, 1, 1, 5, 5, 1, 5, 5, 5, 5, 5, 4, 5, 5, 1, 4, 2, 5, 1, 1, 5, 4, 5, 1, 2, 1, 2, 4, 3, 4, 5, 5, 2, 1, 5, 5, 5, 1, 5, 4, 5, 5, 5, 5, 3, 1, 4, 1, 5, 5, 1, 4, 1, 1, 3, 4, 5, 5, 4, 5, 4, 1, 5, 5, 5, 5, 3, 5, 1, 5, 3, 1, 4, 5, 5, 5, 1, 4, 3, 5, 5, 5, 3, 1, 5, 2, 5, 5, 5, 4, 4, 2, 4, 2, 5, 5, 1, 4, 1, 4, 4, 5, 5, 5, 1, 4, 5, 3, 5, 5, 5, 5, 5, 5, 5, 4, 4, 5, 3, 3, 5, 1, 4, 5, 5, 5, 1, 5, 5, 1, 5, 2, 1, 3, 4, 3, 2, 5, 5, 1, 5, 4, 5, 2, 5, 2, 3, 1, 4, 5, 5, 5, 1, 3, 4, 1, 3, 5, 5, 4, 2, 5, 5, 3, 5, 1, 4, 5, 5, 4, 5, 5, 5, 5, 4, 5, 4, 5, 4, 1, 5, 1, 5, 5, 5, 5, 5, 4, 4, 5, 5, 5, 1, 4, 4, 4, 1, 4, 1, 5, 4, 3, 3, 1, 3, 1, 4, 1, 5, 5, 2, 1, 1, 1, 3, 1, 5, 1, 5, 5, 5, 4, 5, 4, 5, 5, 5, 4, 5, 3, 3, 1, 5, 5, 5, 5, 5, 5, 3, 5, 3, 5, 4, 4, 4, 5, 5, 1, 4, 1, 1, 5, 4, 5, 3, 5, 5, 5, 1, 5, 5, 5, 1, 1, 5, 3, 5, 1, 5, 5, 5, 5, 5, 1, 2, 4, 4, 5, 1, 4, 5, 4, 4, 4, 5, 5, 3, 5, 5, 1, 5, 5, 4, 2, 4, 5, 1, 5, 5, 3, 1, 5, 1, 4, 3, 1, 4, 5, 1, 5, 3, 2, 5, 5, 2, 5, 5, 5, 5, 5, 1, 4, 5, 3, 2, 5, 2, 5, 5, 5, 5, 4, 5, 5, 5, 1, 5, 3, 1, 5, 4, 5, 3, 1, 4, 5, 5, 5, 1, 1, 3, 4, 1, 5, 5, 2, 5, 5, 3, 1, 1, 4, 5, 4, 4, 5, 5, 2, 1, 5, 1, 1, 3, 3, 5, 5, 1, 5, 5, 5, 5, 5, 4, 5, 4, 5, 1, 5, 4, 5, 5, 5, 5, 3, 5, 3, 1, 4, 5, 4, 4, 1, 5, 5, 5, 4, 3, 1, 4, 5, 5, 5, 1, 3, 1, 5, 3, 3, 5, 4, 5, 5, 5, 5, 4, 3, 5, 5, 5, 5, 1, 5, 3, 4, 5, 5, 5, 4, 5, 1, 5, 4, 5, 5, 4, 5, 4, 2, 4, 5, 5, 4, 4, 5, 1, 3, 5, 5, 5, 5, 5, 4, 3, 5, 4, 5, 1, 4, 4, 4, 1, 4, 1, 5, 4, 4, 4, 4, 3, 5, 5, 5, 5, 5, 1, 5, 5, 2, 4, 5, 1, 5, 1, 5, 2, 4, 5, 4, 5, 5, 5, 4, 1, 4, 4, 3, 4, 3, 2, 5, 5, 5, 5, 4, 4, 5, 4, 5, 1, 4, 5, 5, 5, 2, 2, 5, 5, 1, 3, 4, 2, 5, 5, 1, 2, 4, 5, 5, 1, 5, 5, 2, 5, 5, 3, 4, 5, 1, 4, 5, 4, 5, 5, 1, 4, 5, 4, 5, 5, 3, 1, 4, 2, 1, 5, 5, 3, 4, 1, 5, 5, 4, 4, 3, 4, 5, 3, 5, 5, 5, 1, 5, 5, 3, 5, 5, 1, 1, 3, 5, 1, 1, 4, 3, 5, 4, 2, 4, 2, 3, 5, 4, 4, 5, 4, 4, 2, 5, 5, 5, 1, 5, 4, 5, 1, 5, 5, 5, 4, 5, 5, 5, 4, 5, 2, 5, 1, 4, 5, 1, 4, 1, 5, 4, 5, 5, 4, 5, 5, 3, 2, 5, 5, 3, 5, 2, 1, 4, 1, 5, 1, 4, 5, 4, 4, 5, 4, 1, 4, 5, 1, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 4, 3, 5, 3, 4, 5, 4, 4, 5, 5, 4, 3, 2, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 2, 5, 2, 1, 4, 1, 1, 5, 4, 5, 5, 4, 5, 1, 5, 1, 5, 5, 5, 3, 5, 5, 1, 5, 4, 3, 5, 5, 3, 4, 5, 2, 5, 5, 4, 5, 1, 5, 1, 5, 5, 5, 4, 4, 5, 3, 4, 5, 5, 1, 5, 5, 5, 5, 1, 1, 1, 3, 4, 4, 4, 5, 2, 5, 3, 1, 4, 3, 4, 4, 2, 5, 5, 5, 5, 1, 5, 2, 4, 5, 5, 3, 5, 3, 5, 4, 1, 4, 3, 1, 3, 5, 1, 5, 5, 4, 4, 4, 1, 5, 4, 1, 5, 3, 5, 5, 1, 1, 4, 5, 4, 5, 1, 5, 5, 4, 2, 5, 5, 2, 5, 5, 4, 1, 4, 4, 4, 5, 4, 3, 5, 4, 2, 5, 4, 3, 4, 1, 5, 5, 5, 5, 5, 5, 5, 2, 4, 5, 1, 2, 4, 4, 4, 4, 5, 4, 5, 5, 5, 5, 1, 5, 5, 5, 3, 5, 5, 5, 4, 4, 2, 4, 5, 1, 2, 5, 5, 1, 5, 5, 4, 5, 5, 5, 1, 5, 4, 1, 5, 4, 1, 3, 4, 4, 3, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 1, 4, 5, 5, 3, 5, 5, 1, 5, 5, 5, 3, 5, 3, 5, 5, 3, 5, 5, 5, 5, 5, 1, 4, 3, 5, 5, 5, 5, 4, 5, 5, 5, 3, 5, 5, 1, 5, 3, 5, 5, 5, 5, 3, 4, 5, 1, 3, 5, 1, 5, 5, 4, 1, 5, 5, 5, 1, 5, 5, 1, 5, 5, 1, 5, 5, 3, 4, 5, 3, 5, 4, 4, 1, 5, 3, 5, 5, 5, 4, 4, 2, 3, 5, 5, 4, 2, 4, 1, 3, 4, 2, 1, 1, 1, 1, 4, 4, 4, 1, 4, 5, 5, 2, 3, 3, 4, 5, 5, 5, 1, 5, 1, 5, 5, 4, 1, 5, 4, 4, 1, 5, 1, 4, 1, 3, 1, 4, 3, 5, 5, 5, 5, 5, 5, 1, 5, 4, 3, 4, 5, 5, 1, 4, 4, 1, 1, 4, 4, 5, 5, 4, 3, 5, 4, 5, 1, 4, 5, 1, 4, 3, 5, 5, 5, 2, 5, 1, 2, 1, 4, 1, 3, 5, 1, 3, 5, 5, 4, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 4, 5, 4, 1, 4, 5, 1, 2, 4, 4, 5, 5, 5, 3, 5, 5, 5, 5, 2, 5, 5, 5, 1, 4, 5, 5, 4, 2, 5, 5, 1, 5, 4, 5, 5, 1, 1, 4, 5, 2, 5, 5, 4, 4, 5, 1, 5, 5, 5, 5, 5, 5, 3, 5, 5, 1, 1, 1, 5, 5, 2, 5, 5, 4, 4, 1, 5, 5, 5, 5, 5, 3, 5, 5, 5, 4, 5, 5, 5, 3, 5, 5, 5, 5, 5, 2, 5, 5, 3, 5, 5, 3, 4, 5, 4, 5, 1, 5, 4, 5, 1, 4, 1, 5, 5, 5, 5, 1, 3, 4, 5, 2, 1, 5, 5, 5, 2, 5, 1, 2, 2, 3, 1, 5, 4, 4, 1, 5, 5, 5, 5, 1, 5, 5, 4, 3, 5, 2, 5, 2, 5, 5, 4, 2, 2, 4, 1, 1, 5, 4, 3, 2, 5, 1, 4, 2, 5, 5, 4, 4, 4, 1, 5, 4, 5, 1, 5, 1, 1, 5, 5, 5, 4, 5, 4, 5, 5, 1, 4, 5, 1, 5, 5, 1, 1, 5, 4, 1, 5, 4, 1, 1, 5, 5, 4, 5, 4, 1, 5, 4, 5, 3, 4, 5, 5, 4, 2, 5, 4, 4, 5, 5, 3, 1, 5, 5, 3, 5, 4, 5, 3, 4, 5, 5, 5, 1, 5, 1, 5, 1, 3, 4, 5, 4, 4, 5, 1, 1, 1, 1, 4, 5, 1, 5, 4, 5, 1, 5, 3, 1, 1, 5, 5, 5, 5, 3, 4, 5, 1, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 4, 4, 5, 5, 5, 1, 5, 5, 3, 1, 4, 5, 4, 4, 4, 5, 5, 2, 5, 4, 5, 5, 1, 5, 2, 1, 5, 4, 4, 1, 1, 1, 5, 2, 5, 5, 5, 5, 3, 5, 4, 1, 1, 5, 5, 5, 3, 3, 5, 5, 5, 5, 1, 5, 5, 4, 4, 5, 1, 5, 4, 5, 5, 4, 2, 1, 5, 2, 5, 5, 4, 4, 1, 5, 4, 4, 3, 5, 4, 5, 5, 5, 5, 4, 5, 5, 2, 1, 5, 5, 5, 2, 5, 4, 5, 2, 4, 1, 1, 4, 2, 4, 2, 5, 3, 3, 4, 5, 5, 5, 5, 5, 4, 3, 2, 5, 5, 5, 5, 4, 5, 3, 5, 1, 4, 5, 5, 1, 1, 1, 2, 5, 3, 4, 5, 4, 5, 4, 5, 5, 3, 3, 5, 5, 1, 5, 4, 1, 5, 5, 5, 4, 3, 2, 5, 3, 5, 2, 3, 5, 5, 2, 4, 5, 4, 3, 4, 4, 1, 2, 5, 5, 5, 4, 3, 1, 4, 1, 5, 5, 3, 5, 5, 5, 5, 5, 5, 1, 1, 4, 1, 5, 1, 4, 5, 4, 5, 5, 4, 4, 4, 4, 1, 4, 5, 4, 1, 3, 5, 5, 1, 1, 5, 4, 5, 4, 5, 4, 1, 5, 3, 5, 4, 5, 4, 3, 5, 2, 5, 5, 2, 3, 5, 5, 5, 4, 5, 5, 5, 1, 2, 5, 3, 4, 2, 2, 2, 1, 3, 5, 5, 1, 5, 5, 1, 3, 3, 5, 2, 1, 1, 5, 2, 4, 5, 5, 3, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 3, 1, 3, 5, 5, 5, 2, 1, 4, 3, 1, 4, 4, 2, 1, 4, 1, 1, 5, 4, 4, 4, 4, 1, 5, 5, 1, 5, 5, 5, 2, 5, 4, 3, 5, 4, 1, 1, 1, 4, 4, 2, 4, 1, 4, 5, 4, 5, 1, 5, 5, 5, 5, 5, 1, 3, 5, 5, 1, 4, 1, 5, 5, 5, 1, 5, 3, 5, 5, 4, 5, 5, 5, 3, 4, 3, 3, 5, 2, 5, 5, 4, 4, 4, 3, 4, 3, 5, 1, 5, 4, 2, 5, 5, 5, 1, 3, 2, 5, 5, 1, 2, 4, 5, 3, 4, 5, 4, 2, 1, 5, 5, 4, 1, 3, 5, 5, 3, 5, 1, 5, 5, 5, 5, 5, 1, 4, 5, 5, 5, 2, 4, 1, 5, 4, 1, 3, 3, 5, 1, 5, 4, 5, 5, 3, 5, 5, 5, 4, 5, 1, 1, 5, 5, 5, 5, 5, 1, 5, 4, 3, 5, 1, 4, 4, 3, 5, 1, 5, 1, 4, 5, 5, 5, 1, 5, 5, 4, 3, 5, 5, 2, 5, 5, 5, 1, 5, 1, 5, 5, 3, 4, 2, 1, 1, 5, 1, 4, 3, 3, 4, 1, 5, 2, 5, 4, 3, 4, 5, 5, 5, 5, 4, 5, 5, 4, 1, 5, 5, 1, 1, 4, 1, 2, 1, 5, 5, 1, 5, 1, 2, 4, 1, 5, 4, 5, 1, 5, 4, 1, 5, 1, 5, 4, 1, 5, 5, 5, 5, 5, 1, 5, 5, 5, 3, 4, 3, 1, 3, 5, 3, 5, 2, 5, 1, 5, 5, 5, 5, 4, 5, 5, 5, 1, 3, 5, 5, 5, 4, 4, 1, 4, 5, 1, 3, 3, 2, 1, 5, 5, 3, 5, 5, 5, 5, 1, 4, 5, 5, 5, 5, 4, 5, 4, 4, 5, 3, 5, 5, 4, 3, 5, 3, 4, 5, 3, 5, 5, 1, 4, 5, 1, 1, 5, 5, 4, 4, 3, 2, 5, 5, 5, 4, 5, 4, 5, 4, 4, 1, 4, 5, 4, 5, 2, 1, 5, 5, 1, 5, 5, 5, 5, 3, 5, 3, 2, 4, 1, 1, 4, 2, 5, 4, 4, 3, 4, 5, 5, 5, 3, 5, 5, 5, 1, 5, 5, 1, 3, 5, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 4, 4, 1, 5, 5, 3, 1, 2, 5, 5, 4, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 4, 5, 5, 4, 5, 5, 2, 5, 2, 5, 5, 5, 1, 3, 3, 5, 1, 1, 5, 2, 3, 5, 1, 3, 5, 5, 5, 1, 4, 5, 4, 5, 1, 1, 1, 5, 5, 1, 5, 2, 1, 5, 5, 4, 2, 5, 5, 5, 5, 5, 4, 5, 2, 5, 5, 5, 2, 5, 3, 5, 5, 1, 4, 4, 5, 3, 5, 4, 5, 1, 5, 5, 4, 5, 4, 5, 5, 5, 1, 5, 1, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 3, 5, 5, 4, 1, 4, 1, 4, 4, 5, 5, 1, 5, 4, 5, 5, 1, 1, 1, 4, 5, 3, 1, 5, 1, 1, 5, 5, 4, 5, 3, 5, 5, 1, 5, 1, 4, 5, 3, 4, 1, 5, 2, 5, 5, 5, 5, 5, 4, 1, 5, 3, 3, 5, 5, 4, 4, 4, 5, 3, 4, 5, 5, 5, 5, 3, 5, 5, 5, 4, 5, 4, 2, 5, 5, 5, 5, 1, 1, 2, 5, 4, 4, 5, 5, 4, 4, 5, 4, 5, 5, 1, 5, 4, 5, 5, 1, 4, 5, 4, 5, 1, 5, 4, 3, 3, 1, 5, 5, 1, 5, 5, 4, 5, 5, 5, 4, 5, 3, 5, 5, 5, 4, 5, 3, 5, 5, 5, 4, 5, 5, 5, 5, 1, 5, 2, 4, 4, 5, 5, 5, 5, 5, 1, 5, 2, 2, 5, 4, 5, 1, 5, 1, 1, 5, 5, 4, 5, 5, 3, 4, 5, 1, 1, 1, 3, 4, 5, 1, 5, 5, 1, 1, 5, 5, 5, 5, 4, 5, 1, 5, 5, 4, 5, 5, 1, 3, 1, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 1, 1, 1, 5, 4, 5, 5, 3, 5, 3, 5, 4, 5, 1, 5, 4, 5, 5, 1, 5, 5, 4, 1, 1, 4, 5, 5, 5, 3, 4, 2, 5, 4, 3, 4, 5, 5, 4, 2, 4, 2, 5, 5, 5, 5, 4, 4, 5, 3, 5, 4, 5, 5, 3, 5, 5, 2, 5, 5, 5, 4, 1, 5, 1, 5, 5, 4, 5, 1, 5, 1, 5, 4, 5, 4, 3, 5, 5, 3, 4, 3, 1, 5, 4, 1, 4, 4, 5, 5, 5, 1, 5, 5, 1, 1, 2, 1, 4, 5, 1, 2, 5, 5, 1, 2, 1, 1, 5, 2, 5, 5, 5, 4, 5, 5, 3, 5, 4, 4, 4, 3, 4, 2, 3, 5, 5, 5, 5, 5, 5, 1, 5, 4, 3, 4, 3, 1, 5, 5, 5, 3, 5, 4, 5, 5, 2, 3, 3, 3, 5, 1, 4, 5, 3, 1, 4, 4, 4, 1, 5, 5, 4, 5, 5, 5, 5, 3, 5, 5, 2, 4, 5, 4, 4, 5, 4, 5, 4, 5, 1, 5, 1, 5, 5, 4, 5, 5, 5, 4, 4, 4, 5, 4, 4, 5, 1, 1, 1, 5, 5, 1, 5, 4, 5, 5, 4, 1, 5, 5, 5, 5, 4, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 3, 4, 4, 5, 5, 5, 5, 5, 5, 1, 4, 5, 3, 4, 5, 5, 3, 5, 1, 5, 3, 1, 3, 4, 1, 5, 4, 5, 5, 2, 2, 4, 5, 5, 5, 1, 3, 5, 5, 5, 1, 4, 1, 4, 5, 5, 3, 3, 5, 5, 4, 5, 4, 5, 5, 1, 5, 5, 5, 4, 4, 4, 1, 2, 4, 2, 4, 5, 4, 2, 5, 3, 2, 5, 4, 4, 4, 5, 5, 4, 5, 5, 5, 4, 5, 5, 1, 3, 4, 5, 5, 1, 4, 5, 1, 3, 5, 5, 3, 3, 4, 4, 2, 5, 5, 5, 5, 4, 5, 3, 5, 4, 5, 5, 5, 5, 1, 5, 5, 1, 4, 5, 4, 4, 5, 3, 5, 5, 5, 4, 5, 5, 5, 1, 5, 5, 4, 5, 1, 2, 5, 4, 5, 5, 5, 5, 5, 4, 5, 5, 4, 5, 5, 5, 4, 5, 4, 3, 5, 1, 3, 5, 4, 5, 1, 1, 5, 5, 4, 3, 3, 1, 3, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 5, 1, 3, 4, 5, 5, 5, 4, 4, 5, 5, 5, 4, 5, 5, 1, 5, 4, 5, 5, 4, 1, 5, 5, 3, 5, 4, 3, 5, 4, 1, 4, 5, 1, 4, 4, 3, 4, 4, 1, 5, 4, 1, 1, 4, 4, 5, 5, 5, 5, 1, 3, 4, 3, 5, 1, 5, 4, 5, 5, 1, 4, 4, 1, 5, 1, 1, 5, 1, 4, 5, 5, 5, 2, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 4, 1, 1, 4, 5, 3, 1, 1, 4, 4, 1, 5, 4, 4, 5, 5, 5, 5, 1, 5, 4, 5, 1, 5, 5, 1, 1, 4, 5, 4, 3, 2, 3, 3, 5, 5, 5, 1, 4, 3, 5, 5, 1, 2, 5, 4, 5, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 1, 5, 2, 1, 5, 4, 4, 3, 1, 5, 5, 4, 4, 5, 5, 4, 1, 5, 5, 4, 3, 3, 3, 3, 4, 3, 1, 5, 1, 4, 1, 4, 5, 2, 4, 5, 5, 1, 5, 3, 4, 3, 5, 4, 4, 1, 4, 1, 3, 4, 1, 4, 5, 4, 5, 5, 3, 5, 5, 1, 3, 4, 2, 4, 3, 1, 3, 5, 2, 4, 1, 5, 1, 4, 1, 3, 5, 5, 5, 5, 5, 4, 3, 4, 5, 4, 5, 4, 1, 1, 3, 5, 3, 2, 5, 5, 4, 1, 5, 1, 4, 1, 4, 1, 3, 4, 3, 4, 2, 1, 1, 1, 1, 5, 2, 5, 5, 1, 5, 5, 1, 5, 2, 1, 1, 5, 4, 1, 1, 5, 5, 4, 1, 2, 5, 5, 5, 1, 4, 5, 1, 3, 5, 3, 1, 1, 5, 4, 2, 5, 1, 1, 1, 1, 5, 4, 4, 5, 3, 3, 5, 3, 3, 3, 5, 5, 4, 3, 1, 4, 4, 2, 5, 1, 4, 4, 4, 4, 4, 1, 1, 5, 5, 5, 1, 1, 3, 5, 5, 5, 5, 5, 3, 5, 1, 5, 1, 5, 5, 4, 3, 4, 4, 5, 4, 5, 4, 5, 5, 3, 5, 5, 5, 4, 5, 5, 5, 1, 5, 2, 5, 5, 2, 4, 5, 1, 1, 5, 4, 3, 5, 5, 4, 5, 5, 1, 3, 1, 4, 5, 1, 3, 3, 5, 5, 1, 5, 5, 5, 5, 1, 4, 4, 4, 5, 1, 5, 1, 4, 1, 5, 3, 1, 4, 5, 5, 3, 5, 5, 3, 5, 5, 5, 5, 1, 4, 5, 4, 4, 4, 1, 5, 2, 4, 5, 1, 5, 5, 4, 3, 5, 4, 3, 5, 5, 5, 3, 5, 1, 5, 5, 5, 1, 5, 5, 5, 5, 4, 5, 3, 3, 4, 5, 5, 4, 1, 5, 5, 4, 3, 5, 5, 5, 2, 3, 5, 1, 1, 3, 2, 5, 1, 5, 1, 5, 5, 5, 5, 1, 5, 5, 4, 2, 5, 5, 3, 1, 5, 5, 4, 5, 5, 5, 5, 4, 2, 4, 5, 4, 5, 5, 3, 4, 5, 5, 2, 2, 5, 5, 5, 5, 4, 5, 4, 5, 5, 1, 5, 4, 3, 5, 5, 1, 5, 5, 4, 5, 4, 5, 5, 5, 1, 5, 5, 5, 5, 1, 5, 5, 1, 3, 5, 5, 4, 4, 5, 5, 4, 4, 1, 4, 5, 1, 5, 4, 3, 1, 5, 5, 2, 4, 5, 4, 4, 5, 4, 5, 3, 5, 5, 4, 5, 5, 1, 3, 4, 1, 5, 5, 5, 5, 4, 1, 5, 5, 2, 5, 2, 5, 5, 3, 5, 1, 4, 5, 5, 1, 5, 4, 5, 1, 4, 1, 5, 4, 1, 5, 5, 5, 5, 5, 3, 4, 3, 5, 1, 5, 3, 5, 5, 5, 4, 5, 5, 5, 5, 4, 5, 1, 5, 3, 5, 3, 1, 1, 4, 1, 5, 5, 5, 5, 1, 5, 5, 1, 5, 5, 2, 5, 4, 5, 4, 5, 1, 2, 5, 3, 1, 1, 4, 5, 5, 5, 5, 3, 5, 5, 1, 1, 5, 5, 5, 4, 5, 4, 5, 5, 5, 4, 1, 1, 3, 5, 5, 5, 2, 3, 4, 4, 3, 5, 1, 1, 4, 5, 4, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 1, 4, 5, 1, 5, 3, 5, 5, 5, 5, 5, 4, 3, 5, 5, 2, 5, 5, 5, 1, 5, 5, 1, 5, 1, 5, 5, 1, 4, 4, 3, 5, 5, 5, 3, 5, 1, 4, 3, 5, 3, 5, 4, 3, 4, 1, 4, 5, 5, 4, 5, 4, 3, 1, 5, 4, 1, 5, 1, 4, 5, 3, 1, 5, 5, 4, 1, 1, 5, 1, 5, 1, 5, 2, 1, 5, 4, 4, 4, 5, 5, 5, 5, 3, 5, 5, 3, 3, 5, 3, 3, 5, 3, 5, 1, 5, 3, 4, 5, 4, 5, 2, 5, 5, 5, 4, 5, 4, 5, 5, 5, 4, 2, 1, 3, 5, 2, 5, 5, 3, 5, 3, 5, 5, 1, 2, 1, 1, 5, 5, 3, 5, 5, 4, 2, 5, 5, 3, 5, 3, 3, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 4, 5, 5, 5, 2, 5, 5, 1, 5, 5, 5, 1, 3, 5, 4, 3, 2, 3, 1, 5, 5, 1, 5, 4, 5, 5, 5, 1, 5, 1, 3, 5, 4, 1, 2, 5, 5, 4, 4, 3, 5, 1, 4, 3, 5, 3, 5, 2, 5, 4, 5, 5, 2, 1, 5, 4, 1, 1, 5, 5, 3, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 4, 1, 1, 4, 5, 2, 5, 5, 5, 3, 1, 5, 5, 5, 5, 1, 3, 1, 4, 4, 5, 5, 5, 5, 5, 1, 5, 5, 1, 4, 1, 5, 5, 4, 5, 2, 5, 5, 4, 5, 4, 4, 4, 5, 4, 2, 5, 5, 5, 5, 4, 3, 4, 5, 5, 5, 1, 5, 1, 5, 4, 1, 3, 1, 3, 5, 5, 5, 1, 5, 4, 1, 5, 5, 2, 4, 4, 3, 3, 5, 5, 5, 5, 5, 1, 1, 5, 5, 5, 3, 5, 5, 5, 5, 1, 1, 5, 4, 2, 5, 1, 3, 5, 4, 4, 5, 3, 5, 4, 3, 5, 5, 5, 1, 5, 5, 3, 5, 4, 1, 5, 5, 5, 4, 5, 5, 4, 1, 5, 5, 1, 5, 5, 5, 3, 2, 5, 5, 1, 5, 5, 5, 5, 5, 5, 1, 4, 5, 1, 4, 4, 1, 2, 1, 4, 3, 5, 2, 1, 1, 5, 1, 1, 5, 3, 1, 4, 5, 5, 5, 1, 1, 5, 5, 4, 5, 5, 5, 1, 1, 2, 3, 1, 5, 5, 4, 4, 1, 5, 5, 2, 1, 1, 1, 2, 5, 2, 3, 4, 5, 5, 5, 3, 5, 3, 4, 3, 1, 5, 5, 4, 5, 5, 4, 5, 4, 3, 3, 5, 1, 4, 5, 5, 4, 5, 5, 5, 4, 5, 5, 4, 5, 3, 5, 1, 3, 1, 1, 4, 5, 5, 5, 5, 1, 5, 4, 5, 4, 1, 5, 5, 5, 1, 1, 5, 1, 4, 4, 5, 5, 5, 4, 4, 5, 4, 5, 4, 2, 5, 5, 5, 4, 5, 5, 4, 5, 5, 3, 1, 4, 1, 1, 5, 3, 5, 5, 1, 5, 5, 5, 3, 5, 5, 1, 3, 3, 4, 4, 4, 5, 2, 5, 1, 5, 1, 5, 1, 4, 5, 5, 3, 5, 5, 1, 1, 5, 5, 4, 5, 2, 5, 5, 5, 1, 5, 5, 5, 4, 5, 1, 5, 4, 5, 5, 5, 1, 5, 5, 5, 4, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 1, 2, 5, 4, 4, 5, 5, 1, 5, 2, 5, 5, 5, 5, 5, 3, 3, 3, 3, 1, 3, 5, 3, 5, 5, 4, 5, 2, 4, 5, 1, 5, 4, 1, 5, 3, 1, 5, 5, 3, 4, 3, 1, 1, 1, 5, 4, 4, 1, 1, 3, 5, 5, 4, 4, 5, 3, 1, 1, 5, 1, 5, 1, 3, 4, 5, 5, 4, 1, 5, 5, 4, 1, 5, 5, 2, 5, 4, 3, 5, 5, 3, 1, 1, 5, 5, 5, 5, 5, 5, 5, 1, 1, 3, 5, 5, 5, 1, 5, 5, 3, 5, 5, 5, 4, 5, 5, 5, 5, 2, 5, 5, 2, 5, 1, 3, 4, 2, 5, 5, 5, 5, 5, 5, 5, 3, 5, 3, 3, 1, 5, 5, 1, 2, 5, 3, 4, 1, 5, 1, 4, 3, 5, 5, 5, 2, 4, 4, 5, 3, 1, 3, 2, 5, 1, 1, 5, 3, 4, 5, 1, 5, 5, 5, 3, 1, 1, 1, 5, 4, 3, 4, 1, 3, 4, 1, 4, 5, 5, 5, 4, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 4, 1, 1, 5, 3, 4, 3, 1, 5, 5, 5, 4, 2, 1, 4, 4, 1, 3, 3, 3, 5, 5, 4, 4, 4, 1, 4, 3, 4, 3, 5, 5, 5, 1, 5, 5, 4, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 2, 5, 5, 2, 4, 3, 5, 5, 5, 1, 4, 4, 1, 5, 5, 5, 3, 5, 1, 5, 1, 5, 1, 4, 5, 1, 4, 5, 4, 1, 5, 4, 1, 5, 5, 4, 5, 1, 5, 5, 4, 5, 4, 1, 1, 5, 4, 5, 1, 1, 5, 5, 5, 5, 5, 1, 5, 5, 5, 3, 5, 3, 5, 1, 5, 5, 1, 5, 2, 5, 5, 1, 5, 5, 5, 1, 5, 5, 5, 4, 5, 5, 5, 5, 5, 1, 1, 4, 3, 3, 4, 5, 5, 5, 5, 4, 4, 3, 1, 3, 5, 5, 4, 1, 1, 4, 5, 5, 5, 5, 5, 5, 1, 1, 4, 4, 5, 5, 5, 5, 1, 1, 5, 1, 5, 5, 1, 5, 5, 5, 4, 5, 5, 4, 1, 5, 4, 3, 2, 4, 5, 3, 5, 5, 1, 4, 1, 4, 3, 1, 2, 5, 5, 5, 5, 1, 5, 1, 4, 5, 5, 4, 4, 5, 5, 4, 4, 4, 5, 5, 5, 5, 5, 3, 4, 5, 5, 5, 1, 1, 5, 5, 1, 4, 5, 5, 1, 5, 5, 5, 5, 5, 4, 3, 2, 5, 4, 5, 5, 1, 4, 4, 3, 5, 4, 5, 1, 5, 3, 3, 4, 4, 1, 4, 5, 4, 4, 4, 1, 1, 1, 5, 4, 1, 4, 5, 1, 5, 1, 3, 4, 5, 4, 5, 1, 4, 1, 3, 1, 4, 5, 5, 1, 5, 5, 1, 5, 5, 5, 2, 4, 5, 1, 5, 2, 5, 5, 3, 3, 1, 5, 5, 5, 3, 1, 5, 4, 1, 5, 4, 1, 5, 4, 3, 4, 3, 5, 1, 3, 2, 5, 5, 5, 1, 4, 5, 1, 1, 4, 3, 1, 1, 5, 1, 3, 5, 5, 5, 4, 5, 1, 4, 4, 5, 5, 4, 5, 1, 5, 5, 4, 1, 3, 1, 1, 5, 1, 4, 4, 2, 3, 5, 2, 1, 5, 1, 3, 5, 4, 3, 5, 3, 5, 3, 1, 5, 4, 5, 5, 5, 5, 5, 5, 4, 4, 5, 5, 4, 4, 4, 4, 3, 4, 5, 4, 5, 5, 3, 2, 5, 5, 4, 3, 3, 1, 5, 5, 1, 4, 1, 1, 4, 5, 5, 5, 3, 3, 5, 5, 1, 5, 4, 4, 3, 1, 3, 4, 5, 5, 1, 4, 5, 3, 4, 5, 1, 5, 4, 5, 1, 1, 3, 5, 5, 5, 5, 5, 5, 1, 5, 4, 4, 2, 1, 5, 5, 5, 4, 4, 4, 3, 5, 4, 5, 5, 5, 4, 1, 5, 5, 1, 5, 1, 5, 1, 5, 4, 1, 4, 5, 5, 5, 3, 5, 1, 4, 4, 4, 4, 5, 4, 5, 3, 5, 5, 1, 4, 5, 5, 4, 1, 4, 5, 5, 5, 5, 5, 5, 4, 1, 1, 4, 5, 5, 5, 1, 5, 4, 5, 5, 5, 5, 5, 5, 5, 3, 4, 4, 5, 2, 1, 4, 4, 4, 5, 5, 4, 1, 5, 1, 5, 4, 3, 1, 4, 5, 4, 5, 4, 4, 4, 5, 5, 1, 4, 4, 5, 1, 2, 3, 5, 4, 2, 3, 5, 1, 5, 3, 5, 1, 5, 2, 5, 2, 3, 4, 5, 2, 1, 5, 5, 5, 3, 3, 5, 4, 3, 4, 3, 4, 5, 1, 5, 5, 5, 3, 5, 4, 4, 5, 3, 1, 5, 1, 1, 3, 1, 5, 4, 5, 5, 3, 4, 5, 5, 3, 4, 2, 5, 5, 5, 3, 5, 5, 3, 5, 5, 5, 5, 2, 4, 1, 1, 5, 5, 5, 2, 4, 5, 5, 5, 3, 1, 5, 1, 1, 4, 1, 3, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 3, 4, 5, 3, 2, 3, 4, 1, 1, 5, 5, 4, 5, 5, 5, 4, 1, 1, 2, 5, 5, 4, 4, 4, 2, 5, 3, 5, 5, 5, 4, 3, 5, 5, 2, 2, 2, 5, 5, 1, 5, 5, 5, 4, 1, 5, 4, 4, 4, 5, 2, 4, 5, 5, 2, 5, 4, 5, 5, 3, 3, 4, 1, 2, 1, 5, 5, 1, 1, 5, 5, 5, 5, 3, 1, 1, 5, 4, 4, 4, 4, 1, 5, 5, 5, 3, 4, 5, 4, 5, 4, 1, 5, 4, 5, 4, 5, 1, 4, 1, 5, 5, 1, 3, 5, 1, 4, 2, 2, 5, 3, 4, 3, 5, 5, 5, 5, 5, 5, 5, 1, 1, 5, 1, 5, 4, 2, 3, 5, 4, 5, 4, 1, 5, 4, 1, 5, 1, 3, 5, 5, 5, 1, 4, 5, 5, 2, 4, 5, 1, 1, 4, 5, 2, 1, 1, 5, 1, 1, 5, 5, 1, 1, 4, 3, 1, 5, 1, 1, 1, 4, 1, 4, 1, 5, 5, 5, 4, 5, 5, 4, 5, 5, 4, 4, 5, 4, 5, 1, 1, 5, 5, 3, 3, 4, 5, 3, 4, 5, 4, 5, 1, 3, 5, 4, 5, 2, 1, 5, 4, 5, 1, 4, 1, 5, 4, 2, 5, 5, 5, 5, 5, 5, 4, 5, 3, 1, 3, 5, 4, 3, 5, 3, 3, 5, 4, 5, 4, 5, 1, 5, 1, 5, 4, 5, 3, 4, 4, 5, 4, 5, 4, 5, 5, 3, 2, 3, 1, 4, 5, 2, 3, 1, 4, 2, 5, 5, 4, 4, 4, 5, 3, 5, 5, 5, 3, 2, 1, 5, 2, 5, 4, 5, 5, 5, 1, 5, 5, 5, 5, 4, 5, 2, 5, 1, 3, 5, 1, 5, 5, 5, 4, 4, 2, 4, 4, 5, 5, 5, 5, 1, 3, 3, 3, 1, 5, 5, 5, 5, 1, 5, 1, 1, 5, 3, 4, 5, 4, 5, 5, 5, 5, 3, 4, 5, 1, 1, 5, 5, 1, 5, 1, 4, 5, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 3, 1, 5, 5, 5, 5, 5, 4, 4, 1, 5, 3, 2, 4, 4, 5, 3, 4, 4, 5, 5, 5, 3, 5, 1, 4, 5, 4, 2, 5, 3, 3, 1, 5, 3, 5, 1, 5, 5, 2, 4, 2, 5, 5, 1, 5, 5, 4, 1, 5, 4, 5, 5, 1, 5, 4, 2, 1, 1, 1, 4, 5, 5, 1, 4, 5, 4, 5, 1, 5, 5, 5, 1, 1, 1, 5, 3, 5, 1, 1, 5, 5, 4, 4, 5, 4, 5, 1, 5, 1, 5, 5, 1, 1, 1, 2, 5, 2, 5, 4, 4, 2, 3, 5, 5, 5, 5, 1, 4, 5, 4, 1, 4, 1, 1, 1, 4, 5, 4, 5, 5, 2, 2, 3, 5, 4, 4, 1, 5, 4, 5, 5, 3, 5, 2, 1, 4, 4, 5, 5, 3, 5, 4, 1, 5, 4, 5, 5, 5, 4, 5, 5, 3, 4, 5, 5, 5, 1, 1, 3, 5, 2, 4, 5, 5, 2, 1, 5, 5, 1, 5, 5, 4, 5, 4, 5, 1, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 4, 1, 4, 4, 5, 5, 5, 5, 3, 5, 5, 4, 5, 4, 5, 1, 1, 5, 1, 2, 5, 4, 4, 5, 1, 5, 5, 5, 1, 5, 5, 5, 1, 3, 5, 4, 1, 5, 3, 1, 4, 5, 5, 1, 5, 3, 5, 4, 5, 5, 1, 3, 3, 1, 5, 5, 3, 4, 3, 5, 5, 1, 5, 4, 3, 5, 5, 5, 3, 5, 1, 3, 5, 1, 1, 5, 1, 5, 4, 1, 1, 1, 5, 4, 5, 4, 5, 5, 5, 5, 2, 2, 5, 3, 3, 1, 5, 5, 4, 1, 3, 1, 5, 4, 1, 5, 5, 1, 5, 5, 2, 5, 5, 4, 1, 5, 5, 5, 4, 3, 1, 5, 5, 3, 4, 3, 1, 1, 1, 5, 4, 5, 4, 5, 5, 5, 1, 5, 5, 5, 5, 5, 3, 1, 4, 5, 5, 1, 4, 5, 5, 5, 5, 3, 3, 5, 1, 3, 5, 4, 1, 1, 5, 1, 4, 4, 5, 2, 1, 4, 5, 2, 5, 1, 5, 4, 5, 2, 1, 4, 1, 5, 5, 5, 2, 5, 4, 4, 1, 3, 1, 4, 1, 5, 3, 5, 5, 3, 1, 5, 3, 4, 2, 1, 5, 2, 4, 5, 4, 3, 5, 1, 4, 5, 2, 5, 4, 5, 5, 1, 4, 1, 4, 5, 5, 2, 5, 5, 5, 5, 1, 5, 5, 1, 5, 4, 4, 5, 5, 5, 4, 5, 2, 1, 4, 5, 5, 3, 4, 4, 5, 5, 5, 3, 5, 5, 5, 4, 4, 5, 1, 2, 4, 5, 5, 3, 3, 5, 4, 1, 4, 4, 5, 4, 2, 4, 4, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 1, 4, 3, 5, 5, 5, 1, 5, 1, 5, 5, 4, 5, 3, 1, 5, 1, 1, 5, 5, 5, 5, 5, 5, 5, 1, 3, 5, 5, 5, 4, 5, 1, 5, 3, 1, 5, 1, 4, 4, 4, 5, 5, 4, 5, 4, 5, 2, 5, 1, 5, 1, 2, 1, 1, 1, 5, 5, 4, 5, 4, 3, 5, 5, 5, 4, 5, 5, 4, 2, 5, 5, 5, 5, 2, 5, 4, 3, 4, 5, 5, 4, 5, 3, 5, 5, 5, 1, 5, 2, 5, 1, 1, 5, 5, 1, 1, 1, 5, 4, 3, 5, 1, 4, 5, 5, 5, 5, 1, 1, 5, 1, 3, 2, 4, 5, 4, 5, 5, 5, 5, 5, 1, 5, 2, 3, 5, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 1, 5, 2, 4, 5, 5, 4, 2, 3, 3, 4, 5, 5, 1, 1, 4, 5, 5, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 1, 3, 2, 4, 4, 5, 5, 5, 3, 4, 5, 3, 5, 1, 5, 5, 5, 4, 1, 4, 1, 5, 5, 4, 4, 1, 5, 1, 5, 4, 5, 4, 4, 5, 5, 5, 3, 2, 1, 1, 3, 4, 5, 4, 3, 5, 4, 2, 5, 4, 3, 1, 5, 5, 4, 1, 5, 5, 5, 5, 5, 1, 5, 1, 1, 2, 5, 5, 5, 1, 1, 1, 5, 4, 5, 1, 5, 4, 1, 4, 2, 5, 3, 4, 5, 1, 5, 4, 5, 5, 5, 4, 5, 1, 5, 5, 5, 5, 5, 5, 2, 4, 1, 5, 5, 4, 1, 4, 2, 4, 5, 4, 5, 4, 2, 3, 5, 4, 2, 5, 3, 1, 1, 5, 5, 4, 3, 3, 4, 1, 5, 5, 4, 5, 5, 5, 2, 1, 4, 5, 5, 1, 5, 1, 4, 5, 5, 5, 1, 2, 4, 1, 4, 5, 5, 2, 1, 5, 2, 5, 1, 5, 3, 5, 4, 5, 5, 5, 1, 2, 5, 4, 5, 2, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 1, 5, 5, 1, 1, 5, 1, 1, 4, 4, 1, 5, 1, 4, 4, 5, 5, 1, 1, 4, 2, 2, 5, 4, 3, 5, 3, 4, 1, 2, 1, 5, 1, 5, 5, 5, 5, 5, 3, 4, 5, 5, 5, 5, 5, 5, 5, 1, 1, 3, 4, 5, 4, 4, 1, 2, 3, 5, 5, 5, 5, 5, 1, 4, 4, 4, 5, 3, 4, 5, 5, 5, 3, 3, 1, 5, 5, 1, 1, 4, 1, 5, 3, 4, 3, 3, 4, 1, 1, 5, 5, 5, 4, 1, 5, 5, 3, 3, 4, 5, 4, 1, 5, 4, 3, 3, 4, 1, 1, 2, 4, 4, 4, 5, 4, 5, 4, 5, 5, 5, 2, 3, 5, 4, 5, 4, 5, 1, 5, 2, 1, 5, 4, 5, 5, 5, 4, 4, 5, 5, 3, 5, 2, 5, 2, 5, 1, 5, 4, 4, 1, 5, 1, 5, 2, 1, 5, 5, 5, 5, 2, 3, 5, 1, 1, 5, 5, 5, 5, 4, 5, 1, 1, 4, 1, 4, 5, 3, 5, 1, 5, 5, 3, 3, 3, 1, 5, 2, 5, 4, 4, 5, 1, 5, 5, 3, 5, 2, 4, 5, 3, 3, 1, 5, 2, 5, 3, 5, 5, 5, 3, 5, 4, 3, 1, 3, 4, 3, 5, 3, 5, 5, 2, 5, 2, 5, 1, 5, 5, 5, 5, 5, 5, 4, 5, 2, 4, 5, 3, 5, 5, 1, 1, 4, 5, 1, 1, 4, 5, 5, 4, 3, 3, 5, 2, 1, 5, 4, 1, 1, 4, 1, 1, 1, 1, 5, 1, 5, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 1, 3, 5, 3, 5, 1, 5, 5, 4, 5, 4, 5, 4, 1, 5, 1, 5, 1, 2, 4, 2, 5, 3, 1, 4, 1, 4, 4, 3, 1, 5, 5, 5, 1, 3, 4, 5, 5, 5, 4, 5, 5, 5, 5, 5, 4, 5, 4, 4, 5, 2, 5, 5, 4, 5, 1, 1, 5, 3, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 3, 4, 5, 3, 5, 4, 5, 2, 1, 5, 5, 5, 1, 5, 5, 4, 5, 4, 5, 4, 5, 1, 3, 4, 4, 5, 3, 5, 5, 5, 5, 5, 1, 2, 1, 5, 5, 4, 5, 4, 4, 5, 5, 5, 2, 5, 5, 2, 5, 5, 1, 5, 5, 4, 5, 1, 4, 5, 5, 5, 5, 4, 3, 5, 5, 2, 5, 4, 5, 5, 3, 5, 2, 5, 4, 1, 4, 3, 5, 3, 5, 5, 5, 5, 3, 3, 5, 4, 1, 1, 4, 4, 4, 5, 5, 3, 4, 5, 1, 5, 5, 5, 5, 5, 1, 5, 5, 4, 5, 2, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 4, 1, 4, 5, 5, 4, 3, 2, 2, 1, 3, 5, 5, 1, 4, 1, 3, 1, 5, 1, 5, 5, 4, 5, 5, 5, 5, 1, 4, 5, 5, 3, 3, 1, 1, 5, 2, 5, 5, 5, 3, 4, 5, 2, 4, 4, 5, 5, 3, 5, 5, 1, 3, 5, 1, 5, 5, 1, 3, 1, 4, 5, 2, 1, 4, 5, 5, 1, 5, 1, 5, 1, 5, 3, 1, 5, 5, 1, 5, 4, 5, 5, 2, 5, 5, 5, 1, 4, 4, 5, 2, 5, 3, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 3, 1, 5, 2, 4, 5, 5, 4, 5, 3, 5, 1, 5, 5, 2, 4, 3, 5, 5, 5, 4, 4, 5, 4, 1, 5, 5, 2, 4, 3, 5, 5, 4, 5, 4, 5, 1, 1, 5, 3, 4, 5, 4, 3, 5, 5, 1, 3, 5, 4, 5, 5, 4, 2, 5, 5, 5, 2, 1, 1, 4, 5, 3, 5, 1, 4, 1, 3, 5, 5, 5, 2, 2, 3, 4, 5, 3, 4, 3, 5, 1, 4, 3, 4, 5, 5, 1, 5, 3, 4, 5, 5, 5, 1, 5, 4, 3, 3, 4, 4, 5, 4, 1, 5, 2, 4, 4, 3, 4, 4, 5, 1, 5, 3, 5, 5, 5, 5, 5, 4, 4, 3, 5, 4, 4, 5, 5, 1, 5, 2, 3, 4, 5, 4, 5, 4, 5, 3, 2, 5, 5, 1, 5, 5, 2, 5, 5, 4, 5, 5, 4, 5, 5, 4, 1, 4, 5, 5, 1, 4, 1, 3, 1, 5, 1, 5, 5, 4, 3, 5, 5, 1, 5, 4, 5, 5, 4, 4, 1, 5, 5, 5, 4, 5, 1, 5, 5, 5, 5, 1, 4, 3, 3, 5, 5, 5, 4, 4, 4, 5, 1, 5, 3, 3, 4, 5, 5, 3, 5, 2, 5, 1, 4, 5, 1, 5, 4, 5, 1, 5, 3, 4, 3, 5, 4, 4, 3, 5, 5, 5, 5, 5, 5, 5, 4, 5, 1, 4, 1, 1, 5, 4, 5, 1, 4, 5, 5, 5, 5, 2, 5, 5, 5, 1, 1, 5, 5, 5, 2, 2, 5, 1, 4, 5, 5, 1, 5, 1, 1, 4, 5, 5, 1, 3, 5, 3, 2, 1, 5, 1, 5, 5, 5, 1, 4, 5, 5, 5, 4, 5, 5, 1, 3, 5, 5, 4, 5, 2, 4, 5, 4, 5, 5, 1, 5, 5, 1, 4, 5, 5, 5, 5, 5, 1, 5, 5, 5, 4, 5, 5, 5, 4, 4, 5, 1, 5, 2, 3, 2, 4, 1, 4, 4, 3, 5, 1, 1, 5, 4, 4, 3, 4, 5, 4, 5, 5, 5, 5, 3, 5, 5, 5, 5, 4, 4, 5, 1, 4, 5, 5, 5, 5, 2, 1, 4, 5, 4, 2, 5, 5, 1, 5, 5, 1, 5, 1, 5, 5, 5, 5, 5, 5, 4, 1, 5, 1, 5, 5, 1, 5, 5, 3, 5, 1, 4, 5, 5, 3, 4, 1, 5, 3, 4, 5, 4, 5, 1, 1, 5, 4, 5, 5, 5, 3, 5, 2, 4, 3, 3, 1, 1, 1, 5, 3, 4, 5, 5, 1, 5, 1, 5, 4, 5, 5, 4, 1, 1, 1, 5, 4, 5, 5, 5, 2, 2, 4, 5, 5, 3, 4, 5, 4, 5, 1, 3, 5, 4, 1, 4, 5, 4, 4, 5, 5, 5, 1, 1, 5, 5, 5, 1, 4, 5, 3, 5, 5, 1, 4, 4, 5, 5, 5, 1, 4, 4, 5, 5, 5, 1, 5, 5, 1, 1, 5, 4, 4, 1, 5, 5, 5, 5, 3, 5, 2, 1, 5, 5, 4, 1, 3, 4, 5, 1, 4, 5, 3, 1, 1, 2, 5, 1, 5, 1, 5, 1, 2, 5, 3, 3, 5, 1, 5, 4, 3, 4, 5, 2, 4, 4, 3, 5, 4, 2, 5, 5, 1, 4, 5, 5, 5, 4, 5, 5, 5, 1, 1, 2, 5, 4, 5, 4, 4, 4, 4, 2, 3, 5, 5, 4, 5, 2, 4, 5, 1, 2, 5, 5, 1, 5, 5, 2, 5, 5, 2, 4, 5, 5, 1, 5, 5, 4, 5, 5, 4, 4, 4, 5, 3, 5, 4, 3, 1, 4, 1, 5, 5, 5, 1, 1, 1, 5, 4, 5, 4, 1, 4, 5, 5, 4, 4, 5, 5, 5, 5, 5, 3, 5, 1, 3, 5, 1, 5, 1, 5, 4, 5, 5, 4, 5, 5, 5, 1, 5, 5, 1, 5, 5, 1, 5, 1, 1, 5, 5, 5, 5, 3, 5, 3, 1, 5, 5, 4, 2, 5, 3, 2, 5, 5, 3, 1, 1, 5, 5, 1, 1, 5, 4, 3, 4, 4, 5, 5, 3, 4, 1, 5, 5, 5, 4, 5, 5, 5, 5, 3, 3, 4, 2, 4, 4, 5, 4, 5, 1, 1, 4, 4, 1, 4, 4, 3, 5, 4, 4, 5, 1, 4, 2, 1, 5, 5, 5, 5, 1, 1, 5, 2, 1, 5, 5, 4, 3, 1, 5, 3, 5, 3, 4, 4, 5, 5, 5, 5, 2, 5, 5, 4, 1, 5, 5, 5, 5, 4, 5, 5, 1, 4, 1, 5, 5, 2, 5, 5, 4, 4, 5, 5, 5, 5, 5, 5, 3, 1, 4, 4, 1, 2, 5, 5, 1, 5, 4, 5, 5, 1, 1, 5, 4, 5, 3, 5, 5, 5, 2, 5, 5, 5, 4, 5, 5, 5, 4, 3, 5, 4, 5, 4, 5, 2, 1, 5, 2, 1, 4, 4, 4, 3, 5, 4, 1, 5, 5, 5, 5, 1, 1, 5, 2, 3, 4, 2, 2, 4, 5, 5, 5, 4, 3, 1, 5, 5, 5, 4, 4, 5, 5, 5, 1, 4, 5, 4, 5, 4, 5, 5, 5, 4, 3, 4, 4, 1, 5, 4, 5, 2, 1, 3, 4, 3, 5, 1, 4, 5, 5, 3, 1, 5, 1, 5, 4, 1, 5, 3, 5, 5, 1, 4, 5, 1, 3, 4, 4, 4, 4, 5, 5, 5, 4, 5, 5, 5, 5, 1, 5, 1, 4, 5, 5, 5, 5, 5, 3, 1, 3, 5, 5, 4, 3, 5, 1, 4, 5, 3, 1, 4, 1, 5, 1, 5, 5, 3, 1, 1, 4, 4, 3, 5, 4, 5, 5, 4, 5, 5, 1, 5, 2, 5, 5, 1, 2, 4, 3, 4, 5, 1, 4, 5, 5, 2, 5, 5, 4, 5, 4, 5, 4, 5, 1, 5, 5, 5, 1, 4, 1, 5, 5, 5, 3, 3, 2, 5, 2, 5, 5, 5, 4, 5, 5, 5, 3, 5, 1, 5, 1, 3, 4, 5, 5, 3, 4, 5, 1, 5, 4, 5, 4, 4, 5, 4, 5, 3, 3, 5, 1, 5, 5, 3, 5, 4, 5, 1, 4, 5, 4, 5, 5, 5, 4, 5, 5, 3, 5, 4, 1, 5, 5, 4, 5, 4, 5, 4, 2, 2, 5, 5, 4, 5, 3, 1, 5, 5, 4, 5, 3, 5, 1, 4, 4, 3, 4, 4, 1, 5, 5, 5, 1, 3, 1, 5, 4, 5, 4, 5, 3, 4, 4, 5, 5, 5, 4, 2, 5, 4, 4, 2, 1, 3, 5, 3, 5, 5, 1, 3, 4, 5, 5, 1, 5, 4, 3, 5, 5, 4, 4, 4, 4, 1, 5, 5, 5, 3, 4, 1, 1, 1, 1, 1, 1, 5, 5, 4, 1, 1, 3, 1, 2, 4, 5, 1, 4, 5, 5, 4, 5, 5, 3, 5, 5, 2, 5, 1, 5, 5, 3, 5, 1, 5, 1, 5, 4, 5, 5, 4, 5, 5, 4, 5, 3, 4, 5, 2, 1, 5, 4, 5, 4, 5, 3, 5, 5, 4, 4, 5, 5, 5, 5, 3, 5, 1, 5, 5, 5, 1, 1, 5, 5, 4, 5, 4, 5, 5, 5, 5, 4, 1, 5, 5, 5, 1, 2, 4, 5, 4, 3, 3, 4, 5, 4, 5, 5, 5, 5, 2, 5, 5, 1, 2, 4, 3, 5, 5, 5, 1, 1, 5, 5, 4, 5, 5, 5, 3, 1, 1, 1, 5, 5, 3, 3, 5, 1, 5, 1, 5, 2, 4, 3, 5, 1, 5, 1, 4, 5, 5, 5, 4, 1, 5, 1, 5, 4, 5, 5, 3, 5, 5, 1, 2, 5, 5, 3, 5, 5, 3, 4, 4, 4, 5, 4, 1, 1, 5, 5, 5, 5, 4, 4, 5, 5, 4, 5, 4, 3, 5, 5, 5, 5, 5, 1, 5, 5, 3, 5, 1, 4, 4, 5, 4, 5, 5, 5, 1, 5, 5, 5, 1, 1, 4, 5, 4, 5, 4, 1, 5, 5, 4, 4, 5, 5, 4, 3, 5, 1, 1, 1, 3, 5, 3, 5, 3, 5, 5, 5, 1, 3, 3, 2, 5, 3, 4, 5, 5, 4, 1, 1, 5, 5, 5, 3, 5, 1, 3, 5, 4, 1, 5, 5, 5, 5, 4, 4, 4, 4, 5, 4, 1, 5, 5, 5, 2, 3, 1, 5, 4, 3, 1, 4, 5, 2, 5, 3, 5, 4, 5, 5, 2, 3, 1, 5, 2, 5, 5, 2, 4, 1, 1, 5, 1, 5, 3, 5, 5, 1, 3, 1, 4, 5, 3, 5, 1, 5, 4, 1, 4, 5, 1, 3, 5, 5, 2, 3, 5, 5, 3, 5, 1, 5, 5, 1, 4, 1, 4, 2, 1, 2, 5, 5, 5, 1, 1, 1, 5, 5, 5, 4, 5, 5, 4, 5, 1, 5, 4, 5, 2, 1, 5, 4, 5, 5, 1, 1, 5, 5, 1, 1, 4, 3, 1, 1, 1, 1, 3, 5, 3, 5, 4, 5, 1, 4, 1, 5, 2, 5, 4, 3, 3, 1, 3, 4, 4, 4, 1, 5, 1, 5, 1, 4, 5, 3, 3, 3, 5, 4, 1, 1, 5, 5, 5, 1, 5, 4, 1, 2, 5, 4, 4, 1, 5, 2, 4, 4, 5, 5, 1, 5, 5, 5, 5, 5, 5, 1, 4, 4, 5, 5, 4, 5, 5, 5, 3, 5, 2, 5, 5, 5, 4, 1, 2, 4, 3, 1, 1, 1, 5, 1, 5, 4, 4, 1, 4, 1, 1, 5, 1, 4, 2, 4, 5, 5, 5, 4, 5, 5, 4, 5, 1, 4, 5, 3, 5, 5, 4, 4, 2, 3, 3, 5, 4, 5, 1, 5, 5, 1, 5, 3, 2, 3, 1, 4, 1, 4, 5, 3, 5, 5, 1, 5, 4, 1, 1, 3, 3, 5, 5, 4, 4, 5, 5, 5, 5, 1, 3, 5, 5, 3, 3, 4, 5, 5, 4, 1, 5, 5, 1, 4, 5, 3, 3, 1, 5, 3, 2, 5, 3, 2, 1, 3, 4, 5, 2, 1, 1, 4, 1, 2, 1, 1, 5, 5, 4, 3, 5, 2, 5, 5, 1, 2, 4, 1, 5, 4, 4, 5, 5, 3, 2, 5, 1, 4, 5, 5, 5, 5, 1, 5, 5, 1, 5, 1, 2, 1, 5, 5, 1, 2, 5, 1, 4, 4, 5, 5, 2, 1, 5, 5, 4, 1, 5, 3, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 1, 5, 4, 1, 5, 1, 4, 5, 1, 4, 1, 5, 5, 3, 3, 3, 3, 5, 5, 1, 5, 4, 3, 5, 4, 5, 2, 1, 4, 2, 5, 5, 5, 1, 5, 2, 5, 5, 5, 4, 4, 4, 3, 4, 5, 4, 5, 1, 5, 5, 5, 4, 5, 5, 3, 5, 5, 5, 5, 5, 5, 4, 1, 4, 5, 5, 5, 5, 2, 3, 5, 4, 5, 5, 3, 5, 1, 3, 5, 4, 5, 5, 5, 1, 5, 5, 1, 5, 5, 5, 5, 5, 3, 2, 5, 1, 1, 5, 5, 5, 4, 5, 2, 1, 4, 4, 5, 5, 5, 5, 4, 5, 4, 5, 1, 5, 4, 4, 5, 4, 4, 1, 4, 4, 4, 1, 3, 5, 1, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 4, 5, 3, 1, 3, 5, 3, 5, 1, 4, 1, 5, 4, 4, 2, 5, 4, 4, 1, 5, 2, 4, 4, 4, 3, 5, 5, 5, 5, 5, 3, 3, 4, 5, 5, 4, 4, 4, 4, 1, 3, 5, 5, 5, 5, 4, 1, 5, 5, 2, 1, 5, 5, 2, 4, 5, 5, 5, 3, 1, 3, 3, 1, 5, 5, 1, 2, 4, 5, 5, 5, 1, 5, 3, 4, 4, 5, 4, 5, 5, 1, 3, 5, 2, 3, 1, 4, 1, 5, 4, 4, 5, 5, 1, 1, 4, 5, 3, 3, 5, 1, 5, 4, 3, 5, 5, 4, 5, 5, 3, 4, 5, 4, 3, 3, 1, 4, 4, 5, 4, 1, 5, 5, 5, 5, 3, 5, 5, 5, 1, 5, 3, 4, 5, 5, 5, 1, 3, 5, 4, 3, 5, 4, 5, 4, 5, 5, 5, 2, 3, 4, 1, 1, 5, 4, 1, 3, 4, 2, 1, 5, 5, 1, 5, 5, 5, 4, 4, 5, 2, 1, 4, 2, 1, 4, 1, 4, 5, 5, 4, 1, 1, 5, 4, 1, 5, 5, 1, 1, 1, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 3, 1, 1, 5, 5, 5, 5, 1, 1, 4, 4, 5, 4, 1, 5, 5, 2, 3, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 2, 5]
list1=[2, 5, 5, 5, 3, 5, 5, 5, 5, 4, 2, 5, 3, 5, 5, 5, 5, 5, 3, 1, 2, 1, 5, 1, 1, 5, 5, 5, 5, 1, 1, 5, 5, 5, 3, 1, 5, 5, 2, 5, 1, 1, 2, 1, 4, 4, 1, 1, 5, 4, 5, 1, 4, 5, 5, 5, 5, 5, 1, 5, 2, 3, 3, 3, 5, 5, 1, 1, 3, 5, 5, 5, 5, 4, 2, 2, 5, 1, 3, 5, 5, 5, 4, 5, 1, 4, 4, 5, 4, 5, 4, 1, 5, 4, 5, 5, 3, 5, 3, 5, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 1, 3, 1, 5, 5, 5, 5, 1, 4, 5, 1, 4, 4, 5, 4, 5, 5, 3, 5, 4, 5, 4, 5, 5, 4, 2, 5, 1, 4, 5, 5, 4, 5, 1, 5, 5, 5, 5, 3, 5, 5, 3, 5, 5, 5, 5, 5, 4, 5, 1, 5, 3, 1, 4, 5, 1, 1, 1, 4, 4, 4, 5, 5, 5, 5, 5, 5, 4, 4, 5, 1, 1, 5, 5, 5, 5, 5, 1, 1, 5, 2, 5, 5, 4, 4, 4, 1, 5, 3, 5, 5, 5, 2, 5, 2, 4, 1, 5, 5, 5, 4, 4, 5, 5, 5, 5, 5, 5, 4, 4, 5, 4, 5, 1, 4, 4, 5, 1, 3, 5, 5, 5, 5, 5, 5, 1, 3, 5, 4, 5, 1, 1, 2, 4, 1, 4, 3, 3, 1, 5, 2, 5, 4, 4, 5, 3, 4, 1, 1, 1, 4, 2, 5, 5, 5, 4, 3, 5, 1, 1, 5, 5, 5, 5, 2, 5, 5, 5, 4, 5, 5, 1, 5, 5, 2, 5, 5, 5, 1, 3, 5, 2, 5, 5, 2, 1, 1, 4, 4, 5, 5, 3, 5, 4, 5, 5, 2, 5, 1, 2, 1, 5, 3, 5, 3, 5, 5, 2, 2, 5, 5, 5, 4, 5, 5, 5, 1, 4, 5, 5, 5, 3, 3, 4, 1, 4, 5, 4, 3, 3, 4, 5, 5, 5, 1, 4, 5, 2, 5, 5, 4, 1, 5, 5, 5, 5, 3, 5, 1, 4, 3, 1, 4, 1, 5, 5, 4, 4, 1, 5, 5, 3, 1, 5, 1, 5, 5, 1, 1, 5, 5, 1, 4, 5, 2, 4, 5, 5, 1, 5, 4, 4, 5, 5, 4, 5, 5, 4, 3, 5, 5, 5, 4, 2, 4, 1, 1, 5, 1, 4, 1, 5, 1, 1, 4, 5, 3, 4, 5, 5, 5, 5, 4, 5, 2, 5, 4, 5, 5, 4, 4, 5, 5, 1, 4, 3, 1, 5, 4, 5, 5, 1, 5, 5, 5, 4, 2, 1, 5, 4, 4, 3, 4, 5, 5, 5, 4, 5, 3, 5, 1, 1, 1, 5, 4, 5, 5, 5, 5, 1, 5, 4, 5, 4, 5, 4, 4, 5, 5, 3, 5, 3, 5, 4, 1, 2, 5, 5, 5, 3, 4, 5, 5, 5, 5, 5, 5, 5, 3, 5, 4, 5, 5, 1, 1, 5, 5, 3, 5, 1, 5, 5, 4, 5, 5, 1, 5, 5, 5, 5, 2, 1, 4, 5, 3, 5, 1, 5, 1, 4, 5, 5, 2, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 4, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 1, 5, 5, 1, 4, 4, 5, 4, 4, 5, 5, 4, 4, 3, 4, 5, 3, 5, 5, 1, 5, 5, 5, 5, 1, 2, 5, 4, 4, 5, 5, 5, 5, 1, 2, 5, 3, 5, 1, 4, 5, 2, 1, 5, 4, 5, 5, 4, 5, 5, 4, 4, 5, 5, 5, 5, 4, 3, 3, 5, 4, 1, 1, 5, 1, 5, 3, 4, 4, 2, 5, 5, 2, 5, 5, 3, 5, 5, 5, 3, 5, 5, 4, 5, 5, 5, 5, 1, 5, 1, 5, 2, 5, 4, 5, 4, 5, 5, 4, 5, 5, 1, 4, 3, 5, 5, 3, 5, 1, 5, 5, 1, 4, 4, 5, 1, 4, 2, 5, 1, 1, 5, 5, 1, 5, 5, 3, 3, 4, 1, 3, 5, 4, 4, 5, 5, 5, 3, 1, 5, 4, 4, 5, 4, 4, 5, 4, 5, 5, 1, 5, 4, 5, 1, 5, 5, 5, 4, 1, 5, 2, 4, 5, 5, 4, 1, 5, 4, 3, 5, 4, 5, 5, 5, 5, 5, 4, 5, 5, 5, 1, 2, 4, 2, 2, 3, 3, 5, 5, 3, 5, 3, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 2, 1, 4, 5, 5, 4, 4, 5, 5, 1, 5, 5, 1, 5, 2, 5, 2, 5, 5, 5, 4, 5, 5, 2, 2, 5, 3, 1, 1, 5, 2, 5, 1, 4, 1, 5, 4, 5, 5, 5, 5, 4, 3, 4, 5, 4, 5, 5, 4, 5, 3, 5, 4, 1, 5, 1, 4, 4, 5, 3, 1, 5, 5, 5, 3, 3, 5, 5, 2, 5, 1, 4, 4, 5, 5, 4, 5, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 3, 4, 3, 5, 5, 4, 1, 5, 4, 3, 5, 5, 4, 5, 4, 5, 5, 2, 5, 1, 5, 5, 5, 5, 1, 3, 5, 5, 4, 1, 1, 5, 4, 5, 4, 5, 5, 1, 5, 5, 5, 5, 4, 5, 1, 5, 4, 5, 5, 5, 4, 2, 5, 2, 1, 3, 5, 1, 5, 5, 5, 3, 5, 3, 5, 3, 5, 1, 3, 5, 5, 1, 1, 5, 5, 5, 5, 1, 1, 3, 5, 5, 2, 5, 3, 3, 3, 5, 5, 1, 5, 5, 5, 3, 4, 5, 4, 4, 5, 2, 2, 3, 5, 2, 3, 1, 3, 5, 3, 4, 4, 1, 2, 4, 5, 4, 4, 5, 4, 5, 4, 5, 4, 3, 5, 4, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 4, 2, 5, 1, 5, 1, 4, 4, 4, 5, 1, 2, 1, 5, 5, 5, 4, 5, 4, 5, 5, 5, 5, 1, 5, 1, 5, 5, 5, 3, 5, 5, 4, 1, 5, 5, 5, 1, 5, 5, 5, 5, 5, 4, 4, 1, 5, 5, 1, 3, 1, 5, 4, 4, 5, 5, 2, 5, 5, 5, 3, 5, 4, 4, 4, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 1, 1, 5, 2, 5, 1, 2, 5, 5, 3, 5, 1, 4, 4, 5, 5, 4, 5, 4, 5, 3, 5, 1, 3, 2, 1, 4, 5, 1, 3, 5, 5, 3, 5, 4, 5, 5, 5, 3, 5, 4, 5, 5, 5, 5, 5, 2, 4, 5, 4, 5, 5, 5, 4, 5, 2, 5, 5, 5, 2, 5, 5, 4, 1, 5, 4, 4, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5, 1, 2, 2, 5, 5, 3, 5, 1, 5, 5, 4, 2, 5, 1, 2, 5, 5, 1, 1, 5, 5, 4, 5, 5, 4, 5, 4, 2, 4, 5, 3, 5, 5, 5, 5, 4, 4, 3, 3, 4, 4, 5, 5, 1, 1, 5, 5, 1, 5, 5, 4, 5, 5, 4, 4, 5, 1, 4, 2, 5, 1, 1, 5, 4, 5, 1, 3, 2, 2, 4, 3, 4, 5, 5, 1, 1, 5, 5, 5, 1, 5, 4, 5, 5, 5, 5, 3, 1, 4, 1, 5, 5, 1, 4, 1, 1, 3, 4, 5, 5, 5, 5, 4, 1, 5, 5, 5, 5, 3, 5, 1, 5, 3, 1, 4, 5, 5, 5, 1, 4, 3, 5, 5, 5, 3, 1, 5, 2, 5, 5, 5, 4, 4, 2, 4, 2, 5, 5, 1, 4, 1, 4, 4, 5, 5, 5, 1, 4, 5, 3, 5, 5, 5, 5, 5, 5, 5, 4, 4, 5, 3, 3, 5, 1, 4, 5, 5, 5, 1, 5, 5, 1, 5, 2, 1, 3, 4, 3, 2, 5, 4, 1, 5, 4, 5, 2, 5, 2, 3, 1, 4, 5, 5, 5, 1, 3, 4, 1, 3, 5, 5, 4, 2, 5, 5, 2, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 4, 5, 4, 1, 5, 1, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 1, 3, 4, 4, 1, 4, 1, 5, 4, 2, 3, 1, 3, 1, 4, 1, 5, 5, 3, 2, 1, 1, 3, 1, 5, 1, 5, 5, 5, 4, 5, 4, 5, 5, 5, 4, 5, 3, 4, 1, 5, 5, 5, 5, 5, 4, 3, 5, 4, 5, 4, 4, 4, 5, 5, 1, 4, 1, 1, 5, 3, 5, 3, 5, 5, 5, 1, 5, 5, 5, 1, 1, 5, 3, 5, 1, 5, 5, 5, 5, 5, 1, 2, 4, 4, 5, 1, 4, 5, 4, 4, 4, 5, 5, 3, 5, 5, 1, 5, 5, 4, 2, 4, 5, 1, 5, 5, 4, 1, 5, 1, 4, 3, 1, 4, 5, 1, 5, 3, 2, 5, 5, 2, 5, 5, 4, 5, 5, 1, 4, 5, 3, 2, 5, 2, 5, 5, 5, 5, 4, 5, 5, 5, 1, 5, 2, 1, 5, 4, 5, 3, 1, 4, 5, 5, 5, 1, 1, 3, 3, 1, 5, 5, 3, 5, 5, 4, 1, 5, 4, 5, 5, 4, 5, 5, 3, 1, 5, 1, 1, 3, 4, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 4, 5, 1, 5, 4, 5, 4, 5, 5, 3, 5, 3, 1, 4, 5, 4, 4, 1, 5, 5, 5, 4, 3, 1, 4, 5, 5, 5, 1, 3, 1, 5, 2, 3, 5, 4, 5, 5, 5, 5, 4, 3, 5, 5, 5, 5, 1, 5, 3, 4, 4, 5, 4, 4, 5, 1, 5, 4, 5, 5, 4, 5, 3, 2, 4, 5, 5, 4, 4, 5, 1, 3, 5, 5, 5, 5, 5, 4, 3, 5, 4, 5, 1, 4, 4, 4, 1, 4, 1, 5, 4, 4, 4, 4, 3, 5, 5, 5, 5, 5, 1, 5, 5, 3, 5, 5, 1, 5, 1, 5, 2, 4, 5, 4, 5, 5, 5, 4, 1, 4, 4, 3, 4, 3, 2, 5, 5, 5, 5, 4, 4, 5, 4, 5, 1, 4, 5, 5, 5, 1, 2, 5, 5, 1, 4, 4, 2, 5, 5, 1, 2, 4, 5, 5, 1, 5, 5, 4, 5, 5, 3, 4, 5, 1, 4, 5, 3, 4, 5, 1, 4, 5, 4, 5, 5, 4, 1, 4, 2, 1, 5, 5, 3, 4, 1, 5, 5, 4, 4, 3, 3, 5, 3, 5, 5, 5, 2, 5, 5, 3, 5, 5, 1, 1, 3, 4, 1, 1, 4, 3, 5, 4, 2, 4, 2, 3, 5, 3, 4, 5, 4, 4, 2, 5, 5, 5, 1, 5, 4, 5, 1, 5, 5, 5, 5, 5, 5, 5, 4, 5, 2, 5, 1, 4, 5, 1, 4, 1, 5, 4, 5, 5, 5, 5, 5, 4, 2, 5, 5, 3, 5, 3, 1, 4, 1, 5, 1, 4, 5, 4, 4, 5, 5, 1, 4, 5, 1, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 4, 3, 5, 3, 5, 5, 4, 4, 5, 5, 4, 3, 3, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 2, 5, 3, 1, 4, 1, 1, 5, 4, 5, 5, 3, 5, 1, 5, 1, 5, 5, 5, 3, 5, 5, 1, 5, 4, 3, 5, 5, 3, 4, 5, 2, 5, 5, 4, 5, 1, 5, 1, 4, 5, 5, 4, 4, 5, 3, 4, 5, 4, 1, 5, 5, 5, 5, 1, 1, 1, 3, 4, 5, 4, 5, 3, 5, 3, 1, 4, 3, 4, 4, 2, 5, 4, 5, 5, 2, 5, 2, 4, 5, 5, 3, 5, 3, 5, 4, 1, 4, 3, 1, 3, 5, 1, 5, 5, 4, 4, 4, 1, 5, 4, 1, 5, 3, 5, 5, 1, 1, 4, 5, 4, 5, 1, 5, 5, 4, 2, 5, 5, 2, 5, 5, 4, 1, 4, 4, 4, 5, 4, 3, 5, 4, 1, 5, 4, 3, 4, 2, 5, 5, 5, 5, 5, 5, 5, 2, 4, 5, 1, 2, 4, 5, 4, 4, 5, 4, 5, 5, 4, 5, 1, 5, 5, 5, 3, 5, 5, 5, 4, 4, 2, 4, 5, 4, 2, 5, 5, 1, 5, 5, 4, 5, 5, 5, 1, 5, 4, 1, 4, 4, 1, 3, 5, 4, 3, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 1, 4, 5, 5, 3, 5, 5, 1, 5, 5, 5, 3, 5, 3, 5, 5, 3, 5, 5, 5, 5, 5, 1, 4, 4, 5, 5, 5, 5, 4, 5, 5, 5, 3, 5, 5, 1, 5, 2, 5, 5, 5, 5, 3, 4, 5, 1, 4, 5, 1, 5, 5, 4, 1, 5, 5, 5, 1, 5, 5, 1, 5, 5, 1, 5, 5, 3, 4, 5, 3, 5, 4, 4, 1, 5, 2, 5, 5, 5, 4, 4, 3, 3, 5, 5, 4, 2, 4, 1, 3, 4, 2, 1, 1, 1, 1, 4, 5, 4, 1, 4, 5, 5, 2, 3, 3, 4, 5, 5, 5, 1, 5, 1, 5, 5, 4, 1, 5, 4, 4, 1, 5, 1, 4, 1, 3, 1, 4, 3, 5, 5, 5, 5, 5, 5, 1, 5, 4, 3, 4, 5, 4, 1, 4, 4, 1, 1, 4, 4, 4, 5, 4, 3, 5, 4, 5, 1, 4, 5, 1, 4, 3, 5, 5, 5, 2, 5, 1, 2, 1, 4, 1, 3, 5, 1, 4, 5, 5, 4, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 4, 5, 4, 1, 4, 5, 1, 2, 4, 4, 5, 5, 5, 3, 5, 5, 5, 5, 2, 5, 5, 5, 1, 4, 5, 5, 4, 2, 5, 5, 1, 5, 4, 5, 5, 1, 1, 4, 4, 2, 5, 5, 4, 4, 5, 1, 5, 5, 5, 4, 5, 5, 3, 5, 5, 1, 1, 1, 5, 5, 2, 5, 5, 4, 4, 1, 5, 5, 5, 5, 5, 3, 5, 5, 5, 4, 5, 5, 5, 3, 5, 5, 5, 5, 5, 2, 5, 5, 3, 5, 5, 3, 4, 5, 4, 5, 1, 5, 4, 5, 1, 4, 1, 5, 5, 5, 5, 1, 4, 4, 5, 2, 1, 5, 5, 5, 1, 5, 1, 3, 3, 3, 1, 5, 5, 4, 1, 5, 5, 4, 5, 1, 5, 5, 4, 3, 5, 2, 5, 2, 5, 5, 4, 2, 2, 5, 1, 1, 5, 4, 3, 2, 5, 1, 4, 2, 5, 5, 4, 4, 4, 1, 5, 4, 5, 1, 5, 1, 1, 5, 5, 5, 4, 5, 4, 5, 5, 1, 4, 5, 1, 5, 5, 5, 1, 5, 4, 1, 5, 5, 1, 1, 5, 5, 4, 5, 4, 1, 5, 3, 5, 4, 4, 5, 5, 4, 2, 5, 4, 4, 5, 5, 5, 1, 5, 5, 4, 5, 4, 5, 4, 4, 5, 5, 5, 1, 5, 1, 5, 1, 3, 4, 5, 4, 4, 5, 1, 1, 1, 1, 4, 5, 1, 5, 4, 5, 1, 5, 3, 1, 1, 5, 5, 5, 5, 3, 4, 5, 1, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 4, 4, 5, 5, 5, 1, 5, 5, 4, 1, 4, 5, 5, 5, 4, 5, 5, 2, 5, 4, 5, 5, 1, 5, 5, 1, 5, 4, 4, 1, 2, 1, 5, 2, 5, 5, 5, 5, 3, 5, 4, 1, 1, 5, 5, 5, 3, 3, 5, 5, 5, 5, 1, 5, 5, 4, 4, 5, 1, 5, 4, 5, 5, 4, 2, 1, 5, 2, 5, 5, 4, 4, 1, 5, 4, 4, 3, 5, 4, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 2, 5, 4, 5, 2, 4, 1, 1, 4, 1, 4, 2, 5, 3, 3, 5, 5, 5, 5, 5, 5, 4, 3, 3, 5, 5, 5, 5, 4, 5, 3, 5, 1, 4, 5, 5, 1, 1, 2, 2, 5, 2, 4, 5, 1, 5, 4, 5, 5, 3, 3, 5, 5, 1, 5, 3, 1, 5, 5, 5, 4, 3, 3, 5, 3, 5, 2, 3, 5, 5, 2, 4, 5, 4, 4, 4, 4, 1, 2, 5, 5, 5, 4, 3, 1, 4, 1, 5, 5, 3, 5, 5, 5, 5, 5, 5, 1, 1, 4, 1, 5, 1, 4, 5, 4, 5, 5, 3, 4, 4, 3, 1, 3, 5, 4, 1, 3, 5, 5, 1, 1, 5, 4, 5, 4, 5, 4, 1, 5, 3, 5, 5, 5, 4, 3, 5, 1, 5, 5, 3, 3, 5, 5, 5, 4, 5, 5, 5, 1, 2, 5, 3, 4, 2, 2, 2, 1, 3, 5, 5, 1, 5, 5, 1, 3, 3, 5, 2, 1, 1, 5, 2, 4, 5, 5, 3, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 3, 1, 3, 5, 5, 5, 2, 1, 4, 3, 1, 3, 4, 2, 1, 4, 1, 2, 5, 4, 4, 4, 4, 1, 5, 5, 1, 4, 5, 5, 2, 5, 4, 3, 5, 4, 1, 1, 1, 5, 4, 2, 4, 1, 4, 5, 4, 5, 2, 5, 5, 5, 5, 5, 1, 3, 5, 5, 1, 5, 1, 5, 5, 5, 1, 5, 3, 5, 5, 4, 5, 5, 5, 4, 4, 3, 3, 5, 2, 5, 5, 4, 4, 4, 3, 4, 2, 5, 1, 5, 4, 3, 5, 5, 5, 2, 3, 2, 5, 5, 1, 2, 4, 5, 3, 4, 5, 4, 2, 1, 5, 5, 4, 1, 3, 5, 5, 3, 4, 1, 5, 1, 5, 5, 5, 1, 4, 5, 5, 5, 2, 4, 1, 5, 4, 1, 3, 3, 5, 1, 5, 4, 5, 5, 3, 5, 5, 5, 4, 5, 1, 1, 5, 5, 5, 5, 5, 1, 5, 4, 3, 5, 1, 4, 4, 3, 5, 1, 5, 1, 4, 5, 5, 5, 1, 5, 5, 4, 3, 5, 5, 2, 5, 5, 5, 1, 5, 1, 5, 5, 3, 4, 2, 1, 1, 5, 1, 4, 3, 3, 4, 5, 5, 2, 5, 4, 3, 4, 5, 5, 5, 5, 4, 5, 5, 4, 1, 5, 5, 1, 1, 4, 1, 3, 1, 5, 5, 1, 5, 1, 2, 4, 1, 5, 4, 5, 1, 5, 4, 1, 5, 1, 5, 4, 1, 5, 5, 5, 5, 5, 1, 5, 5, 5, 3, 4, 4, 1, 3, 5, 3, 5, 3, 5, 1, 5, 5, 5, 5, 4, 5, 5, 5, 1, 3, 5, 5, 5, 4, 4, 1, 4, 5, 1, 3, 3, 2, 1, 5, 5, 3, 5, 5, 5, 5, 1, 4, 5, 5, 5, 5, 4, 5, 4, 4, 5, 3, 5, 5, 4, 3, 5, 3, 4, 5, 3, 5, 5, 1, 4, 5, 1, 1, 5, 5, 4, 4, 3, 2, 5, 5, 5, 4, 5, 4, 5, 3, 4, 1, 4, 5, 4, 5, 2, 1, 5, 5, 1, 5, 5, 5, 5, 3, 5, 3, 2, 4, 1, 1, 4, 2, 5, 3, 4, 3, 5, 5, 5, 5, 3, 5, 5, 5, 1, 5, 5, 1, 3, 5, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 3, 5, 1, 5, 5, 3, 1, 2, 5, 5, 4, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 4, 5, 5, 4, 5, 5, 2, 5, 2, 5, 5, 5, 1, 3, 3, 5, 1, 1, 5, 2, 3, 5, 1, 4, 5, 5, 5, 1, 4, 5, 4, 4, 1, 1, 1, 5, 5, 1, 5, 2, 1, 5, 5, 4, 3, 5, 5, 5, 5, 5, 4, 5, 2, 5, 5, 5, 2, 5, 3, 5, 5, 1, 4, 4, 5, 3, 5, 4, 5, 2, 5, 5, 4, 5, 4, 5, 5, 5, 1, 5, 1, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 3, 5, 5, 4, 1, 3, 1, 4, 4, 5, 5, 1, 5, 4, 5, 5, 1, 2, 1, 4, 5, 3, 1, 5, 1, 1, 5, 5, 4, 5, 3, 5, 5, 1, 5, 2, 5, 5, 3, 4, 1, 5, 1, 5, 5, 5, 5, 5, 4, 1, 5, 3, 5, 5, 5, 4, 4, 4, 5, 4, 4, 5, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 2, 5, 5, 5, 5, 1, 1, 2, 5, 4, 4, 5, 5, 4, 4, 5, 4, 5, 5, 1, 5, 4, 5, 5, 1, 4, 5, 4, 5, 2, 5, 4, 3, 3, 1, 5, 5, 1, 5, 5, 4, 5, 5, 5, 4, 5, 3, 5, 5, 5, 4, 5, 3, 5, 5, 5, 4, 5, 5, 5, 5, 1, 5, 2, 5, 4, 5, 5, 5, 5, 5, 1, 5, 2, 2, 5, 4, 5, 1, 5, 1, 1, 5, 5, 4, 5, 5, 3, 4, 5, 1, 1, 1, 3, 4, 5, 1, 5, 5, 1, 1, 5, 5, 5, 5, 4, 5, 1, 5, 5, 4, 5, 5, 1, 3, 1, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 1, 1, 1, 5, 4, 5, 5, 3, 5, 3, 5, 4, 5, 1, 5, 4, 5, 5, 1, 5, 5, 5, 1, 2, 4, 5, 5, 5, 3, 4, 2, 5, 4, 3, 4, 5, 5, 4, 2, 4, 2, 5, 5, 5, 5, 4, 5, 5, 3, 5, 4, 5, 5, 3, 5, 5, 2, 5, 5, 5, 4, 1, 5, 1, 5, 5, 4, 5, 1, 5, 1, 5, 4, 5, 4, 3, 5, 5, 3, 4, 3, 1, 5, 4, 1, 4, 4, 5, 5, 5, 3, 5, 5, 1, 1, 2, 1, 4, 5, 1, 2, 5, 5, 1, 2, 1, 1, 5, 5, 5, 5, 5, 4, 5, 5, 4, 5, 4, 4, 4, 3, 4, 2, 3, 5, 5, 5, 5, 5, 5, 1, 5, 4, 3, 4, 4, 1, 5, 5, 5, 3, 5, 4, 5, 5, 2, 3, 3, 3, 5, 1, 5, 5, 3, 1, 4, 4, 4, 1, 5, 5, 4, 5, 5, 5, 5, 3, 5, 5, 2, 4, 5, 4, 4, 5, 4, 5, 4, 5, 1, 5, 1, 5, 5, 5, 5, 5, 5, 4, 4, 4, 5, 4, 4, 5, 1, 1, 1, 5, 5, 1, 5, 4, 5, 5, 4, 1, 5, 5, 5, 5, 4, 5, 4, 5, 5, 5, 5, 5, 5, 5, 4, 3, 4, 4, 5, 5, 5, 5, 5, 5, 1, 4, 5, 3, 4, 5, 5, 3, 5, 1, 5, 3, 1, 3, 4, 1, 5, 4, 5, 5, 2, 2, 4, 4, 5, 5, 1, 3, 5, 5, 5, 1, 4, 1, 4, 5, 5, 3, 3, 5, 5, 4, 5, 4, 5, 5, 1, 5, 5, 5, 4, 3, 4, 1, 2, 4, 3, 4, 5, 4, 3, 5, 2, 2, 5, 4, 4, 4, 5, 5, 4, 5, 5, 5, 4, 5, 5, 1, 4, 4, 5, 5, 5, 4, 5, 1, 3, 5, 5, 3, 2, 4, 4, 2, 5, 5, 5, 4, 4, 5, 4, 5, 5, 5, 5, 5, 5, 1, 5, 5, 1, 4, 5, 3, 4, 5, 3, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 4, 5, 1, 2, 5, 4, 5, 5, 5, 4, 5, 4, 5, 4, 4, 5, 5, 5, 4, 5, 4, 3, 5, 1, 3, 5, 4, 5, 1, 1, 5, 5, 4, 3, 3, 1, 3, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 5, 1, 3, 4, 5, 5, 5, 4, 4, 5, 5, 5, 4, 5, 5, 3, 5, 4, 5, 5, 4, 1, 5, 5, 3, 5, 4, 3, 5, 4, 1, 4, 5, 1, 4, 4, 3, 4, 4, 1, 5, 4, 1, 1, 4, 4, 5, 5, 5, 5, 1, 4, 5, 3, 5, 1, 4, 4, 5, 5, 1, 4, 4, 1, 5, 1, 1, 5, 1, 4, 5, 5, 5, 2, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 4, 1, 1, 4, 5, 3, 1, 1, 4, 4, 2, 5, 4, 3, 5, 4, 5, 5, 1, 5, 4, 5, 1, 5, 5, 1, 1, 4, 5, 4, 3, 2, 1, 3, 5, 5, 5, 1, 4, 3, 4, 5, 1, 3, 5, 4, 5, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 1, 4, 2, 1, 5, 4, 4, 3, 1, 5, 5, 4, 4, 5, 5, 4, 1, 5, 5, 4, 3, 3, 3, 3, 4, 3, 1, 5, 1, 4, 1, 4, 5, 2, 4, 5, 5, 1, 5, 3, 4, 3, 5, 4, 4, 1, 4, 1, 3, 5, 1, 4, 5, 4, 5, 5, 3, 5, 5, 1, 3, 5, 2, 4, 3, 1, 2, 5, 2, 4, 1, 5, 1, 4, 1, 4, 4, 5, 5, 5, 5, 4, 2, 5, 5, 4, 5, 4, 1, 2, 3, 5, 3, 2, 5, 5, 4, 2, 5, 1, 4, 1, 4, 1, 3, 4, 3, 4, 2, 1, 1, 1, 1, 5, 2, 5, 5, 1, 5, 5, 2, 5, 3, 1, 1, 5, 4, 1, 1, 5, 5, 4, 1, 3, 5, 5, 5, 1, 4, 5, 1, 3, 5, 4, 1, 1, 5, 4, 2, 5, 1, 1, 1, 1, 5, 4, 4, 5, 3, 3, 5, 3, 3, 3, 5, 5, 4, 3, 1, 4, 4, 2, 5, 1, 4, 4, 4, 5, 4, 1, 1, 5, 5, 5, 1, 1, 3, 5, 5, 5, 5, 5, 3, 5, 1, 5, 1, 5, 5, 4, 3, 4, 4, 5, 4, 5, 4, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5, 1, 5, 2, 5, 5, 5, 4, 5, 1, 1, 5, 4, 3, 5, 5, 4, 5, 5, 1, 3, 1, 4, 5, 1, 2, 2, 5, 5, 1, 5, 5, 5, 4, 1, 4, 3, 4, 5, 1, 5, 1, 4, 1, 5, 3, 1, 4, 5, 5, 3, 5, 5, 3, 5, 5, 5, 5, 1, 4, 5, 4, 4, 4, 1, 5, 2, 5, 5, 1, 5, 5, 4, 3, 5, 4, 2, 5, 5, 5, 3, 5, 2, 5, 5, 5, 1, 5, 5, 5, 5, 4, 5, 3, 3, 4, 5, 5, 5, 1, 5, 5, 4, 3, 5, 5, 5, 2, 3, 5, 1, 1, 3, 2, 5, 1, 5, 1, 5, 5, 5, 5, 1, 5, 5, 4, 2, 4, 5, 3, 1, 5, 5, 5, 5, 5, 5, 5, 4, 2, 4, 5, 4, 5, 5, 3, 4, 5, 5, 2, 2, 5, 5, 5, 5, 4, 5, 5, 5, 5, 1, 5, 5, 3, 5, 5, 1, 5, 5, 4, 5, 4, 5, 5, 5, 1, 5, 5, 4, 5, 1, 5, 5, 1, 3, 5, 5, 4, 4, 5, 5, 4, 4, 2, 4, 5, 1, 5, 4, 3, 1, 5, 5, 2, 3, 5, 4, 4, 5, 4, 5, 4, 5, 5, 4, 5, 5, 1, 4, 4, 1, 5, 5, 5, 5, 4, 1, 5, 5, 2, 5, 2, 5, 5, 3, 5, 1, 4, 4, 5, 1, 5, 4, 5, 1, 4, 1, 5, 4, 1, 5, 5, 5, 5, 5, 3, 4, 3, 5, 1, 5, 3, 5, 5, 5, 4, 5, 5, 5, 5, 4, 5, 1, 5, 3, 5, 3, 1, 1, 3, 1, 5, 5, 5, 5, 1, 5, 5, 1, 5, 5, 2, 5, 4, 5, 4, 5, 1, 3, 5, 3, 1, 1, 4, 5, 5, 5, 5, 3, 5, 5, 1, 1, 5, 5, 5, 4, 5, 4, 4, 5, 5, 4, 1, 1, 3, 5, 5, 5, 2, 3, 4, 4, 3, 5, 1, 1, 4, 5, 4, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 1, 4, 5, 1, 5, 3, 5, 5, 5, 4, 5, 4, 3, 5, 5, 2, 5, 5, 5, 1, 5, 5, 1, 5, 1, 5, 5, 1, 4, 4, 3, 5, 5, 5, 3, 4, 5, 4, 4, 4, 3, 5, 4, 3, 4, 1, 4, 5, 4, 4, 5, 4, 3, 1, 5, 4, 1, 5, 1, 4, 5, 4, 1, 5, 5, 4, 1, 1, 5, 1, 5, 1, 5, 2, 1, 5, 4, 3, 4, 5, 5, 5, 5, 3, 5, 5, 3, 3, 5, 3, 3, 5, 3, 5, 1, 5, 3, 4, 5, 4, 5, 2, 5, 5, 5, 4, 5, 4, 5, 5, 5, 4, 3, 1, 3, 5, 2, 5, 5, 4, 5, 3, 5, 5, 1, 2, 1, 1, 5, 5, 2, 5, 5, 4, 2, 5, 5, 3, 5, 3, 4, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 4, 5, 5, 5, 2, 5, 5, 1, 5, 5, 5, 1, 3, 5, 4, 3, 2, 3, 1, 5, 5, 1, 5, 4, 5, 5, 5, 1, 5, 1, 3, 5, 4, 1, 2, 5, 5, 4, 4, 3, 5, 1, 4, 3, 5, 4, 5, 2, 5, 4, 5, 5, 2, 1, 5, 4, 1, 1, 5, 5, 4, 5, 5, 5, 5, 5, 5, 1, 5, 5, 4, 1, 5, 4, 1, 1, 4, 5, 2, 5, 5, 5, 3, 1, 5, 5, 5, 5, 1, 3, 1, 4, 4, 5, 5, 5, 5, 5, 1, 5, 5, 3, 4, 1, 5, 5, 5, 5, 2, 5, 5, 4, 5, 3, 4, 4, 5, 4, 2, 5, 5, 5, 5, 4, 3, 4, 5, 5, 5, 1, 5, 1, 5, 5, 1, 3, 1, 3, 5, 5, 5, 1, 5, 4, 1, 5, 5, 2, 4, 4, 3, 3, 5, 5, 5, 5, 4, 1, 1, 5, 5, 5, 4, 5, 5, 5, 5, 1, 1, 5, 4, 2, 5, 1, 3, 5, 4, 4, 5, 3, 5, 4, 3, 5, 5, 5, 1, 5, 5, 3, 5, 4, 1, 5, 5, 5, 4, 5, 5, 4, 1, 5, 5, 1, 5, 5, 5, 3, 2, 5, 5, 2, 5, 5, 5, 5, 5, 5, 1, 4, 5, 1, 4, 4, 1, 2, 1, 4, 4, 5, 4, 1, 1, 5, 1, 1, 5, 3, 1, 4, 5, 5, 5, 1, 1, 5, 5, 4, 5, 5, 5, 1, 1, 2, 3, 1, 5, 5, 4, 4, 1, 5, 5, 2, 1, 1, 1, 2, 5, 2, 2, 4, 5, 5, 5, 3, 5, 3, 4, 3, 1, 5, 5, 4, 5, 5, 5, 5, 4, 3, 3, 5, 1, 4, 5, 5, 4, 5, 5, 5, 4, 5, 5, 4, 5, 3, 5, 1, 3, 1, 1, 4, 5, 5, 4, 5, 1, 5, 4, 5, 3, 2, 5, 4, 5, 1, 1, 5, 1, 4, 4, 5, 5, 5, 4, 4, 5, 4, 5, 5, 2, 5, 5, 5, 4, 5, 5, 4, 5, 5, 3, 1, 5, 1, 1, 5, 3, 5, 5, 1, 5, 5, 5, 3, 1, 5, 1, 3, 3, 5, 4, 4, 5, 2, 5, 1, 5, 1, 5, 1, 4, 5, 5, 4, 5, 5, 1, 1, 4, 5, 4, 5, 3, 5, 5, 5, 1, 5, 5, 5, 5, 5, 1, 5, 4, 5, 5, 5, 1, 5, 5, 4, 4, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 1, 2, 5, 4, 4, 5, 5, 1, 5, 2, 5, 5, 5, 5, 5, 3, 3, 3, 3, 1, 3, 5, 3, 5, 5, 4, 5, 3, 5, 5, 1, 5, 4, 1, 5, 3, 1, 5, 5, 3, 4, 3, 1, 1, 1, 5, 3, 4, 1, 1, 3, 5, 5, 4, 4, 5, 3, 1, 1, 5, 1, 5, 1, 3, 4, 5, 5, 4, 1, 5, 5, 4, 1, 5, 5, 2, 4, 4, 3, 5, 5, 3, 1, 1, 5, 5, 5, 5, 5, 5, 5, 1, 1, 3, 5, 5, 5, 1, 5, 5, 3, 5, 5, 5, 4, 5, 5, 5, 5, 1, 5, 5, 2, 5, 1, 3, 4, 2, 5, 5, 5, 5, 5, 5, 4, 3, 5, 3, 3, 1, 5, 5, 1, 2, 5, 3, 4, 1, 5, 1, 4, 3, 5, 5, 5, 2, 4, 4, 5, 3, 1, 3, 2, 5, 1, 1, 5, 3, 4, 5, 1, 5, 5, 4, 3, 1, 1, 1, 5, 4, 3, 5, 1, 4, 4, 1, 3, 5, 5, 5, 4, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 1, 5, 4, 5, 5, 5, 4, 1, 1, 5, 3, 4, 3, 1, 5, 5, 5, 4, 1, 1, 4, 4, 1, 3, 3, 3, 4, 5, 4, 4, 4, 1, 4, 3, 5, 3, 5, 4, 5, 1, 4, 5, 4, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 3, 5, 5, 2, 4, 4, 5, 5, 5, 1, 4, 5, 1, 5, 5, 5, 3, 5, 1, 5, 1, 5, 1, 4, 5, 1, 4, 1, 4, 1, 5, 4, 1, 5, 5, 4, 5, 1, 5, 5, 5, 5, 4, 1, 1, 5, 4, 5, 1, 1, 5, 5, 5, 5, 5, 1, 5, 5, 5, 3, 5, 3, 5, 1, 5, 5, 1, 5, 2, 5, 5, 1, 5, 5, 5, 1, 5, 5, 4, 4, 5, 5, 5, 5, 5, 1, 1, 4, 3, 3, 5, 5, 5, 5, 5, 4, 4, 2, 1, 4, 5, 5, 4, 1, 1, 4, 5, 5, 5, 5, 5, 5, 1, 1, 5, 4, 5, 5, 5, 5, 1, 2, 5, 1, 5, 5, 1, 5, 5, 5, 4, 5, 5, 4, 1, 5, 4, 3, 2, 4, 5, 3, 5, 5, 1, 4, 1, 4, 4, 1, 3, 5, 5, 5, 5, 1, 5, 1, 4, 5, 5, 4, 4, 5, 5, 4, 4, 4, 5, 5, 5, 5, 5, 3, 4, 5, 5, 5, 1, 1, 5, 5, 1, 4, 5, 5, 1, 5, 5, 5, 5, 5, 4, 3, 2, 5, 4, 5, 4, 1, 4, 4, 3, 5, 4, 5, 1, 5, 3, 3, 4, 4, 1, 4, 5, 4, 4, 4, 1, 1, 1, 5, 4, 1, 4, 5, 1, 5, 1, 3, 4, 5, 4, 5, 1, 4, 1, 3, 1, 4, 5, 5, 1, 5, 5, 1, 5, 5, 5, 3, 5, 5, 1, 5, 2, 5, 5, 3, 3, 1, 5, 5, 5, 3, 1, 5, 4, 1, 5, 5, 1, 5, 4, 3, 4, 3, 5, 1, 3, 2, 5, 5, 5, 1, 5, 5, 1, 1, 4, 3, 1, 1, 5, 1, 3, 5, 5, 5, 4, 5, 1, 3, 4, 5, 5, 4, 5, 1, 5, 5, 4, 1, 3, 1, 1, 5, 1, 4, 5, 2, 3, 4, 2, 1, 5, 1, 3, 5, 4, 4, 5, 3, 5, 3, 1, 5, 4, 5, 5, 5, 4, 5, 5, 3, 4, 5, 5, 4, 3, 4, 3, 3, 4, 5, 3, 5, 4, 3, 3, 5, 5, 4, 3, 3, 1, 5, 5, 1, 4, 1, 1, 4, 5, 5, 5, 3, 3, 5, 5, 1, 5, 4, 4, 3, 1, 3, 4, 5, 5, 1, 4, 5, 4, 4, 5, 1, 5, 5, 5, 1, 1, 3, 5, 5, 5, 5, 5, 5, 1, 5, 4, 4, 2, 1, 5, 5, 5, 4, 4, 4, 3, 5, 4, 5, 5, 5, 4, 1, 5, 5, 1, 5, 5, 5, 1, 5, 4, 1, 4, 5, 5, 5, 3, 5, 1, 4, 4, 4, 4, 5, 4, 5, 3, 5, 5, 1, 4, 5, 5, 4, 1, 2, 5, 5, 5, 5, 5, 5, 5, 2, 1, 4, 5, 5, 5, 1, 5, 4, 5, 5, 5, 5, 5, 5, 5, 3, 4, 4, 5, 2, 1, 4, 4, 4, 5, 5, 4, 1, 5, 1, 5, 4, 3, 1, 4, 5, 4, 5, 4, 4, 4, 5, 5, 1, 4, 4, 5, 1, 2, 3, 5, 4, 2, 3, 5, 1, 5, 3, 5, 1, 5, 2, 5, 2, 3, 4, 5, 2, 1, 5, 5, 5, 3, 3, 5, 4, 3, 3, 3, 4, 5, 1, 5, 5, 5, 3, 5, 4, 4, 5, 3, 1, 5, 1, 5, 3, 5, 5, 5, 5, 5, 2, 4, 5, 5, 3, 4, 2, 5, 5, 5, 3, 5, 5, 3, 5, 5, 5, 5, 2, 4, 1, 1, 5, 5, 5, 3, 4, 5, 5, 5, 3, 1, 5, 1, 1, 4, 1, 3, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 3, 4, 5, 3, 2, 4, 5, 5, 1, 5, 5, 4, 5, 5, 1, 4, 1, 1, 2, 5, 5, 4, 4, 4, 2, 5, 3, 5, 5, 5, 4, 3, 5, 5, 2, 2, 1, 5, 5, 1, 5, 5, 5, 4, 1, 5, 4, 4, 4, 4, 2, 4, 5, 5, 2, 5, 4, 5, 5, 3, 3, 5, 1, 2, 1, 5, 5, 1, 1, 5, 5, 5, 5, 3, 1, 5, 5, 4, 4, 4, 4, 1, 5, 5, 5, 3, 4, 5, 4, 5, 4, 1, 5, 4, 5, 4, 5, 1, 4, 1, 5, 5, 1, 3, 5, 1, 4, 2, 2, 5, 3, 4, 3, 5, 5, 5, 5, 5, 5, 5, 1, 1, 5, 1, 5, 4, 2, 3, 5, 4, 5, 4, 1, 5, 4, 1, 5, 1, 3, 5, 5, 5, 1, 4, 4, 5, 2, 4, 5, 1, 1, 4, 5, 2, 1, 1, 5, 1, 1, 5, 5, 1, 1, 5, 3, 1, 5, 1, 2, 1, 4, 1, 4, 1, 5, 5, 5, 3, 5, 5, 4, 5, 5, 4, 4, 5, 4, 5, 1, 1, 5, 5, 4, 3, 4, 5, 3, 4, 5, 4, 5, 2, 3, 5, 3, 5, 2, 1, 5, 4, 5, 1, 4, 1, 5, 4, 2, 5, 5, 5, 5, 5, 5, 4, 5, 3, 1, 3, 5, 4, 3, 5, 3, 3, 5, 4, 5, 3, 5, 1, 5, 1, 5, 4, 5, 3, 4, 4, 5, 4, 5, 4, 5, 5, 3, 2, 3, 1, 4, 5, 2, 3, 1, 3, 2, 5, 5, 4, 4, 4, 5, 3, 5, 5, 5, 3, 2, 1, 5, 2, 5, 4, 5, 5, 5, 1, 5, 5, 5, 5, 4, 5, 1, 5, 5, 3, 5, 1, 5, 5, 5, 5, 4, 2, 5, 4, 5, 5, 5, 5, 2, 3, 3, 4, 1, 5, 5, 5, 4, 1, 5, 1, 1, 4, 1, 4, 5, 4, 5, 5, 5, 5, 3, 4, 5, 1, 1, 5, 5, 1, 5, 1, 4, 5, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 3, 1, 5, 5, 5, 5, 4, 3, 4, 1, 5, 3, 2, 3, 4, 4, 3, 4, 4, 5, 5, 5, 3, 5, 1, 4, 5, 4, 2, 5, 3, 3, 1, 5, 3, 5, 1, 5, 5, 3, 4, 2, 5, 5, 1, 5, 5, 4, 1, 5, 4, 5, 5, 1, 5, 5, 2, 1, 1, 1, 4, 5, 5, 1, 4, 5, 4, 5, 1, 5, 5, 5, 1, 1, 1, 5, 3, 5, 5, 1, 5, 5, 4, 4, 5, 3, 5, 1, 5, 1, 5, 5, 1, 1, 1, 2, 5, 2, 5, 4, 4, 1, 3, 5, 5, 5, 5, 1, 4, 5, 4, 2, 4, 1, 1, 1, 4, 5, 4, 5, 5, 2, 2, 3, 5, 5, 4, 1, 5, 5, 5, 5, 3, 5, 2, 1, 4, 4, 5, 5, 3, 5, 5, 1, 5, 5, 5, 5, 5, 4, 5, 5, 3, 4, 5, 5, 5, 1, 1, 3, 5, 2, 4, 5, 5, 2, 1, 5, 5, 1, 5, 5, 4, 5, 4, 5, 1, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 4, 1, 4, 4, 4, 5, 5, 5, 3, 5, 5, 4, 5, 4, 5, 1, 1, 5, 1, 2, 5, 4, 4, 5, 1, 5, 5, 5, 1, 5, 5, 5, 1, 1, 5, 4, 1, 5, 3, 2, 4, 5, 5, 1, 5, 4, 5, 5, 5, 5, 1, 3, 3, 1, 5, 5, 3, 4, 3, 5, 5, 1, 5, 4, 4, 5, 5, 5, 3, 5, 1, 3, 5, 1, 1, 5, 1, 5, 4, 2, 1, 1, 5, 4, 5, 4, 5, 5, 5, 5, 2, 2, 5, 3, 3, 1, 4, 5, 4, 1, 3, 1, 5, 4, 1, 5, 5, 2, 5, 5, 2, 5, 5, 4, 1, 5, 5, 5, 4, 3, 1, 5, 5, 3, 4, 3, 1, 1, 1, 5, 4, 5, 4, 5, 5, 5, 1, 5, 5, 5, 5, 5, 4, 1, 4, 4, 5, 2, 4, 5, 5, 5, 5, 3, 3, 5, 1, 3, 5, 4, 1, 1, 5, 1, 4, 4, 5, 2, 1, 4, 5, 2, 5, 1, 5, 4, 5, 2, 1, 4, 1, 5, 5, 5, 2, 5, 5, 4, 1, 4, 1, 4, 1, 5, 3, 5, 5, 3, 3, 5, 3, 4, 1, 1, 5, 2, 4, 5, 4, 3, 5, 1, 4, 5, 2, 5, 4, 5, 5, 1, 4, 1, 4, 5, 5, 3, 5, 4, 5, 5, 1, 5, 5, 2, 5, 3, 4, 5, 5, 5, 4, 5, 2, 1, 4, 4, 5, 3, 4, 4, 5, 5, 5, 3, 5, 5, 5, 4, 4, 5, 1, 2, 4, 5, 5, 3, 3, 5, 4, 1, 4, 4, 5, 4, 2, 4, 4, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 1, 4, 3, 5, 5, 5, 1, 5, 1, 5, 5, 4, 4, 3, 1, 5, 1, 1, 5, 5, 5, 5, 5, 5, 5, 1, 3, 5, 5, 5, 4, 5, 1, 5, 3, 1, 5, 2, 4, 4, 4, 5, 5, 4, 5, 5, 5, 2, 5, 1, 5, 1, 2, 1, 1, 1, 5, 5, 4, 5, 4, 3, 5, 4, 5, 4, 5, 5, 4, 1, 5, 5, 5, 5, 2, 5, 4, 3, 4, 5, 5, 5, 5, 3, 5, 5, 5, 1, 5, 3, 5, 1, 1, 5, 5, 1, 1, 1, 5, 4, 3, 5, 1, 4, 5, 5, 5, 5, 1, 1, 5, 1, 3, 2, 4, 5, 4, 5, 5, 5, 5, 5, 1, 5, 2, 4, 4, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 2, 4, 5, 5, 4, 2, 3, 3, 4, 5, 5, 1, 1, 3, 5, 5, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 1, 3, 2, 4, 4, 5, 5, 5, 3, 4, 5, 3, 5, 1, 5, 5, 5, 4, 1, 4, 2, 5, 5, 4, 4, 1, 5, 1, 5, 4, 5, 4, 4, 5, 5, 4, 3, 2, 1, 1, 3, 4, 5, 4, 3, 5, 4, 2, 5, 4, 4, 1, 5, 5, 4, 1, 5, 5, 5, 5, 5, 1, 5, 1, 1, 2, 5, 5, 5, 1, 1, 1, 5, 4, 5, 1, 5, 4, 1, 5, 2, 5, 3, 4, 5, 1, 5, 4, 5, 5, 5, 4, 5, 2, 5, 5, 5, 5, 5, 5, 3, 4, 1, 5, 5, 4, 1, 4, 3, 4, 5, 4, 5, 4, 2, 4, 5, 4, 2, 5, 3, 1, 1, 5, 5, 4, 3, 3, 5, 1, 5, 5, 4, 5, 5, 5, 2, 1, 4, 5, 5, 1, 5, 1, 5, 5, 5, 5, 1, 2, 5, 1, 4, 5, 5, 3, 1, 5, 2, 5, 1, 5, 3, 5, 4, 5, 5, 5, 1, 2, 5, 4, 5, 2, 1, 1, 4, 5, 5, 5, 5, 5, 5, 5, 2, 5, 1, 5, 5, 1, 1, 5, 1, 1, 4, 4, 1, 5, 1, 4, 4, 5, 5, 1, 1, 4, 2, 2, 5, 4, 3, 5, 3, 4, 1, 2, 1, 5, 2, 5, 5, 5, 5, 5, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 1, 3, 4, 5, 4, 3, 3, 2, 3, 5, 5, 5, 5, 5, 1, 4, 4, 4, 5, 3, 4, 5, 5, 5, 3, 3, 1, 5, 5, 1, 1, 4, 1, 5, 3, 4, 3, 3, 4, 1, 1, 5, 5, 5, 4, 1, 5, 5, 3, 3, 4, 5, 4, 1, 5, 4, 3, 3, 4, 1, 1, 2, 4, 4, 4, 5, 4, 5, 4, 5, 4, 5, 2, 3, 5, 4, 5, 4, 5, 1, 5, 2, 1, 5, 4, 5, 5, 5, 4, 4, 5, 5, 4, 5, 2, 5, 3, 5, 1, 5, 4, 5, 1, 5, 2, 5, 2, 1, 5, 5, 5, 5, 2, 3, 5, 1, 1, 5, 5, 5, 5, 4, 5, 1, 1, 4, 1, 4, 5, 3, 5, 1, 5, 5, 3, 3, 4, 1, 5, 2, 5, 4, 4, 5, 2, 5, 5, 3, 5, 2, 4, 5, 3, 3, 1, 5, 2, 5, 3, 5, 5, 5, 3, 5, 4, 3, 1, 3, 4, 4, 5, 3, 5, 5, 2, 5, 2, 5, 2, 5, 5, 5, 5, 5, 5, 4, 5, 2, 4, 5, 3, 5, 5, 1, 1, 4, 5, 1, 1, 5, 5, 5, 4, 3, 3, 5, 1, 1, 5, 4, 1, 1, 4, 1, 1, 1, 1, 5, 1, 5, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 1, 3, 5, 3, 5, 1, 4, 5, 5, 5, 4, 5, 4, 1, 5, 1, 5, 1, 2, 4, 2, 4, 3, 1, 4, 1, 4, 4, 3, 1, 5, 4, 5, 1, 3, 3, 5, 5, 5, 4, 5, 5, 5, 5, 5, 4, 5, 4, 4, 5, 2, 5, 5, 4, 5, 1, 1, 5, 3, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 3, 4, 5, 4, 5, 4, 5, 2, 1, 5, 5, 5, 1, 5, 5, 4, 5, 4, 5, 4, 5, 1, 3, 4, 4, 5, 2, 5, 5, 5, 5, 5, 1, 2, 1, 5, 5, 4, 5, 4, 4, 5, 5, 5, 2, 5, 5, 2, 5, 5, 1, 5, 5, 4, 5, 1, 4, 5, 5, 5, 5, 4, 3, 5, 5, 2, 5, 4, 5, 5, 3, 5, 3, 5, 4, 1, 4, 3, 4, 3, 5, 5, 5, 5, 3, 3, 5, 4, 1, 1, 4, 4, 4, 5, 5, 3, 5, 5, 1, 5, 5, 5, 5, 5, 1, 5, 5, 4, 5, 2, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 4, 1, 4, 5, 5, 4, 3, 2, 2, 1, 3, 5, 5, 1, 4, 1, 3, 1, 5, 1, 5, 5, 4, 5, 5, 5, 5, 1, 4, 5, 5, 3, 4, 1, 1, 5, 2, 5, 5, 5, 2, 5, 5, 2, 4, 5, 5, 5, 3, 5, 4, 1, 3, 5, 1, 5, 5, 1, 3, 1, 4, 5, 2, 1, 4, 5, 5, 1, 5, 1, 5, 1, 5, 3, 1, 5, 5, 1, 5, 5, 5, 5, 2, 5, 5, 5, 1, 4, 4, 5, 3, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 1, 5, 2, 4, 5, 5, 4, 5, 3, 5, 1, 5, 5, 3, 4, 3, 5, 5, 5, 4, 4, 5, 4, 1, 5, 5, 1, 4, 3, 5, 5, 4, 5, 4, 5, 1, 1, 5, 3, 4, 5, 4, 3, 5, 5, 1, 3, 5, 4, 5, 5, 4, 2, 5, 5, 5, 2, 1, 1, 4, 5, 3, 5, 1, 4, 1, 3, 5, 5, 5, 1, 2, 3, 4, 5, 4, 4, 3, 5, 2, 4, 3, 4, 5, 5, 1, 5, 3, 4, 5, 5, 5, 1, 5, 4, 3, 2, 4, 4, 5, 4, 1, 5, 2, 4, 4, 3, 4, 4, 5, 1, 5, 3, 5, 5, 5, 5, 5, 5, 4, 3, 5, 4, 4, 5, 5, 1, 5, 2, 3, 4, 5, 4, 5, 4, 5, 3, 2, 5, 5, 1, 5, 5, 2, 5, 5, 4, 5, 5, 4, 5, 5, 4, 1, 4, 5, 5, 1, 5, 1, 3, 2, 5, 1, 5, 5, 4, 3, 5, 5, 1, 5, 4, 5, 5, 4, 4, 1, 5, 5, 5, 4, 5, 1, 5, 5, 5, 5, 1, 4, 3, 3, 5, 5, 5, 4, 4, 4, 5, 1, 5, 3, 3, 4, 5, 5, 3, 5, 3, 5, 1, 4, 5, 1, 5, 4, 5, 1, 5, 3, 4, 3, 5, 4, 4, 3, 5, 5, 5, 5, 5, 5, 5, 4, 5, 1, 4, 1, 1, 5, 4, 5, 1, 4, 5, 5, 5, 5, 2, 5, 5, 5, 1, 1, 5, 5, 5, 2, 2, 5, 1, 4, 5, 5, 1, 5, 1, 1, 4, 5, 5, 1, 3, 5, 3, 2, 1, 5, 1, 5, 5, 5, 1, 4, 5, 5, 5, 4, 5, 5, 2, 3, 5, 5, 5, 5, 2, 4, 5, 3, 4, 5, 1, 5, 5, 1, 4, 5, 5, 5, 5, 5, 3, 5, 5, 5, 4, 5, 5, 5, 4, 4, 5, 1, 5, 2, 3, 2, 4, 1, 4, 3, 3, 5, 1, 1, 5, 4, 5, 3, 4, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 4, 4, 5, 1, 4, 5, 5, 5, 5, 2, 1, 4, 5, 4, 2, 5, 5, 1, 5, 5, 1, 5, 1, 5, 5, 4, 5, 5, 5, 5, 1, 5, 1, 5, 5, 1, 5, 5, 3, 5, 1, 5, 5, 5, 3, 4, 1, 5, 3, 4, 5, 4, 5, 1, 1, 5, 4, 5, 5, 5, 4, 5, 3, 4, 3, 3, 1, 1, 1, 5, 3, 4, 5, 5, 1, 5, 1, 5, 4, 5, 5, 4, 1, 1, 1, 5, 4, 5, 5, 5, 3, 1, 4, 5, 5, 4, 4, 5, 4, 5, 1, 3, 5, 4, 1, 4, 5, 4, 4, 4, 5, 5, 1, 1, 5, 5, 5, 1, 4, 5, 3, 5, 5, 1, 4, 4, 5, 5, 5, 1, 4, 4, 5, 5, 5, 1, 5, 5, 1, 2, 5, 4, 4, 1, 5, 5, 5, 5, 3, 5, 2, 1, 5, 5, 4, 1, 3, 4, 5, 1, 4, 5, 3, 1, 1, 1, 5, 1, 5, 1, 5, 1, 2, 5, 3, 3, 5, 1, 5, 4, 3, 4, 5, 2, 4, 4, 3, 5, 4, 1, 5, 5, 1, 4, 5, 5, 5, 5, 5, 5, 5, 1, 1, 3, 5, 4, 5, 4, 5, 4, 4, 2, 4, 5, 5, 4, 5, 2, 4, 5, 1, 2, 5, 5, 1, 5, 5, 3, 5, 5, 2, 5, 5, 5, 2, 5, 5, 4, 5, 5, 4, 4, 4, 5, 3, 5, 4, 3, 1, 4, 1, 5, 5, 5, 1, 1, 1, 5, 4, 5, 4, 1, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 3, 5, 1, 3, 5, 1, 5, 1, 5, 4, 5, 5, 4, 5, 5, 5, 1, 5, 5, 1, 5, 5, 1, 5, 1, 1, 5, 5, 5, 5, 3, 5, 3, 1, 5, 5, 4, 2, 4, 3, 2, 5, 5, 4, 1, 1, 5, 5, 1, 1, 5, 4, 3, 4, 4, 5, 5, 3, 4, 1, 5, 5, 5, 4, 5, 5, 5, 5, 3, 3, 4, 2, 4, 4, 5, 4, 5, 1, 1, 4, 4, 1, 4, 4, 4, 5, 4, 4, 5, 1, 4, 3, 5, 5, 5, 5, 5, 1, 1, 5, 3, 1, 5, 5, 4, 3, 1, 5, 3, 5, 3, 4, 4, 4, 5, 5, 5, 3, 5, 5, 4, 1, 5, 5, 5, 5, 4, 5, 5, 1, 4, 1, 5, 5, 2, 5, 5, 4, 4, 5, 5, 5, 5, 5, 5, 3, 1, 4, 3, 1, 2, 5, 5, 1, 4, 4, 5, 5, 1, 1, 5, 4, 5, 3, 5, 5, 5, 1, 5, 5, 5, 4, 5, 5, 5, 5, 4, 5, 4, 5, 4, 5, 2, 1, 5, 2, 1, 4, 4, 4, 3, 5, 4, 1, 5, 5, 5, 5, 1, 1, 5, 2, 3, 4, 2, 3, 4, 5, 5, 5, 4, 3, 1, 5, 5, 5, 4, 4, 5, 5, 5, 5, 4, 5, 4, 5, 4, 5, 5, 5, 4, 3, 4, 4, 1, 5, 5, 5, 2, 1, 3, 4, 3, 5, 1, 4, 5, 5, 3, 1, 5, 1, 5, 4, 1, 5, 3, 5, 5, 2, 4, 5, 5, 3, 4, 4, 4, 4, 5, 5, 4, 4, 5, 5, 5, 5, 1, 5, 1, 4, 5, 5, 5, 5, 5, 4, 1, 4, 5, 5, 4, 3, 5, 1, 4, 5, 3, 1, 4, 1, 5, 1, 5, 5, 3, 1, 1, 4, 4, 3, 5, 4, 5, 5, 4, 5, 5, 1, 5, 2, 5, 5, 1, 2, 4, 3, 5, 5, 1, 4, 5, 5, 2, 5, 5, 4, 5, 4, 5, 4, 5, 1, 5, 5, 5, 1, 4, 5, 4, 5, 5, 4, 3, 2, 5, 3, 5, 5, 5, 4, 5, 5, 5, 3, 5, 1, 5, 1, 3, 4, 5, 5, 3, 4, 5, 1, 5, 4, 5, 4, 4, 4, 4, 5, 3, 3, 5, 1, 5, 5, 3, 5, 5, 5, 1, 4, 5, 4, 5, 5, 5, 4, 5, 5, 3, 5, 4, 1, 5, 5, 4, 5, 4, 5, 4, 3, 2, 4, 5, 4, 5, 3, 1, 5, 5, 4, 5, 3, 5, 1, 4, 5, 3, 4, 4, 1, 5, 5, 5, 1, 3, 1, 5, 4, 5, 4, 5, 3, 5, 4, 5, 5, 5, 4, 2, 5, 4, 4, 2, 3, 3, 5, 3, 5, 5, 1, 3, 5, 5, 5, 1, 5, 4, 3, 5, 5, 4, 4, 4, 4, 1, 5, 5, 5, 3, 4, 1, 1, 1, 1, 1, 1, 5, 5, 4, 1, 1, 4, 1, 2, 4, 5, 1, 4, 5, 5, 4, 3, 5, 3, 5, 5, 1, 5, 1, 5, 5, 3, 5, 1, 5, 1, 5, 4, 5, 5, 4, 5, 5, 4, 5, 4, 4, 5, 2, 1, 5, 4, 5, 4, 5, 3, 5, 5, 4, 4, 5, 5, 5, 5, 3, 5, 1, 5, 5, 5, 1, 1, 5, 5, 4, 5, 4, 5, 5, 5, 5, 4, 1, 5, 5, 5, 1, 1, 4, 5, 4, 3, 3, 4, 5, 4, 4, 5, 5, 4, 2, 4, 5, 1, 2, 4, 3, 5, 5, 5, 1, 1, 5, 5, 4, 5, 5, 5, 3, 1, 1, 1, 5, 5, 3, 4, 5, 1, 5, 1, 5, 2, 5, 3, 5, 1, 5, 1, 4, 5, 5, 5, 4, 1, 5, 1, 5, 4, 5, 5, 3, 5, 5, 1, 2, 5, 5, 2, 5, 5, 3, 4, 5, 4, 5, 4, 1, 1, 5, 5, 5, 5, 4, 4, 5, 5, 4, 5, 4, 4, 5, 5, 5, 5, 5, 1, 5, 5, 4, 5, 1, 4, 5, 5, 4, 5, 5, 5, 1, 5, 5, 5, 1, 1, 4, 5, 4, 5, 4, 1, 5, 5, 4, 4, 5, 5, 5, 3, 5, 5, 1, 1, 3, 5, 3, 5, 3, 5, 5, 5, 1, 3, 3, 2, 5, 3, 3, 5, 5, 4, 1, 1, 5, 5, 4, 3, 5, 1, 3, 5, 5, 1, 5, 5, 5, 5, 3, 4, 4, 4, 5, 4, 2, 5, 5, 5, 2, 3, 1, 5, 4, 4, 1, 5, 5, 2, 5, 3, 5, 4, 5, 5, 3, 3, 1, 5, 3, 5, 5, 3, 4, 1, 1, 5, 1, 5, 3, 5, 5, 1, 3, 1, 4, 4, 3, 5, 1, 5, 4, 1, 1, 5, 1, 3, 5, 5, 2, 3, 5, 5, 3, 5, 1, 5, 5, 1, 4, 1, 4, 3, 1, 2, 5, 5, 4, 1, 1, 1, 5, 5, 5, 4, 5, 5, 4, 5, 1, 5, 4, 5, 3, 1, 5, 4, 5, 5, 1, 1, 5, 5, 1, 1, 5, 4, 1, 1, 1, 1, 3, 5, 3, 5, 4, 5, 1, 4, 1, 5, 2, 5, 4, 3, 3, 1, 3, 4, 4, 4, 1, 5, 5, 5, 1, 2, 5, 2, 3, 3, 5, 4, 1, 1, 5, 5, 5, 1, 5, 4, 1, 2, 5, 4, 4, 1, 5, 2, 4, 4, 5, 5, 1, 5, 5, 5, 5, 5, 5, 1, 4, 4, 5, 5, 4, 5, 5, 5, 4, 5, 3, 5, 5, 5, 5, 1, 2, 4, 3, 1, 1, 1, 5, 1, 5, 5, 4, 1, 4, 1, 1, 5, 1, 4, 2, 4, 5, 5, 5, 4, 5, 5, 4, 5, 1, 4, 5, 4, 5, 5, 3, 4, 2, 3, 3, 5, 4, 1, 1, 5, 5, 1, 5, 3, 2, 3, 1, 4, 1, 4, 5, 3, 5, 5, 1, 5, 4, 1, 1, 3, 3, 5, 5, 4, 4, 5, 5, 5, 5, 1, 3, 5, 5, 3, 3, 4, 5, 5, 4, 1, 5, 5, 1, 4, 5, 3, 3, 1, 5, 3, 2, 5, 4, 2, 1, 3, 4, 5, 3, 1, 1, 4, 1, 2, 1, 1, 5, 5, 4, 3, 5, 2, 4, 5, 1, 2, 4, 1, 5, 4, 4, 5, 5, 4, 2, 5, 1, 4, 5, 5, 5, 5, 1, 5, 5, 1, 5, 1, 2, 1, 5, 5, 1, 2, 5, 1, 4, 4, 5, 5, 2, 1, 5, 5, 4, 1, 5, 3, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 1, 5, 4, 1, 5, 1, 4, 5, 1, 5, 1, 5, 5, 3, 3, 3, 3, 5, 5, 1, 5, 4, 3, 5, 4, 5, 1, 1, 4, 1, 5, 5, 5, 1, 5, 2, 4, 5, 5, 4, 4, 4, 3, 4, 5, 4, 5, 1, 5, 5, 5, 4, 5, 5, 3, 5, 5, 5, 5, 5, 5, 4, 1, 4, 5, 5, 5, 5, 2, 3, 5, 4, 5, 5, 3, 5, 1, 3, 5, 3, 5, 5, 5, 1, 5, 5, 1, 5, 5, 5, 5, 5, 3, 2, 5, 1, 1, 5, 5, 5, 3, 5, 2, 1, 4, 4, 4, 5, 5, 5, 4, 5, 4, 5, 1, 5, 4, 4, 5, 5, 4, 1, 4, 4, 4, 1, 3, 5, 1, 5, 5, 4, 5, 5, 5, 5, 2, 5, 5, 5, 4, 5, 3, 1, 3, 5, 4, 5, 1, 4, 1, 5, 4, 4, 2, 5, 4, 4, 1, 5, 2, 4, 4, 4, 2, 5, 5, 5, 5, 5, 3, 3, 4, 5, 5, 4, 4, 4, 4, 1, 3, 5, 5, 5, 5, 4, 1, 4, 5, 2, 1, 5, 5, 2, 4, 5, 5, 5, 3, 1, 3, 3, 1, 5, 5, 1, 3, 4, 5, 5, 5, 1, 5, 2, 4, 4, 5, 3, 5, 5, 1, 3, 5, 2, 3, 1, 4, 1, 5, 4, 4, 5, 5, 1, 1, 4, 5, 3, 3, 5, 4, 5, 4, 2, 5, 5, 4, 5, 5, 3, 4, 5, 4, 3, 3, 1, 5, 4, 5, 4, 1, 5, 5, 5, 5, 3, 5, 5, 5, 1, 5, 3, 4, 5, 5, 5, 1, 3, 5, 4, 3, 1, 5, 5, 4, 5, 5, 4, 2, 3, 4, 1, 1, 5, 4, 1, 3, 4, 2, 1, 5, 5, 1, 5, 5, 5, 4, 4, 5, 3, 1, 4, 2, 1, 4, 1, 4, 5, 5, 4, 1, 2, 5, 4, 1, 5, 5, 1, 1, 1, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 3, 1, 1, 5, 5, 5, 5, 1, 2, 4, 4, 5, 4, 1, 5, 5, 2, 3, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 3, 5]
list2=[2, 5, 5, 5, 3, 5, 5, 5, 5, 4, 2, 5, 3, 5, 5, 5, 5, 5, 3, 1, 2, 1, 5, 1, 1, 5, 5, 5, 5, 1, 1, 5, 5, 5, 3, 1, 5, 5, 2, 5, 2, 1, 2, 1, 4, 4, 1, 1, 5, 4, 5, 1, 4, 5, 5, 5, 5, 5, 1, 5, 2, 3, 3, 3, 5, 5, 1, 1, 3, 5, 5, 5, 5, 4, 2, 2, 5, 1, 3, 5, 5, 5, 4, 5, 1, 4, 4, 4, 4, 5, 4, 1, 5, 4, 5, 5, 3, 5, 3, 5, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 1, 3, 1, 5, 5, 5, 5, 1, 4, 5, 1, 4, 4, 5, 5, 5, 5, 3, 5, 4, 5, 4, 5, 5, 4, 2, 5, 1, 4, 5, 5, 4, 5, 1, 5, 5, 5, 5, 3, 5, 5, 3, 5, 5, 5, 5, 5, 4, 5, 1, 5, 3, 1, 4, 5, 1, 1, 1, 4, 4, 4, 5, 5, 5, 5, 5, 5, 4, 4, 5, 1, 1, 5, 5, 5, 5, 5, 1, 1, 5, 2, 5, 5, 5, 4, 4, 1, 5, 3, 5, 5, 5, 2, 5, 2, 3, 1, 5, 5, 5, 4, 4, 5, 5, 5, 5, 5, 5, 4, 4, 5, 4, 5, 1, 4, 4, 5, 1, 3, 5, 5, 5, 5, 5, 5, 1, 3, 5, 5, 5, 1, 1, 2, 4, 1, 4, 5, 3, 1, 5, 1, 5, 4, 4, 5, 3, 4, 1, 1, 1, 4, 2, 5, 5, 5, 4, 3, 5, 1, 1, 5, 5, 5, 5, 2, 5, 5, 5, 4, 5, 5, 1, 5, 5, 2, 5, 5, 5, 1, 2, 5, 2, 5, 5, 3, 1, 1, 4, 4, 5, 5, 3, 5, 4, 5, 5, 2, 5, 2, 2, 1, 5, 3, 5, 2, 5, 5, 2, 2, 5, 5, 5, 4, 5, 5, 5, 1, 4, 5, 5, 5, 4, 3, 4, 1, 4, 5, 4, 3, 3, 4, 5, 5, 5, 1, 4, 5, 2, 5, 5, 4, 1, 5, 5, 5, 5, 3, 5, 1, 4, 4, 1, 4, 1, 5, 5, 4, 4, 1, 5, 5, 3, 1, 5, 5, 5, 5, 1, 1, 5, 5, 1, 4, 5, 2, 4, 5, 5, 1, 4, 4, 4, 5, 5, 4, 5, 5, 4, 4, 5, 5, 5, 4, 2, 4, 1, 1, 5, 1, 4, 1, 5, 1, 1, 4, 5, 3, 4, 5, 5, 5, 5, 4, 5, 2, 5, 4, 5, 5, 4, 5, 5, 5, 1, 4, 3, 1, 5, 4, 5, 5, 1, 5, 5, 5, 4, 2, 1, 5, 4, 4, 3, 4, 5, 5, 5, 4, 5, 3, 5, 1, 1, 1, 5, 4, 5, 5, 5, 5, 1, 5, 4, 5, 4, 5, 4, 4, 5, 5, 3, 5, 3, 5, 4, 1, 2, 5, 5, 5, 2, 4, 5, 5, 5, 5, 5, 5, 5, 3, 5, 4, 5, 5, 1, 1, 5, 5, 3, 5, 1, 5, 5, 4, 5, 5, 1, 5, 5, 5, 5, 2, 1, 4, 5, 3, 5, 1, 5, 1, 5, 5, 5, 2, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 4, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 1, 4, 4, 5, 4, 4, 5, 5, 4, 4, 4, 4, 5, 3, 5, 5, 1, 5, 5, 5, 5, 1, 3, 5, 4, 4, 5, 5, 5, 5, 1, 3, 5, 3, 5, 1, 4, 5, 2, 1, 5, 4, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 4, 3, 3, 5, 4, 1, 1, 5, 1, 5, 3, 4, 5, 2, 5, 5, 2, 5, 5, 3, 5, 5, 5, 3, 5, 5, 4, 5, 5, 5, 5, 1, 5, 1, 5, 2, 5, 4, 5, 4, 5, 5, 4, 5, 5, 1, 4, 3, 5, 5, 3, 5, 1, 5, 5, 1, 4, 4, 5, 1, 4, 2, 5, 1, 1, 5, 5, 1, 5, 5, 4, 3, 4, 1, 3, 5, 4, 4, 5, 5, 5, 3, 1, 5, 5, 5, 5, 4, 4, 5, 4, 5, 5, 1, 5, 4, 5, 1, 5, 5, 5, 4, 1, 5, 2, 4, 5, 5, 4, 1, 5, 4, 3, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 1, 2, 4, 2, 2, 3, 3, 5, 5, 3, 5, 3, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 2, 1, 3, 5, 5, 4, 4, 5, 5, 1, 5, 5, 1, 5, 2, 5, 2, 5, 5, 5, 4, 5, 5, 2, 2, 5, 3, 1, 1, 5, 2, 5, 1, 4, 1, 5, 4, 5, 5, 5, 5, 4, 3, 4, 5, 4, 5, 5, 4, 5, 3, 5, 4, 1, 5, 1, 4, 4, 5, 3, 1, 5, 5, 5, 3, 3, 5, 5, 2, 5, 1, 4, 4, 5, 5, 4, 5, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 3, 4, 3, 5, 5, 4, 1, 5, 4, 4, 5, 5, 4, 5, 4, 5, 5, 2, 5, 1, 5, 5, 5, 5, 1, 3, 5, 5, 4, 1, 1, 5, 4, 5, 4, 5, 5, 1, 5, 5, 5, 5, 4, 5, 1, 5, 4, 5, 5, 5, 4, 2, 5, 2, 1, 3, 5, 1, 5, 5, 5, 3, 5, 3, 5, 3, 5, 1, 3, 5, 5, 1, 1, 5, 5, 5, 5, 1, 1, 3, 5, 5, 2, 5, 1, 3, 3, 5, 5, 1, 5, 5, 5, 3, 4, 5, 4, 4, 5, 2, 2, 3, 5, 2, 3, 1, 3, 5, 3, 4, 4, 1, 2, 4, 5, 4, 4, 5, 4, 5, 5, 5, 4, 3, 5, 4, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 4, 3, 5, 1, 5, 1, 4, 5, 5, 5, 1, 2, 1, 5, 5, 5, 4, 5, 4, 5, 5, 5, 5, 1, 5, 1, 5, 5, 5, 3, 5, 5, 4, 1, 5, 5, 5, 1, 5, 5, 5, 5, 5, 4, 5, 1, 5, 5, 1, 3, 1, 5, 4, 5, 5, 5, 2, 5, 5, 5, 3, 5, 4, 4, 4, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 1, 1, 5, 2, 5, 1, 2, 5, 5, 3, 5, 1, 4, 4, 5, 5, 4, 5, 4, 5, 4, 5, 1, 3, 2, 1, 4, 5, 1, 3, 5, 5, 3, 5, 4, 5, 5, 5, 3, 5, 4, 5, 5, 5, 5, 5, 2, 4, 5, 4, 5, 5, 5, 4, 5, 2, 5, 5, 5, 3, 5, 5, 4, 1, 5, 4, 4, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5, 1, 2, 2, 5, 5, 4, 5, 1, 5, 5, 4, 2, 5, 1, 2, 4, 5, 1, 1, 5, 5, 4, 5, 5, 4, 5, 4, 2, 4, 5, 3, 5, 5, 5, 5, 5, 4, 3, 3, 4, 4, 5, 5, 1, 1, 5, 5, 1, 5, 5, 5, 5, 5, 4, 4, 5, 1, 4, 2, 5, 1, 1, 5, 4, 5, 1, 3, 2, 2, 5, 3, 4, 5, 5, 1, 1, 5, 5, 5, 1, 5, 4, 5, 5, 5, 5, 3, 1, 4, 1, 5, 5, 1, 4, 1, 2, 3, 4, 5, 5, 5, 5, 4, 1, 5, 5, 5, 5, 4, 5, 1, 5, 3, 1, 4, 5, 5, 5, 1, 4, 4, 5, 5, 5, 3, 1, 5, 2, 5, 5, 5, 4, 4, 2, 4, 2, 5, 5, 1, 4, 1, 4, 4, 5, 5, 5, 1, 4, 5, 3, 5, 5, 5, 5, 5, 5, 5, 4, 4, 5, 3, 3, 5, 1, 4, 5, 5, 5, 1, 5, 5, 1, 5, 2, 1, 3, 4, 3, 2, 5, 4, 1, 5, 4, 5, 2, 5, 2, 3, 1, 4, 5, 5, 5, 1, 2, 4, 1, 3, 5, 5, 4, 2, 5, 5, 2, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 4, 5, 5, 1, 5, 1, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 1, 3, 4, 4, 1, 4, 1, 5, 4, 2, 3, 1, 3, 1, 4, 1, 5, 5, 3, 1, 1, 1, 3, 1, 5, 1, 5, 5, 5, 4, 5, 4, 5, 5, 5, 5, 5, 3, 4, 1, 5, 5, 5, 5, 5, 4, 3, 5, 4, 5, 4, 4, 4, 5, 5, 1, 4, 1, 1, 5, 3, 5, 3, 5, 5, 5, 1, 5, 5, 5, 1, 1, 5, 3, 5, 1, 5, 5, 5, 5, 5, 1, 2, 4, 4, 5, 1, 4, 5, 4, 4, 4, 5, 5, 3, 5, 5, 1, 5, 5, 4, 1, 4, 5, 1, 5, 5, 4, 1, 5, 1, 4, 3, 1, 4, 5, 1, 5, 3, 2, 5, 5, 2, 5, 5, 4, 5, 5, 1, 4, 5, 3, 2, 5, 2, 5, 5, 5, 5, 4, 5, 5, 5, 1, 5, 2, 1, 5, 5, 5, 3, 1, 4, 5, 5, 5, 1, 1, 3, 3, 1, 5, 5, 3, 5, 5, 4, 1, 5, 4, 5, 5, 4, 5, 5, 3, 1, 5, 1, 1, 3, 4, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 4, 5, 1, 5, 4, 5, 4, 5, 5, 3, 5, 3, 1, 4, 5, 4, 4, 1, 5, 5, 5, 4, 3, 1, 4, 5, 5, 5, 1, 3, 1, 5, 2, 4, 5, 4, 5, 5, 5, 5, 4, 3, 5, 5, 5, 5, 1, 5, 3, 4, 4, 5, 4, 4, 5, 1, 5, 4, 5, 5, 4, 5, 3, 2, 4, 5, 5, 5, 4, 5, 1, 3, 5, 5, 5, 5, 5, 4, 3, 5, 4, 5, 1, 4, 4, 4, 1, 4, 1, 5, 4, 4, 4, 4, 3, 5, 5, 5, 5, 5, 1, 5, 5, 3, 5, 5, 1, 5, 1, 5, 2, 5, 5, 4, 5, 5, 5, 4, 1, 4, 4, 3, 4, 3, 2, 5, 5, 5, 5, 4, 4, 5, 4, 5, 1, 4, 5, 5, 5, 1, 2, 5, 5, 1, 4, 4, 3, 5, 5, 1, 2, 5, 5, 5, 1, 5, 5, 3, 5, 5, 3, 4, 5, 1, 4, 5, 3, 4, 5, 1, 4, 5, 4, 5, 5, 4, 1, 4, 2, 1, 5, 5, 3, 4, 1, 5, 5, 4, 4, 3, 3, 5, 3, 5, 5, 5, 2, 5, 5, 3, 5, 5, 1, 1, 3, 4, 1, 1, 4, 2, 5, 4, 2, 4, 3, 3, 5, 4, 4, 5, 4, 4, 2, 5, 5, 5, 1, 5, 4, 5, 1, 5, 5, 5, 5, 5, 5, 5, 4, 5, 2, 5, 1, 4, 5, 1, 4, 1, 5, 4, 5, 5, 5, 5, 5, 4, 2, 5, 5, 3, 5, 3, 1, 4, 1, 5, 1, 4, 5, 4, 4, 5, 5, 1, 4, 5, 1, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 4, 3, 5, 3, 5, 5, 4, 4, 5, 5, 4, 3, 3, 5, 5, 2, 4, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 2, 5, 2, 1, 4, 1, 1, 5, 4, 5, 5, 3, 5, 1, 5, 1, 5, 5, 5, 3, 5, 5, 1, 5, 4, 3, 5, 5, 4, 4, 5, 2, 5, 5, 4, 5, 1, 5, 1, 4, 5, 5, 4, 4, 5, 3, 5, 5, 5, 1, 5, 5, 5, 5, 1, 1, 1, 3, 4, 5, 4, 5, 3, 5, 3, 1, 4, 3, 4, 4, 2, 5, 4, 5, 5, 2, 5, 2, 4, 5, 5, 3, 5, 3, 5, 5, 1, 4, 3, 1, 3, 5, 1, 5, 5, 4, 4, 4, 1, 5, 4, 1, 5, 3, 5, 5, 1, 1, 4, 5, 4, 5, 1, 5, 5, 4, 2, 5, 5, 2, 5, 5, 4, 1, 4, 4, 4, 5, 4, 3, 5, 4, 1, 5, 4, 3, 4, 2, 5, 5, 5, 5, 5, 5, 5, 2, 4, 5, 1, 2, 4, 5, 4, 4, 5, 4, 5, 5, 4, 5, 1, 5, 5, 5, 3, 5, 5, 5, 4, 4, 2, 4, 5, 5, 2, 5, 5, 1, 5, 5, 4, 5, 5, 4, 1, 5, 4, 1, 4, 4, 1, 3, 5, 4, 3, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 1, 4, 5, 5, 3, 5, 5, 1, 5, 5, 5, 3, 5, 3, 5, 5, 3, 5, 5, 5, 5, 5, 1, 4, 4, 5, 5, 5, 5, 4, 5, 5, 5, 3, 5, 5, 1, 5, 2, 5, 5, 5, 5, 3, 4, 5, 1, 4, 5, 1, 5, 5, 4, 1, 5, 5, 5, 1, 5, 5, 1, 5, 5, 1, 5, 5, 3, 4, 5, 3, 5, 4, 4, 1, 5, 2, 5, 5, 5, 4, 4, 3, 3, 5, 5, 4, 2, 4, 1, 3, 4, 2, 1, 1, 1, 1, 4, 5, 4, 1, 4, 5, 5, 2, 3, 3, 4, 5, 5, 5, 1, 5, 1, 5, 5, 4, 1, 5, 4, 4, 1, 5, 1, 4, 1, 3, 1, 4, 3, 5, 5, 5, 5, 5, 5, 1, 5, 4, 3, 4, 5, 4, 1, 4, 4, 1, 1, 4, 4, 5, 5, 4, 3, 5, 4, 5, 1, 4, 5, 1, 4, 3, 5, 5, 5, 2, 5, 1, 2, 1, 4, 1, 3, 4, 1, 4, 5, 5, 4, 5, 5, 5, 4, 5, 5, 4, 5, 5, 5, 5, 5, 4, 5, 4, 1, 4, 5, 1, 2, 4, 4, 5, 5, 5, 3, 5, 5, 5, 5, 2, 5, 5, 5, 1, 4, 5, 5, 4, 2, 5, 5, 1, 5, 4, 5, 5, 1, 1, 4, 4, 2, 5, 5, 4, 4, 5, 1, 5, 5, 5, 4, 5, 5, 3, 5, 5, 1, 1, 1, 5, 5, 2, 5, 5, 4, 4, 1, 5, 5, 5, 5, 5, 3, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5, 5, 5, 2, 5, 5, 2, 5, 5, 3, 4, 5, 4, 5, 1, 5, 4, 5, 1, 4, 1, 5, 5, 5, 5, 1, 4, 4, 5, 2, 1, 5, 5, 5, 1, 5, 1, 2, 3, 3, 2, 5, 5, 4, 1, 5, 5, 4, 5, 1, 5, 5, 4, 3, 5, 2, 5, 2, 5, 5, 4, 2, 2, 5, 1, 1, 5, 4, 3, 2, 5, 1, 4, 2, 5, 5, 4, 4, 4, 1, 5, 4, 5, 1, 5, 1, 1, 5, 5, 5, 4, 5, 4, 5, 5, 1, 4, 5, 1, 5, 5, 5, 1, 5, 4, 1, 5, 5, 1, 1, 5, 5, 4, 5, 4, 1, 1, 3, 5, 4, 4, 5, 5, 4, 2, 5, 5, 4, 5, 5, 5, 1, 5, 5, 4, 5, 4, 5, 4, 4, 5, 5, 5, 1, 5, 1, 5, 1, 3, 4, 5, 4, 4, 5, 1, 1, 1, 1, 4, 5, 1, 5, 5, 5, 1, 5, 3, 1, 1, 5, 5, 5, 5, 3, 4, 5, 1, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 4, 4, 5, 4, 5, 1, 5, 5, 4, 1, 4, 5, 5, 5, 4, 5, 4, 2, 5, 4, 5, 5, 1, 5, 5, 1, 5, 4, 4, 1, 2, 1, 5, 2, 5, 5, 5, 5, 3, 5, 4, 1, 1, 5, 5, 5, 4, 3, 5, 5, 5, 5, 1, 5, 5, 4, 4, 5, 1, 5, 4, 5, 5, 4, 2, 1, 5, 2, 5, 5, 4, 4, 1, 5, 4, 5, 3, 5, 4, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 2, 5, 4, 5, 2, 4, 1, 1, 4, 1, 4, 2, 5, 3, 3, 5, 5, 5, 5, 5, 5, 4, 3, 1, 5, 5, 5, 5, 4, 5, 3, 5, 1, 4, 5, 5, 1, 1, 1, 2, 5, 2, 5, 5, 1, 5, 4, 5, 5, 3, 3, 5, 5, 1, 5, 3, 1, 5, 5, 5, 4, 3, 3, 5, 3, 5, 1, 3, 5, 5, 2, 4, 5, 4, 4, 4, 4, 1, 2, 5, 5, 5, 4, 3, 1, 4, 1, 5, 5, 2, 5, 5, 5, 5, 4, 5, 1, 1, 4, 1, 5, 1, 4, 5, 4, 5, 5, 3, 4, 4, 3, 1, 3, 5, 4, 1, 3, 5, 5, 1, 1, 5, 4, 5, 4, 5, 4, 1, 5, 3, 5, 5, 5, 4, 3, 5, 1, 5, 5, 3, 3, 5, 5, 5, 3, 5, 5, 5, 1, 2, 5, 3, 4, 2, 2, 2, 1, 3, 5, 5, 1, 5, 5, 1, 3, 3, 5, 2, 1, 1, 5, 2, 4, 5, 5, 3, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 3, 1, 3, 5, 5, 5, 2, 2, 4, 3, 1, 3, 4, 2, 1, 4, 1, 2, 5, 4, 4, 4, 4, 1, 5, 5, 1, 4, 5, 5, 2, 5, 4, 3, 5, 4, 1, 1, 1, 5, 4, 2, 4, 1, 4, 5, 4, 5, 2, 5, 5, 5, 5, 5, 1, 3, 5, 5, 2, 5, 1, 5, 5, 5, 1, 5, 3, 5, 5, 4, 5, 5, 5, 4, 4, 3, 3, 5, 2, 5, 5, 4, 4, 4, 3, 4, 2, 5, 1, 5, 4, 3, 5, 5, 5, 2, 3, 2, 5, 5, 1, 2, 4, 5, 3, 4, 5, 4, 2, 1, 5, 5, 4, 1, 3, 5, 5, 3, 4, 1, 5, 1, 5, 5, 5, 1, 4, 5, 5, 5, 2, 4, 1, 5, 4, 1, 3, 3, 5, 1, 5, 4, 5, 5, 2, 5, 5, 5, 4, 5, 1, 1, 5, 5, 5, 5, 5, 1, 5, 4, 1, 5, 1, 4, 4, 3, 5, 1, 5, 1, 4, 5, 5, 5, 1, 5, 5, 4, 3, 5, 5, 2, 5, 5, 5, 1, 5, 1, 5, 5, 3, 4, 2, 1, 1, 5, 1, 4, 3, 3, 4, 5, 5, 2, 5, 4, 3, 4, 5, 5, 5, 5, 4, 5, 5, 4, 1, 5, 5, 1, 1, 5, 1, 3, 1, 5, 5, 1, 5, 1, 2, 4, 1, 5, 4, 5, 1, 5, 4, 1, 5, 1, 5, 4, 1, 5, 5, 4, 5, 5, 1, 5, 5, 5, 3, 4, 4, 1, 3, 5, 3, 5, 3, 5, 1, 5, 5, 5, 5, 4, 5, 5, 5, 1, 3, 5, 5, 5, 4, 4, 1, 4, 5, 1, 3, 3, 2, 1, 5, 1, 3, 5, 5, 5, 5, 1, 4, 5, 5, 5, 5, 5, 5, 4, 4, 5, 3, 5, 5, 4, 3, 5, 3, 4, 5, 3, 5, 5, 1, 4, 5, 1, 1, 5, 5, 4, 4, 3, 2, 5, 5, 5, 4, 5, 4, 5, 3, 4, 1, 4, 5, 4, 5, 2, 1, 5, 5, 1, 5, 5, 5, 5, 3, 5, 3, 2, 4, 1, 1, 4, 2, 5, 3, 4, 3, 5, 5, 5, 5, 3, 5, 5, 5, 1, 5, 5, 1, 3, 5, 4, 4, 5, 5, 5, 5, 5, 5, 5, 4, 4, 5, 5, 4, 5, 1, 5, 5, 3, 1, 2, 5, 5, 4, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 4, 5, 5, 4, 5, 5, 2, 5, 2, 5, 5, 5, 1, 3, 3, 5, 1, 1, 5, 2, 3, 5, 1, 4, 5, 5, 5, 1, 4, 5, 4, 4, 1, 1, 1, 5, 5, 1, 5, 2, 1, 5, 5, 4, 3, 5, 5, 5, 5, 5, 4, 5, 2, 5, 5, 5, 2, 5, 4, 5, 5, 1, 4, 4, 5, 3, 5, 4, 5, 2, 5, 5, 5, 5, 4, 5, 5, 5, 1, 5, 1, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 3, 5, 5, 4, 1, 3, 1, 4, 4, 5, 5, 1, 5, 4, 5, 5, 1, 2, 1, 4, 5, 3, 1, 5, 1, 1, 5, 5, 4, 5, 3, 5, 5, 1, 5, 2, 5, 5, 3, 4, 1, 5, 1, 5, 5, 5, 5, 5, 4, 1, 5, 3, 5, 5, 5, 4, 4, 4, 5, 4, 4, 5, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 2, 5, 5, 5, 5, 1, 1, 2, 5, 4, 5, 5, 5, 4, 4, 5, 4, 5, 5, 1, 5, 4, 5, 5, 1, 4, 5, 4, 5, 2, 5, 4, 3, 3, 1, 5, 5, 1, 5, 5, 4, 5, 5, 5, 4, 5, 3, 5, 5, 5, 4, 5, 3, 5, 5, 5, 4, 5, 5, 5, 5, 1, 5, 2, 5, 4, 5, 5, 5, 5, 5, 1, 5, 2, 2, 5, 4, 5, 1, 5, 1, 1, 5, 5, 4, 5, 5, 3, 4, 5, 1, 1, 1, 3, 4, 5, 1, 5, 5, 1, 1, 5, 5, 5, 5, 4, 5, 1, 5, 5, 4, 5, 5, 1, 3, 1, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 1, 1, 1, 5, 4, 5, 5, 3, 5, 3, 5, 4, 5, 1, 5, 4, 5, 5, 1, 5, 5, 5, 1, 2, 4, 5, 5, 5, 3, 4, 2, 5, 4, 3, 4, 5, 5, 4, 2, 4, 2, 5, 5, 5, 5, 4, 5, 5, 3, 5, 4, 5, 5, 3, 5, 5, 2, 5, 5, 5, 4, 1, 5, 1, 5, 5, 4, 5, 1, 5, 1, 5, 4, 5, 4, 3, 5, 5, 3, 4, 3, 1, 5, 4, 1, 4, 4, 5, 5, 5, 3, 5, 5, 1, 1, 2, 1, 4, 5, 1, 2, 5, 5, 1, 2, 1, 1, 5, 5, 5, 5, 5, 4, 5, 5, 4, 5, 4, 3, 4, 3, 4, 2, 3, 5, 5, 5, 5, 5, 5, 1, 5, 4, 3, 4, 4, 1, 5, 5, 5, 3, 5, 4, 5, 5, 2, 3, 3, 3, 5, 1, 5, 5, 3, 1, 4, 4, 4, 1, 5, 5, 4, 5, 5, 5, 5, 3, 5, 5, 2, 4, 5, 4, 4, 5, 4, 5, 4, 5, 1, 5, 1, 5, 5, 5, 5, 5, 5, 4, 4, 4, 5, 4, 4, 5, 1, 1, 1, 5, 5, 1, 5, 4, 5, 5, 4, 1, 5, 5, 5, 5, 4, 5, 4, 5, 5, 5, 5, 5, 5, 5, 4, 3, 4, 4, 5, 5, 5, 5, 5, 5, 1, 4, 5, 3, 4, 5, 5, 2, 5, 1, 5, 3, 1, 3, 4, 1, 5, 4, 5, 5, 2, 2, 4, 4, 5, 5, 1, 3, 5, 5, 5, 1, 4, 1, 4, 5, 5, 3, 4, 5, 5, 4, 5, 4, 5, 5, 1, 5, 5, 5, 4, 3, 4, 1, 2, 4, 3, 4, 5, 4, 3, 5, 2, 2, 5, 5, 4, 4, 5, 5, 4, 5, 5, 5, 4, 5, 5, 1, 4, 4, 5, 5, 5, 4, 5, 1, 3, 5, 5, 3, 2, 4, 4, 2, 5, 5, 5, 5, 4, 5, 4, 5, 5, 5, 5, 5, 5, 1, 5, 5, 1, 5, 5, 3, 4, 5, 3, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 4, 5, 1, 2, 5, 4, 5, 5, 5, 4, 5, 4, 5, 4, 4, 5, 5, 5, 4, 5, 4, 3, 5, 1, 3, 5, 4, 5, 1, 1, 5, 5, 4, 3, 3, 1, 3, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 5, 1, 3, 4, 5, 5, 5, 4, 4, 5, 5, 5, 4, 5, 5, 3, 5, 4, 5, 4, 4, 1, 5, 5, 3, 5, 4, 3, 5, 4, 1, 4, 5, 1, 4, 4, 3, 4, 4, 1, 5, 4, 1, 1, 4, 4, 5, 5, 5, 5, 1, 4, 5, 3, 5, 1, 4, 4, 5, 5, 1, 4, 5, 1, 5, 1, 1, 5, 1, 4, 5, 5, 5, 2, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 4, 1, 1, 4, 5, 3, 1, 1, 4, 5, 2, 5, 4, 3, 5, 4, 5, 5, 1, 5, 4, 5, 1, 5, 5, 1, 1, 4, 5, 4, 3, 3, 1, 3, 5, 5, 5, 1, 4, 3, 4, 5, 1, 3, 5, 4, 5, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 1, 4, 2, 1, 5, 4, 4, 3, 1, 5, 5, 4, 4, 5, 5, 4, 1, 5, 5, 4, 3, 3, 3, 3, 4, 3, 1, 5, 1, 4, 1, 4, 5, 2, 4, 5, 5, 1, 5, 3, 4, 3, 5, 4, 4, 1, 4, 1, 3, 5, 1, 4, 5, 4, 5, 5, 3, 5, 5, 1, 3, 5, 2, 4, 3, 5, 2, 5, 2, 4, 1, 5, 1, 4, 1, 4, 4, 5, 5, 5, 5, 4, 2, 5, 5, 4, 5, 4, 1, 2, 3, 5, 3, 2, 5, 5, 4, 2, 5, 1, 5, 1, 4, 1, 3, 4, 3, 4, 2, 1, 1, 1, 1, 5, 2, 5, 5, 1, 5, 5, 2, 5, 3, 1, 1, 5, 4, 1, 1, 5, 5, 4, 1, 3, 5, 5, 5, 1, 4, 5, 1, 3, 5, 4, 1, 1, 5, 4, 2, 5, 1, 1, 1, 1, 5, 4, 4, 5, 4, 3, 5, 3, 3, 3, 5, 5, 4, 3, 1, 4, 5, 1, 5, 1, 4, 5, 4, 5, 4, 1, 1, 5, 5, 5, 1, 1, 3, 5, 5, 5, 5, 5, 3, 5, 1, 5, 1, 5, 5, 4, 3, 4, 4, 5, 4, 5, 4, 5, 5, 3, 5, 5, 5, 4, 5, 5, 5, 1, 5, 2, 5, 5, 5, 4, 5, 1, 1, 5, 4, 3, 5, 5, 4, 5, 5, 1, 3, 1, 5, 5, 1, 2, 3, 5, 5, 1, 5, 5, 5, 4, 1, 4, 3, 4, 5, 1, 5, 1, 5, 1, 5, 3, 1, 4, 5, 5, 3, 5, 5, 3, 5, 5, 5, 5, 1, 5, 5, 4, 4, 4, 1, 5, 2, 5, 5, 1, 5, 5, 4, 3, 5, 4, 2, 5, 5, 5, 3, 5, 2, 5, 5, 5, 1, 5, 5, 5, 5, 4, 5, 3, 3, 4, 5, 5, 5, 1, 5, 5, 4, 3, 5, 5, 5, 2, 3, 5, 1, 1, 3, 2, 5, 1, 5, 1, 5, 5, 5, 5, 1, 5, 5, 4, 2, 4, 5, 3, 1, 5, 5, 5, 5, 5, 5, 5, 4, 2, 4, 5, 4, 5, 5, 3, 4, 5, 5, 2, 2, 5, 5, 5, 5, 4, 5, 5, 5, 5, 1, 5, 5, 3, 5, 5, 1, 5, 5, 4, 5, 4, 5, 5, 5, 1, 5, 5, 4, 5, 1, 5, 5, 1, 3, 5, 5, 4, 4, 5, 5, 4, 4, 1, 4, 5, 1, 5, 4, 3, 1, 5, 5, 2, 3, 5, 4, 4, 5, 4, 5, 4, 5, 5, 4, 5, 5, 1, 4, 5, 1, 5, 5, 5, 5, 4, 1, 5, 5, 2, 5, 2, 5, 5, 3, 5, 1, 4, 4, 5, 1, 5, 4, 5, 1, 4, 1, 5, 4, 1, 5, 5, 5, 5, 5, 3, 4, 3, 5, 1, 5, 3, 5, 5, 5, 4, 5, 5, 5, 5, 4, 5, 1, 5, 3, 5, 3, 1, 1, 3, 1, 5, 5, 4, 5, 1, 5, 5, 1, 5, 5, 2, 5, 5, 5, 4, 5, 1, 2, 5, 3, 1, 1, 4, 5, 5, 5, 5, 2, 5, 5, 1, 1, 5, 5, 5, 4, 5, 4, 4, 5, 5, 4, 1, 1, 3, 5, 5, 5, 2, 3, 4, 4, 2, 5, 1, 1, 4, 5, 4, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 1, 4, 5, 1, 5, 3, 5, 5, 5, 4, 5, 5, 3, 5, 5, 2, 5, 5, 5, 1, 5, 5, 1, 5, 1, 5, 1, 1, 4, 4, 4, 5, 5, 5, 3, 4, 5, 4, 4, 5, 3, 5, 4, 3, 4, 1, 4, 5, 5, 5, 5, 4, 3, 1, 5, 4, 1, 5, 1, 4, 5, 4, 1, 5, 5, 4, 1, 1, 5, 1, 5, 1, 5, 2, 1, 5, 4, 3, 5, 5, 5, 5, 5, 4, 5, 5, 3, 3, 5, 3, 3, 5, 3, 5, 1, 5, 3, 4, 5, 4, 5, 2, 5, 5, 5, 4, 5, 4, 5, 5, 5, 4, 3, 1, 2, 5, 2, 5, 5, 4, 5, 3, 5, 5, 1, 2, 1, 1, 5, 5, 2, 5, 5, 4, 1, 5, 5, 3, 5, 3, 4, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 4, 5, 5, 5, 1, 5, 5, 1, 5, 5, 1, 1, 3, 5, 4, 3, 2, 2, 1, 5, 5, 1, 5, 4, 5, 5, 5, 1, 5, 1, 3, 5, 4, 1, 2, 5, 5, 4, 4, 3, 5, 1, 4, 3, 5, 4, 5, 2, 5, 4, 5, 5, 2, 1, 5, 4, 1, 1, 5, 5, 4, 5, 5, 5, 5, 5, 5, 1, 5, 5, 4, 1, 5, 4, 1, 1, 4, 5, 2, 5, 5, 5, 3, 1, 5, 5, 5, 5, 1, 3, 1, 4, 5, 5, 5, 5, 5, 5, 1, 5, 5, 3, 4, 1, 5, 5, 5, 5, 2, 5, 5, 4, 5, 3, 4, 4, 5, 4, 2, 5, 5, 5, 5, 4, 3, 4, 5, 5, 5, 1, 5, 1, 5, 5, 1, 3, 1, 3, 5, 5, 5, 1, 5, 4, 1, 5, 5, 2, 4, 4, 3, 4, 5, 5, 5, 5, 4, 1, 1, 5, 5, 5, 4, 5, 5, 5, 5, 1, 1, 5, 4, 2, 5, 1, 3, 5, 4, 4, 5, 4, 5, 4, 3, 5, 5, 5, 1, 5, 5, 4, 5, 4, 1, 5, 5, 5, 4, 5, 5, 4, 1, 5, 5, 1, 5, 5, 5, 3, 2, 5, 5, 2, 5, 5, 5, 5, 5, 5, 1, 4, 5, 1, 4, 4, 1, 2, 1, 4, 4, 5, 5, 1, 1, 5, 1, 1, 5, 3, 1, 4, 5, 5, 5, 1, 1, 5, 5, 4, 5, 5, 5, 1, 1, 2, 3, 1, 5, 5, 4, 4, 1, 5, 5, 2, 1, 1, 1, 2, 5, 2, 2, 4, 5, 5, 5, 3, 5, 3, 4, 3, 1, 5, 5, 4, 5, 5, 5, 5, 4, 3, 4, 5, 1, 4, 5, 5, 4, 5, 5, 5, 4, 5, 5, 4, 5, 3, 5, 1, 3, 1, 1, 4, 5, 5, 4, 5, 1, 5, 4, 5, 2, 2, 5, 4, 5, 1, 1, 5, 1, 4, 3, 5, 5, 5, 4, 4, 5, 4, 5, 5, 2, 5, 5, 5, 4, 5, 5, 4, 5, 5, 3, 1, 5, 1, 1, 5, 3, 5, 5, 1, 5, 5, 5, 3, 1, 5, 1, 3, 4, 5, 4, 4, 5, 2, 5, 1, 5, 1, 5, 1, 4, 5, 5, 4, 5, 5, 1, 1, 4, 5, 4, 5, 2, 5, 5, 5, 1, 5, 5, 5, 5, 5, 1, 5, 4, 5, 5, 5, 1, 5, 5, 4, 4, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 1, 2, 5, 4, 5, 5, 5, 1, 5, 2, 5, 5, 5, 5, 5, 3, 3, 3, 3, 1, 3, 4, 3, 5, 5, 4, 5, 3, 5, 5, 1, 5, 4, 1, 5, 3, 1, 5, 5, 3, 4, 3, 2, 1, 1, 5, 3, 4, 1, 1, 4, 5, 5, 4, 4, 5, 3, 1, 1, 5, 1, 5, 1, 1, 4, 5, 5, 4, 1, 5, 5, 4, 1, 5, 5, 2, 4, 4, 2, 5, 5, 3, 1, 1, 5, 5, 5, 5, 5, 5, 5, 1, 1, 3, 5, 5, 5, 1, 5, 5, 3, 5, 5, 5, 4, 5, 5, 5, 5, 1, 5, 5, 2, 5, 1, 3, 4, 2, 5, 5, 5, 5, 5, 5, 4, 4, 5, 3, 3, 1, 5, 5, 1, 2, 5, 3, 4, 1, 5, 4, 4, 3, 5, 5, 5, 2, 4, 4, 5, 3, 1, 3, 2, 5, 1, 1, 5, 2, 4, 5, 1, 5, 5, 4, 3, 1, 1, 1, 5, 4, 4, 5, 1, 4, 4, 1, 3, 5, 5, 5, 4, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 1, 5, 4, 5, 5, 5, 4, 1, 1, 5, 3, 4, 3, 1, 5, 5, 5, 4, 1, 1, 4, 4, 1, 2, 3, 3, 4, 5, 4, 4, 4, 1, 4, 3, 5, 3, 5, 4, 5, 1, 4, 5, 4, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 3, 5, 5, 2, 5, 4, 5, 5, 5, 1, 4, 5, 1, 5, 5, 5, 3, 5, 1, 5, 1, 5, 1, 4, 5, 1, 4, 1, 4, 1, 5, 4, 1, 5, 5, 4, 5, 1, 5, 5, 5, 5, 4, 1, 1, 5, 4, 5, 1, 1, 5, 5, 5, 5, 5, 1, 5, 5, 5, 3, 5, 3, 5, 1, 5, 5, 1, 5, 2, 5, 5, 1, 5, 5, 5, 1, 5, 5, 4, 4, 5, 5, 5, 5, 5, 1, 1, 4, 3, 3, 5, 5, 5, 5, 5, 4, 4, 2, 1, 4, 5, 5, 4, 1, 1, 4, 5, 5, 4, 5, 5, 5, 1, 1, 5, 4, 5, 5, 5, 5, 1, 2, 5, 1, 5, 5, 1, 5, 5, 5, 4, 5, 5, 4, 1, 5, 4, 3, 2, 4, 5, 3, 5, 5, 1, 4, 1, 4, 4, 1, 3, 5, 5, 5, 5, 1, 5, 1, 4, 5, 5, 4, 4, 5, 5, 5, 4, 4, 5, 5, 5, 5, 5, 3, 4, 5, 5, 5, 1, 2, 5, 5, 1, 4, 5, 5, 1, 5, 5, 5, 5, 5, 4, 3, 2, 5, 4, 5, 4, 1, 4, 4, 3, 5, 4, 5, 1, 5, 3, 3, 4, 4, 1, 4, 5, 4, 4, 4, 1, 1, 1, 5, 4, 1, 4, 5, 1, 5, 1, 3, 3, 5, 4, 5, 1, 4, 1, 3, 1, 4, 5, 5, 1, 5, 5, 1, 5, 5, 5, 3, 5, 5, 1, 5, 2, 5, 5, 3, 3, 1, 5, 5, 5, 3, 1, 5, 4, 1, 5, 5, 1, 5, 4, 4, 4, 3, 5, 1, 3, 2, 5, 5, 5, 1, 5, 5, 1, 1, 4, 3, 1, 1, 5, 1, 3, 5, 5, 5, 4, 5, 1, 3, 4, 5, 5, 4, 5, 1, 5, 5, 4, 1, 3, 1, 1, 5, 1, 4, 5, 2, 3, 4, 2, 1, 5, 1, 4, 5, 4, 4, 5, 3, 5, 3, 1, 5, 4, 5, 5, 5, 4, 5, 5, 3, 4, 5, 5, 4, 3, 4, 3, 3, 4, 5, 3, 5, 5, 3, 3, 5, 5, 4, 3, 4, 1, 5, 5, 1, 4, 1, 1, 4, 5, 5, 5, 3, 3, 5, 5, 1, 5, 4, 4, 3, 1, 3, 4, 5, 5, 1, 4, 5, 4, 4, 5, 1, 5, 5, 5, 1, 1, 3, 5, 5, 5, 5, 5, 5, 1, 5, 4, 4, 1, 1, 5, 5, 5, 4, 4, 4, 3, 5, 4, 5, 5, 5, 4, 1, 5, 5, 1, 5, 1, 5, 1, 5, 4, 1, 5, 5, 5, 5, 3, 5, 1, 4, 4, 4, 4, 5, 4, 5, 3, 5, 5, 1, 4, 5, 5, 5, 1, 2, 5, 5, 5, 5, 5, 5, 5, 2, 1, 4, 5, 5, 5, 1, 5, 4, 5, 5, 5, 5, 5, 5, 5, 3, 4, 4, 5, 2, 1, 4, 4, 4, 5, 5, 4, 1, 5, 1, 5, 4, 3, 1, 4, 5, 4, 5, 4, 4, 4, 5, 5, 1, 4, 5, 5, 1, 2, 3, 5, 4, 2, 3, 5, 1, 5, 3, 5, 1, 5, 2, 5, 2, 3, 4, 5, 2, 1, 5, 5, 5, 3, 3, 5, 4, 3, 3, 3, 4, 5, 1, 5, 5, 5, 3, 5, 4, 4, 5, 3, 1, 5, 1, 5, 3, 1, 5, 5, 5, 5, 2, 4, 5, 5, 3, 4, 2, 5, 5, 5, 3, 5, 5, 3, 5, 5, 5, 5, 2, 4, 1, 1, 5, 5, 5, 3, 4, 5, 5, 5, 3, 1, 5, 1, 2, 5, 1, 3, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 3, 4, 5, 3, 2, 4, 5, 5, 1, 5, 5, 4, 5, 5, 1, 4, 1, 1, 2, 5, 5, 4, 4, 4, 2, 5, 3, 5, 5, 5, 4, 3, 5, 5, 2, 2, 1, 5, 5, 1, 5, 5, 5, 4, 1, 5, 4, 4, 4, 4, 2, 4, 5, 5, 2, 5, 4, 5, 5, 3, 3, 5, 1, 2, 1, 5, 5, 1, 1, 5, 5, 5, 5, 3, 1, 5, 5, 4, 4, 4, 4, 1, 5, 5, 5, 3, 4, 5, 4, 5, 4, 1, 5, 4, 5, 4, 5, 1, 4, 1, 5, 5, 1, 3, 5, 1, 4, 2, 2, 5, 3, 4, 3, 5, 5, 5, 5, 5, 5, 5, 1, 1, 5, 1, 5, 4, 2, 3, 5, 4, 5, 4, 1, 5, 4, 1, 5, 1, 3, 5, 5, 5, 1, 4, 4, 5, 2, 5, 5, 1, 1, 4, 5, 2, 1, 1, 5, 1, 1, 5, 5, 1, 1, 5, 3, 1, 5, 1, 2, 1, 4, 1, 4, 1, 5, 5, 5, 4, 5, 5, 5, 5, 5, 4, 4, 5, 5, 5, 1, 1, 5, 5, 4, 3, 4, 5, 3, 4, 5, 4, 5, 1, 3, 5, 3, 5, 2, 1, 5, 4, 5, 1, 4, 1, 5, 4, 2, 5, 5, 5, 5, 5, 5, 4, 5, 3, 1, 3, 5, 4, 3, 5, 3, 3, 5, 4, 5, 3, 5, 1, 5, 1, 5, 4, 5, 3, 4, 4, 5, 4, 5, 4, 5, 5, 3, 2, 3, 1, 4, 5, 2, 4, 1, 3, 2, 5, 5, 4, 4, 4, 5, 3, 5, 5, 5, 3, 2, 1, 5, 2, 5, 4, 5, 5, 5, 1, 5, 5, 5, 5, 4, 5, 1, 5, 5, 3, 5, 1, 5, 5, 5, 5, 4, 2, 5, 4, 5, 5, 3, 5, 2, 3, 3, 4, 1, 5, 5, 5, 4, 1, 5, 1, 1, 4, 1, 4, 5, 4, 5, 5, 5, 5, 3, 4, 5, 1, 1, 4, 5, 1, 5, 1, 4, 5, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 3, 1, 5, 5, 5, 5, 4, 3, 4, 1, 5, 3, 2, 3, 4, 4, 3, 4, 4, 5, 5, 5, 3, 5, 3, 4, 5, 4, 1, 5, 3, 3, 1, 5, 3, 5, 1, 5, 5, 3, 4, 2, 5, 5, 1, 5, 5, 4, 1, 5, 4, 5, 5, 1, 5, 5, 2, 1, 1, 1, 5, 5, 5, 1, 4, 5, 4, 5, 1, 5, 5, 5, 1, 1, 1, 5, 3, 5, 5, 1, 5, 5, 4, 4, 5, 3, 5, 1, 5, 1, 5, 5, 1, 1, 1, 1, 5, 2, 5, 4, 4, 1, 3, 5, 5, 5, 5, 1, 4, 5, 4, 2, 4, 3, 1, 1, 4, 5, 4, 5, 5, 2, 3, 3, 5, 5, 4, 1, 5, 5, 5, 5, 3, 5, 2, 1, 4, 4, 5, 5, 3, 5, 5, 1, 5, 5, 5, 5, 5, 4, 5, 5, 4, 4, 5, 5, 5, 1, 1, 3, 5, 2, 4, 5, 5, 2, 1, 5, 5, 1, 5, 5, 4, 5, 4, 5, 1, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 4, 1, 4, 4, 4, 5, 5, 5, 3, 5, 5, 4, 5, 4, 5, 1, 1, 5, 1, 2, 5, 4, 4, 5, 1, 5, 5, 5, 1, 5, 5, 5, 1, 1, 5, 4, 1, 5, 3, 1, 4, 5, 5, 1, 5, 4, 5, 4, 5, 5, 1, 3, 3, 1, 5, 5, 3, 4, 3, 5, 5, 1, 5, 4, 4, 5, 5, 5, 3, 5, 1, 3, 5, 1, 1, 5, 1, 5, 4, 2, 1, 1, 5, 4, 5, 4, 5, 5, 5, 5, 2, 3, 1, 3, 3, 1, 5, 5, 4, 1, 3, 1, 5, 4, 1, 5, 5, 1, 5, 5, 2, 5, 5, 4, 1, 5, 5, 5, 4, 3, 1, 5, 5, 3, 5, 3, 1, 1, 1, 5, 4, 5, 4, 5, 5, 5, 1, 5, 5, 5, 5, 5, 4, 1, 4, 4, 5, 2, 4, 5, 5, 5, 5, 3, 3, 5, 1, 3, 5, 4, 1, 1, 5, 1, 4, 4, 5, 2, 1, 4, 5, 2, 5, 1, 5, 4, 5, 3, 1, 4, 1, 5, 5, 5, 1, 5, 5, 4, 1, 4, 1, 4, 2, 5, 4, 5, 5, 3, 3, 5, 3, 4, 1, 1, 5, 2, 4, 5, 5, 3, 5, 1, 4, 5, 4, 5, 4, 5, 5, 1, 4, 1, 4, 5, 5, 3, 5, 4, 5, 5, 1, 5, 5, 2, 5, 3, 4, 5, 5, 5, 4, 5, 2, 1, 4, 4, 5, 3, 4, 4, 5, 5, 5, 3, 5, 5, 5, 3, 4, 5, 1, 2, 4, 5, 5, 4, 3, 5, 4, 1, 4, 4, 5, 4, 2, 4, 4, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 1, 4, 3, 5, 5, 5, 1, 5, 1, 5, 5, 4, 4, 3, 1, 5, 1, 1, 5, 5, 5, 5, 5, 5, 5, 1, 3, 5, 5, 5, 4, 5, 1, 5, 3, 1, 5, 2, 4, 4, 4, 5, 5, 5, 5, 5, 5, 2, 5, 1, 5, 1, 2, 1, 1, 1, 5, 5, 4, 5, 4, 4, 5, 4, 5, 4, 5, 5, 4, 1, 5, 5, 5, 5, 2, 5, 4, 3, 4, 5, 5, 5, 5, 3, 5, 5, 5, 1, 5, 3, 5, 1, 1, 5, 5, 1, 1, 1, 5, 4, 3, 5, 1, 4, 5, 5, 5, 5, 1, 1, 5, 1, 3, 2, 4, 5, 4, 5, 5, 5, 5, 5, 1, 5, 2, 4, 4, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 1, 5, 2, 4, 5, 5, 4, 2, 3, 3, 4, 5, 5, 1, 1, 3, 5, 5, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 1, 3, 1, 4, 4, 5, 5, 5, 3, 3, 5, 3, 5, 1, 5, 5, 5, 4, 1, 4, 2, 5, 5, 4, 4, 1, 5, 1, 5, 3, 5, 4, 4, 5, 5, 4, 3, 2, 1, 1, 3, 4, 5, 4, 3, 5, 4, 2, 5, 4, 4, 1, 5, 5, 4, 1, 5, 5, 5, 5, 5, 1, 5, 1, 1, 2, 5, 5, 5, 1, 1, 1, 5, 4, 5, 1, 5, 4, 1, 5, 2, 5, 3, 4, 5, 1, 5, 4, 5, 5, 5, 4, 5, 1, 5, 5, 5, 5, 5, 5, 3, 4, 1, 5, 5, 4, 1, 4, 2, 4, 5, 4, 5, 4, 2, 4, 5, 4, 2, 5, 3, 1, 1, 5, 5, 4, 3, 3, 5, 1, 5, 5, 4, 5, 5, 5, 2, 1, 4, 5, 5, 1, 5, 1, 5, 5, 5, 5, 1, 2, 5, 1, 4, 5, 5, 3, 1, 5, 2, 5, 1, 5, 3, 5, 4, 5, 5, 5, 1, 2, 5, 4, 5, 2, 1, 1, 4, 5, 5, 5, 5, 5, 5, 5, 2, 5, 1, 5, 5, 1, 1, 5, 1, 1, 4, 4, 1, 5, 1, 4, 4, 5, 5, 1, 1, 4, 2, 2, 5, 4, 3, 5, 3, 4, 1, 2, 1, 5, 1, 5, 5, 5, 5, 5, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 1, 3, 4, 5, 4, 3, 3, 2, 3, 5, 5, 5, 5, 5, 1, 4, 4, 4, 5, 3, 4, 5, 5, 5, 3, 3, 1, 5, 5, 1, 1, 4, 1, 5, 3, 4, 3, 3, 4, 1, 1, 5, 5, 5, 4, 1, 5, 5, 3, 3, 4, 5, 4, 1, 5, 4, 3, 3, 4, 1, 1, 2, 5, 4, 4, 5, 4, 5, 4, 5, 4, 5, 2, 3, 5, 4, 5, 4, 5, 1, 5, 2, 1, 5, 4, 5, 5, 5, 4, 4, 5, 5, 4, 5, 2, 5, 3, 5, 1, 5, 4, 5, 1, 5, 2, 5, 3, 1, 5, 5, 4, 5, 2, 4, 5, 1, 1, 5, 5, 5, 5, 4, 5, 1, 1, 4, 1, 4, 5, 3, 5, 1, 5, 5, 3, 3, 4, 1, 5, 3, 5, 4, 4, 5, 2, 5, 5, 3, 5, 2, 4, 5, 3, 3, 1, 5, 2, 5, 2, 5, 5, 5, 3, 5, 4, 3, 1, 3, 4, 4, 5, 3, 5, 5, 2, 5, 2, 5, 2, 5, 5, 5, 5, 5, 5, 4, 5, 2, 4, 5, 3, 5, 5, 1, 1, 4, 5, 1, 1, 5, 5, 5, 4, 3, 3, 5, 1, 1, 5, 4, 1, 1, 4, 1, 1, 1, 1, 5, 1, 5, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 1, 3, 5, 3, 5, 1, 4, 5, 5, 5, 4, 5, 4, 1, 5, 1, 5, 1, 2, 4, 2, 4, 3, 1, 4, 1, 4, 4, 3, 1, 5, 4, 5, 1, 3, 3, 5, 5, 5, 4, 5, 5, 5, 5, 5, 4, 5, 4, 4, 5, 2, 5, 5, 4, 5, 1, 1, 5, 3, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 1, 4, 5, 4, 5, 4, 5, 2, 1, 5, 5, 5, 1, 5, 5, 4, 5, 4, 5, 4, 5, 1, 2, 4, 4, 5, 2, 5, 5, 5, 5, 5, 1, 2, 1, 5, 5, 4, 5, 4, 4, 5, 5, 5, 2, 5, 5, 2, 5, 5, 1, 5, 5, 4, 5, 1, 4, 5, 5, 5, 5, 4, 3, 5, 5, 2, 5, 4, 5, 5, 3, 5, 3, 5, 4, 1, 4, 3, 4, 3, 5, 5, 5, 5, 3, 3, 5, 4, 1, 1, 4, 4, 4, 5, 5, 3, 5, 5, 1, 5, 5, 5, 5, 5, 1, 5, 5, 4, 5, 2, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 4, 1, 4, 5, 5, 4, 3, 2, 2, 1, 3, 5, 5, 1, 4, 1, 3, 1, 5, 1, 5, 5, 4, 5, 5, 5, 5, 1, 4, 5, 5, 3, 4, 1, 1, 5, 2, 5, 5, 5, 2, 5, 5, 2, 4, 5, 5, 5, 3, 5, 5, 1, 3, 5, 1, 5, 5, 1, 3, 1, 4, 5, 2, 1, 4, 4, 5, 1, 5, 1, 5, 1, 5, 3, 1, 5, 5, 1, 5, 5, 5, 5, 2, 5, 5, 5, 1, 4, 4, 5, 3, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 1, 5, 2, 4, 4, 5, 4, 5, 3, 5, 1, 5, 5, 2, 4, 3, 5, 5, 5, 4, 4, 5, 4, 1, 5, 5, 1, 4, 3, 5, 5, 4, 5, 4, 5, 1, 1, 5, 3, 4, 5, 4, 3, 5, 5, 1, 3, 5, 4, 5, 5, 1, 3, 5, 5, 5, 2, 1, 1, 4, 5, 3, 5, 1, 4, 1, 3, 5, 5, 5, 1, 2, 3, 4, 5, 4, 4, 3, 5, 2, 4, 4, 4, 5, 5, 1, 5, 3, 5, 5, 5, 5, 1, 5, 5, 3, 1, 4, 4, 5, 4, 1, 5, 2, 4, 4, 3, 4, 5, 5, 1, 5, 3, 5, 5, 5, 5, 5, 5, 4, 3, 5, 4, 4, 5, 5, 1, 5, 2, 3, 4, 5, 4, 5, 4, 5, 3, 2, 5, 5, 1, 5, 5, 2, 5, 5, 4, 5, 5, 4, 5, 5, 4, 1, 4, 5, 5, 1, 5, 1, 3, 2, 5, 1, 5, 5, 4, 3, 5, 5, 1, 5, 4, 5, 5, 4, 5, 1, 5, 5, 5, 4, 5, 1, 5, 5, 5, 5, 1, 4, 3, 2, 5, 5, 5, 4, 4, 4, 5, 1, 5, 3, 3, 4, 5, 5, 3, 5, 3, 5, 1, 4, 5, 1, 5, 4, 5, 1, 5, 3, 4, 3, 5, 4, 4, 3, 5, 5, 5, 5, 5, 5, 5, 4, 5, 1, 4, 1, 1, 5, 4, 5, 1, 4, 5, 5, 5, 5, 2, 5, 5, 5, 1, 1, 5, 5, 5, 2, 2, 5, 1, 4, 5, 5, 1, 5, 1, 1, 4, 5, 5, 1, 4, 5, 3, 2, 1, 5, 1, 5, 5, 5, 1, 4, 5, 5, 5, 4, 5, 5, 2, 3, 5, 5, 5, 5, 2, 4, 5, 4, 5, 5, 1, 5, 5, 1, 4, 5, 5, 5, 5, 5, 3, 5, 5, 5, 4, 5, 5, 5, 4, 4, 5, 1, 5, 2, 3, 2, 4, 1, 4, 3, 3, 5, 1, 1, 5, 4, 5, 3, 3, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 4, 4, 5, 1, 4, 5, 5, 5, 5, 2, 1, 5, 5, 4, 2, 5, 5, 1, 5, 4, 1, 5, 1, 5, 5, 4, 5, 5, 5, 5, 1, 5, 1, 5, 5, 1, 5, 5, 3, 4, 1, 5, 5, 5, 3, 4, 1, 5, 3, 4, 5, 4, 5, 1, 1, 5, 4, 5, 5, 5, 4, 5, 3, 4, 3, 3, 1, 1, 1, 5, 3, 4, 5, 5, 1, 5, 1, 5, 5, 5, 5, 4, 1, 1, 1, 5, 4, 5, 5, 5, 2, 1, 4, 5, 5, 4, 4, 5, 4, 5, 1, 3, 5, 4, 1, 4, 5, 4, 4, 4, 5, 5, 1, 1, 5, 5, 5, 1, 4, 5, 3, 5, 5, 1, 4, 4, 5, 5, 5, 1, 4, 4, 5, 5, 5, 1, 5, 5, 1, 2, 5, 4, 4, 1, 5, 5, 5, 5, 3, 5, 2, 1, 5, 5, 4, 1, 3, 4, 5, 1, 4, 5, 3, 1, 1, 1, 5, 1, 5, 1, 5, 1, 2, 5, 3, 3, 5, 1, 5, 4, 3, 4, 5, 3, 4, 4, 3, 5, 4, 1, 5, 5, 1, 4, 5, 5, 5, 5, 5, 5, 5, 1, 1, 3, 5, 4, 5, 4, 5, 4, 4, 2, 3, 5, 5, 4, 5, 2, 4, 5, 1, 2, 5, 5, 1, 5, 5, 3, 5, 5, 2, 5, 5, 5, 2, 5, 5, 4, 5, 5, 4, 4, 4, 5, 3, 5, 4, 3, 1, 4, 1, 5, 5, 5, 1, 1, 2, 5, 4, 5, 4, 1, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 3, 5, 1, 3, 5, 1, 5, 1, 5, 4, 5, 5, 4, 5, 5, 5, 1, 5, 5, 1, 5, 5, 1, 5, 1, 1, 5, 5, 5, 5, 3, 5, 3, 1, 5, 5, 4, 2, 4, 3, 2, 5, 5, 4, 1, 1, 5, 5, 1, 1, 5, 4, 3, 4, 4, 5, 5, 3, 4, 1, 5, 1, 5, 4, 5, 5, 5, 5, 3, 3, 4, 2, 4, 4, 5, 4, 5, 1, 1, 4, 4, 1, 4, 4, 4, 5, 4, 4, 5, 1, 4, 3, 5, 5, 5, 5, 5, 1, 1, 5, 3, 1, 5, 5, 4, 3, 1, 5, 3, 5, 3, 4, 5, 4, 5, 5, 5, 3, 5, 5, 4, 1, 5, 5, 5, 5, 4, 5, 5, 1, 4, 1, 5, 5, 2, 5, 5, 4, 4, 5, 5, 5, 5, 5, 5, 3, 1, 4, 3, 1, 2, 5, 5, 1, 4, 4, 5, 5, 1, 1, 5, 4, 5, 3, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 3, 5, 2, 1, 5, 2, 1, 4, 5, 3, 3, 5, 4, 1, 5, 5, 5, 5, 1, 1, 5, 2, 2, 4, 2, 3, 4, 5, 5, 5, 5, 5, 1, 5, 5, 5, 4, 4, 5, 5, 5, 5, 4, 5, 4, 5, 4, 5, 5, 5, 4, 3, 4, 4, 1, 5, 5, 5, 2, 1, 3, 4, 3, 5, 1, 4, 5, 5, 3, 1, 5, 1, 5, 4, 1, 5, 3, 5, 5, 2, 4, 5, 5, 3, 4, 4, 4, 4, 5, 5, 4, 4, 5, 5, 5, 5, 1, 5, 1, 4, 5, 5, 5, 5, 5, 4, 1, 4, 5, 5, 4, 3, 5, 1, 4, 5, 3, 1, 4, 1, 5, 1, 5, 4, 3, 1, 1, 4, 4, 3, 5, 4, 5, 5, 4, 5, 5, 1, 5, 2, 5, 5, 1, 2, 4, 3, 5, 5, 1, 3, 5, 5, 2, 5, 5, 4, 5, 4, 5, 4, 5, 1, 5, 5, 5, 1, 4, 5, 4, 5, 5, 4, 3, 2, 5, 3, 5, 5, 5, 4, 5, 5, 5, 3, 5, 1, 5, 1, 1, 4, 5, 5, 3, 4, 5, 1, 5, 4, 5, 4, 4, 4, 4, 5, 3, 3, 5, 1, 5, 5, 3, 5, 5, 5, 1, 4, 5, 4, 5, 5, 5, 5, 5, 5, 3, 5, 4, 1, 5, 5, 4, 5, 4, 5, 4, 3, 2, 5, 5, 4, 5, 3, 1, 5, 5, 4, 5, 3, 5, 1, 4, 5, 3, 4, 4, 1, 5, 5, 5, 1, 3, 1, 5, 4, 5, 4, 5, 3, 5, 4, 5, 5, 5, 4, 2, 5, 4, 4, 2, 3, 3, 5, 3, 5, 5, 1, 3, 5, 5, 5, 1, 5, 4, 3, 5, 5, 4, 5, 4, 4, 1, 5, 5, 5, 3, 4, 1, 1, 1, 1, 1, 1, 5, 5, 4, 1, 1, 4, 1, 2, 4, 5, 1, 4, 5, 5, 4, 3, 5, 3, 5, 5, 1, 5, 1, 5, 5, 3, 5, 1, 5, 1, 5, 4, 5, 5, 4, 5, 5, 4, 5, 4, 4, 5, 2, 1, 5, 4, 5, 4, 5, 3, 5, 5, 4, 4, 5, 5, 5, 5, 3, 5, 1, 5, 5, 5, 1, 1, 5, 5, 4, 5, 4, 5, 5, 5, 5, 4, 1, 5, 5, 5, 1, 1, 4, 5, 4, 3, 3, 4, 5, 4, 4, 5, 5, 4, 2, 4, 5, 1, 2, 4, 3, 5, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 3, 1, 1, 1, 5, 5, 3, 4, 5, 1, 5, 1, 5, 2, 5, 3, 5, 1, 5, 1, 4, 5, 5, 5, 4, 1, 5, 1, 5, 4, 5, 5, 3, 5, 5, 1, 2, 5, 5, 2, 5, 5, 3, 4, 5, 4, 5, 4, 1, 1, 5, 5, 5, 5, 4, 4, 5, 5, 4, 5, 4, 4, 5, 5, 5, 5, 5, 1, 5, 5, 4, 5, 1, 4, 5, 5, 4, 5, 5, 5, 1, 5, 5, 5, 1, 1, 4, 5, 4, 5, 4, 1, 5, 5, 4, 4, 5, 5, 5, 3, 5, 5, 1, 1, 3, 5, 3, 5, 3, 5, 5, 5, 1, 3, 3, 3, 5, 3, 4, 5, 5, 4, 1, 1, 5, 5, 4, 2, 5, 1, 3, 5, 5, 1, 1, 5, 5, 5, 3, 4, 4, 4, 5, 4, 2, 5, 5, 5, 2, 3, 1, 5, 4, 4, 1, 5, 5, 2, 5, 3, 5, 4, 5, 5, 3, 3, 1, 5, 3, 5, 5, 3, 4, 1, 1, 5, 1, 5, 3, 5, 5, 1, 3, 1, 4, 4, 3, 5, 1, 5, 4, 1, 1, 5, 1, 5, 5, 5, 2, 3, 5, 5, 3, 5, 1, 5, 5, 1, 4, 1, 4, 3, 1, 2, 5, 5, 4, 1, 1, 1, 5, 5, 5, 4, 5, 5, 4, 5, 1, 5, 4, 5, 3, 1, 5, 4, 5, 5, 1, 1, 5, 5, 1, 1, 5, 4, 1, 1, 1, 1, 3, 5, 3, 5, 4, 5, 1, 4, 1, 5, 2, 5, 4, 3, 3, 1, 3, 4, 4, 4, 1, 5, 5, 5, 1, 2, 5, 2, 3, 3, 5, 4, 1, 1, 5, 5, 5, 1, 5, 4, 1, 2, 5, 4, 4, 1, 5, 2, 4, 4, 5, 5, 1, 5, 5, 5, 5, 5, 5, 1, 4, 4, 5, 5, 4, 5, 5, 5, 4, 5, 3, 5, 5, 5, 5, 1, 2, 3, 3, 1, 1, 1, 5, 1, 5, 5, 4, 1, 4, 1, 1, 5, 1, 4, 2, 4, 5, 5, 5, 4, 5, 5, 4, 5, 1, 4, 5, 4, 5, 5, 3, 4, 3, 3, 3, 5, 4, 1, 1, 5, 5, 1, 5, 3, 2, 3, 1, 4, 1, 4, 5, 4, 5, 5, 1, 5, 4, 1, 2, 3, 3, 5, 5, 4, 4, 5, 5, 5, 5, 1, 3, 5, 5, 3, 3, 4, 5, 5, 4, 1, 5, 5, 1, 4, 5, 3, 3, 1, 5, 3, 2, 5, 3, 2, 1, 3, 4, 5, 3, 1, 1, 4, 1, 2, 1, 1, 5, 5, 4, 3, 5, 2, 5, 5, 1, 2, 4, 1, 5, 4, 4, 5, 5, 4, 1, 5, 1, 4, 5, 5, 5, 5, 1, 5, 5, 1, 5, 1, 2, 1, 5, 5, 1, 2, 5, 1, 4, 5, 5, 5, 2, 1, 5, 5, 4, 1, 5, 3, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 1, 5, 4, 1, 5, 1, 4, 5, 2, 5, 1, 5, 5, 3, 3, 3, 3, 5, 5, 1, 5, 4, 3, 5, 4, 5, 1, 1, 4, 1, 5, 5, 5, 1, 5, 2, 4, 5, 5, 4, 4, 4, 3, 4, 5, 4, 5, 1, 5, 5, 5, 4, 5, 5, 3, 5, 5, 5, 5, 5, 5, 4, 1, 4, 5, 5, 5, 5, 2, 3, 5, 4, 5, 5, 3, 5, 1, 3, 5, 4, 5, 5, 5, 1, 5, 5, 1, 5, 5, 5, 5, 5, 3, 2, 5, 1, 1, 5, 5, 5, 3, 5, 2, 1, 4, 4, 5, 5, 5, 5, 4, 5, 4, 5, 1, 5, 4, 4, 5, 5, 4, 1, 4, 4, 4, 1, 3, 5, 1, 5, 5, 4, 5, 5, 5, 5, 2, 5, 5, 5, 4, 5, 3, 1, 3, 5, 4, 5, 1, 4, 1, 5, 4, 4, 3, 5, 4, 4, 1, 5, 2, 4, 4, 4, 1, 5, 5, 5, 5, 5, 3, 3, 4, 5, 5, 4, 4, 4, 4, 1, 3, 5, 5, 5, 5, 4, 1, 4, 5, 2, 1, 5, 5, 2, 4, 5, 5, 5, 3, 1, 3, 3, 1, 5, 5, 5, 3, 4, 5, 5, 5, 1, 5, 2, 4, 4, 5, 3, 5, 5, 1, 3, 5, 2, 3, 1, 4, 1, 5, 4, 4, 5, 5, 1, 1, 4, 5, 2, 3, 5, 4, 5, 4, 2, 5, 5, 4, 5, 5, 3, 4, 5, 4, 3, 3, 1, 5, 4, 5, 4, 1, 5, 5, 5, 5, 4, 5, 5, 5, 1, 5, 3, 4, 5, 5, 5, 1, 3, 5, 4, 3, 1, 5, 5, 4, 5, 5, 4, 2, 3, 4, 1, 1, 5, 4, 1, 3, 5, 2, 1, 5, 5, 1, 5, 5, 5, 4, 4, 5, 3, 1, 4, 2, 1, 4, 1, 4, 5, 5, 4, 1, 2, 5, 4, 1, 5, 5, 1, 1, 2, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 3, 1, 1, 5, 5, 5, 5, 1, 2, 4, 4, 5, 4, 1, 5, 5, 3, 3, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 3, 5]
accuracy_score(list2, list_best)

0.9146